*ttr, ttr_sq, ttr_hd*

In [1]:
from cltk.lemmatize.lat import LatinBackoffLemmatizer
import string
import numpy
from numpy import log as ln

def basic_TTR(sentence):
    
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Lemmatize the words
    lemmatizer = LatinBackoffLemmatizer()
    lemmas = lemmatizer.lemmatize(words)
    
    # the LatinBackoffLemmatizer gives us a tuple of the token and the lemma. Because we only need the lemma, we cut out the token.
    # and we cut out the numerals 1 to 4, which may stand after a lemma.
    lemmata = [re.sub(r'[1-4]$', '', lemma[1]) for lemma in lemmas]

    # Count unique and total lemmas
    unique_lemmas = set(lemmata)
    total_lemmas = len(lemmata)

    # Compute the original TTR
    original_ttr = round(len(unique_lemmas) / total_lemmas,3)
    
    # Compute the TTR with square root correction
    ttr_with_sqrt = round(len(unique_lemmas) / (total_lemmas ** 0.5),3)
    
    # Compute the TTR with Herdan's formula
    ttr_herdan = round(ln(len(unique_lemmas)) / ln(total_lemmas),3)
    
    # Return both values as a 3-tuple
    return original_ttr, ttr_with_sqrt, ttr_herdan

*ttrsw, ttrsw_sq, ttrsw_hd*

In [2]:
from cltk.lemmatize.lat import LatinBackoffLemmatizer
from cltk.stops.words import Stops
import string
import re
import numpy
from numpy import log as ln

def basic_TTR_cont(sentence):
    
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Lemmatize the words
    lemmatizer = LatinBackoffLemmatizer()
    lemmas = lemmatizer.lemmatize(words)
    
    # the LatinBackoffLemmatizer gives us a tuple of the token and the lemma. Because we only need the lemma, we cut out the token.
    # and we cut out the numerals 1 to 4, which may stand after a lemma.
    lemmata = [re.sub(r'[1-4]$', '', lemma[1]) for lemma in lemmas]
    
    # we remove the Latin stop words.
    stops_obj = Stops(iso_code="lat")
    tokens_filtered = stops_obj.remove_stopwords(tokens=lemmata)
    
    # Count unique and total lemmas
    unique_lemmas = set(tokens_filtered)
    total_lemmas = len(tokens_filtered)

    # Compute the original TTR
    original_ttr = round(len(unique_lemmas) / total_lemmas,3)
    
    # Compute the TTR with square root correction
    ttr_with_sqrt = round(len(unique_lemmas) / (total_lemmas ** 0.5),3)
    
    # Compute the TTR with Herdan's formula
    ttr_herdan = round(ln(len(unique_lemmas)) / ln(total_lemmas),3)
    
    # Return both values as a 3-tuple
    return original_ttr, ttr_with_sqrt, ttr_herdan

*i/f, f/i, i/w, f/w*

In [3]:
from cltk.lemmatize.lat import LatinBackoffLemmatizer
from cltk.stops.words import Stops
import string
import re
import numpy
from numpy import log as ln

def content_word_ratio(sentence):
    
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Lemmatize the words
    lemmatizer = LatinBackoffLemmatizer()
    lemmas = lemmatizer.lemmatize(words)
    
    # the LatinBackoffLemmatizer gives us a tuple of the token and the lemma. Because we only need the lemma, we cut out the token.
    # and we cut out the numerals 1 to 4, which may stand after a lemma.
    lemmata = [re.sub(r'[1-4]$', '', lemma[1]) for lemma in lemmas]
    
    # we remove the Latin stop words.
    stops_obj = Stops(iso_code="lat")
    tokens_filtered = stops_obj.remove_stopwords(tokens=lemmata)
    
    # create list of content and function words
    content_words = []
    function_words = []
    for token in lemmata:
        if token in tokens_filtered:
            content_words.append(token)
        else:
            function_words.append(token)

    # Calculate content-function word ratio
    ratio_1 = round(len(content_words) / len(function_words),3)
    ratio_2 = round(len(function_words) / len(content_words),3)
    ratio_3 = round(len(content_words) / len(lemmata),3)
    ratio_4 = round(len(function_words) / len(lemmata),3)
    
    return ratio_1, ratio_2, ratio_3, ratio_4

*R*

In [4]:
from cltk.lemmatize.lat import LatinBackoffLemmatizer
import string
import re
from collections import Counter
import math

def curve_length(sentence):
    
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Lemmatize the words
    lemmatizer = LatinBackoffLemmatizer()
    lemmas = lemmatizer.lemmatize(words)
    
    # the LatinBackoffLemmatizer gives us a tuple of the token and the lemma. Because we only need the lemma, we cut out the token.
    # and we cut out the numerals 1 to 4, which may stand after a lemma.
    lemmata = [re.sub(r'[1-4]$', '', lemma[1]) for lemma in lemmas]
    
    # we here count the frequencies of the lemmata and sort them.
    # in our frequency list are three information: frequency of lemma, rank and the 'sqrt-term' out of L
    freq_counter = Counter(lemmata)
    sorted_freq = sorted(freq_counter.items(), key=lambda x: x[1], reverse=True)
    freq_dict = {}
    for i, (lemma, freq) in enumerate(sorted_freq):
        next_freq = sorted_freq[i+1][1] if i+1 < len(sorted_freq) else 0
        freq_dict[lemma] = {'freq': freq, 'rank': i+1, 'sqrt_term': math.sqrt((freq - next_freq)**2 + 1)}
    sorted_lemma = list(freq_dict.values())[:-1]
    
    # Compute h-point
    h_point = 0
    for lemma, freq_data in freq_dict.items():
        if freq_data['rank'] <= freq_data['freq']:
            h_point = max(h_point, freq_data['rank'])
            
        
    #sum of sqrt-term, i.e curve length L
    sqrti = sum(lemma_data['sqrt_term'] for lemma_data in sorted_lemma)
    
    #sum until the h-point, i.e. curve length above h-point, L_h
    sum_sqrt_terms = sum(lemma_data['sqrt_term'] for lemma_data in sorted_lemma[:h_point])
    
    #curve length R
    R = round(1-sum_sqrt_terms/sqrti,3)
    
    return R

*nr, vr, nst*

In [5]:
import stanza
import string
from cltk.lemmatize.lat import LatinBackoffLemmatizer

stanza.download('la', processors='tokenize,pos,lemma', verbose=False)
nlp = stanza.Pipeline('la', processors='tokenize,pos,lemma')

def POS(sentence):
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()

    # Lemmatize the words
    lemmatizer = LatinBackoffLemmatizer()
    lemmas = lemmatizer.lemmatize(words)
    
    # the LatinBackoffLemmatizer gives us a tuple of the token and the lemma. Because we only need the lemma, we cut out the token.
    # and we cut out the numerals 1 to 4, which may stand after a lemma.
    lemmata = [re.sub(r'[1-4]$', '', lemma[1]) for lemma in lemmas]

    # POS tag the lemmas
    pos_tags = [word.upos for word in nlp(' '.join(lemmata)).sentences[0].words]

    # Count instances of NOUN and VERB
    noun_count = pos_tags.count('NOUN')
    verb_count = pos_tags.count('VERB')
    
    # Calculate the ratio of NOUNs and VERBs to the total number of words
    word_count = len(pos_tags)
    noun_ratio = round(noun_count / word_count,3)
    verb_ratio = round(verb_count / word_count,3)
    noun_stil = round(verb_count / noun_count,3)
    
    return noun_ratio, verb_ratio, noun_stil

*fvr, ivr, fir*

In [6]:
import stanza
import string
from cltk.lemmatize.lat import LatinBackoffLemmatizer

stanza.download('la', processors='tokenize,pos,lemma', verbose=False)
nlp = stanza.Pipeline('la', processors='tokenize,pos,lemma')

def POS_verb(sentence):
    # Remove punctuation marks from sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Split sentence into words
    words = sentence.split()
    doc = nlp(' '.join(words))

    # Extract the POS tags and adjust for finite/infinite verb forms
    pos_tags = []
    for sent in doc.sentences:
        for word in sent.words:
            if word.upos == 'VERB':
                if 'VerbForm=Fin' in word.feats:
                    pos_tags.append('FIN')
                else:
                    pos_tags.append('INF')
            else:
                pos_tags.append(word.upos)

    # Count instances of finite VERB and infinite VERB
    finite_verb_count = pos_tags.count('FIN')
    infinite_verb_count = pos_tags.count('INF')

    # Calculate the ratio of finite and infinite VERBs
    word_count = len(pos_tags)
    finite_verb_ratio = round(finite_verb_count / word_count,3)
    infinite_verb_ratio = round(infinite_verb_count / word_count,3)
    finite_inf_ratio = round(finite_verb_count / (finite_verb_count + infinite_verb_count),3)

    return finite_verb_ratio, infinite_verb_ratio, finite_inf_ratio

In [23]:
sentence = "Desine ergo philosophis pecunia interdicere: nemo sapientiam paupertate damnavit. Habebit philosophus amplas opes, sed nulli detractas nec alieno sanguine cruentas, sine cuiusquam iniuria partas, sine sordidis quaestibus, quarum tam honestus sit exitus quam introitus, quibus nemo ingemescat nisi malignus. In quantum vis, exaggera illas: honestae sunt, in quibus, cum multa sint, quae sua quisque dici velit, nihil est, quod quisquam suum possit dicere. Ille vero fortunae benignitatem a se non summovebit et patrimonio per honesta quaesito nec gloriabitur nec erubescet. Habebit tamen etiam, quo glorietur, si aperta domo et admissa in res suas civitate poterit dicere: quod quisque agnoverit, tollat. O magnum virum, o optime divitem, si post hanc vocem tantundem habuerit! Ita dico: si tuto et securus scrutationem populo praebuerit, si nihil quisquam apud illum invenerit, quod manus iniciat, audaciter et propalam erit dives. Sapiens nullum denarium intra limen suum admittet male intrantem; idem magnas opes, munus fortunae fructumque virtutis, non repudiabit nec excludet. Quid enim est, quare illis bono loco invideat? veniant, hospitentur. Nec iactabit illas nec abscondet, alterum infruniti animi est, alterum timidi et pusilli, velut magnum bonum intra sinum continentis, nec, ut dixi, eiciet illas e domo. Quid enim dicet? utrumne inutiles estis an ego uti divitiis nescio? Quemadmdum etiam pedibus suis poterit iter conficere, escendere tamen vehiculum malet, sic pauper poterit esse, dives volet. Habebit itaque opes, sed tamquam leves et avolaturas, nec ulli alii eas nec sibi graves esse patietur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.657, 10.09, 0.923)
(0.799, 9.749, 0.955)
(1.713, 0.584, 0.631, 0.369)
0.953
(0.194, 0.224, 1.152)
(0.169, 0.064, 0.727)


In [24]:
sentence = "Desine ergo philosophis pecunia interdicere: nemo sapientiam paupertate damnavit. Habebit philosophus amplas opes, sed nulli detractas nec alieno sanguine cruentas, sine cuiusquam iniuria partas, sine sordidis quaestibus, quarum tam honestus sit exitus quam, quibus nemo ingemescat nisi. In quantum vis, illas: honestae sunt, in quibus, cum multa sint, quae sua quisque dici velit, quisquam suum possit dicere. Ille vero fortunae benignitatem a se non summovebit et patrimonio per honesta quaesito nec gloriabitur nec. Habebit tamen etiam, quo glorietur, si aperta domo et admissa in res suas civitate poterit dicere: quod quisque agnoverit, tollat. O magnum virum, o optime divitem, si post hanc vocem habuerit! Ita dico: si tuto et populo praebuerit, si nihil quisquam apud illum invenerit, quod audaciter et propalam erit dives. Sapiens nullum intra limen suum admittet male intrantem; idem magnas opes, munus fortunae fructumque virtutis, non repudiabit nec excludet. Quid enim est, quare illis bono loco invideat? veniant. Nec illas nec, alterum animi est, alterum timidi et, velut magnum bonum intra sinum continentis, nec, ut dixi, eiciet illas e domo. Quid enim dicet? utrumne inutiles estis an ego uti divitiis nescio? Quemadmdum etiam pedibus suis poterit iter conficere, tamen malet, sic pauper poterit esse, dives volet. Habebit itaque opes, sed tamquam leves et, nec ulli alii eas nec sibi graves esse patietur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.637, 9.343, 0.916)
(0.777, 8.858, 0.948)
(1.529, 0.654, 0.605, 0.395)
0.949
(0.185, 0.227, 1.225)
(0.163, 0.065, 0.714)


In [25]:
sentence = "Nunc experiar, sitne aceto tibi cor acre in pectore. sequere! Quo sequar? quo ducis nunc me? Ad illam, quae tuom perdidit, pessum dedit tibi filium unice unicum. Heia, Lyde, leniter qui saeviunt, sapiunt magis. minus mirandumst, illaec aetas si quid illorum facit, quam si non faciat. feci ego istaec itidem in adulescentia. dum caveatur, praeter aequom ne quid delinquat, sine! Non sino neque equidem illum me vivo corrumpi sinam. sed tu, qui pro tam corrupto dicis causam filio, eademne erat haec disciplina tibi, cum tu adulescens eras? ante solem exorientem nisi in palaestram veneras, gymnasi praefecto haud mediocris poenas penderes. ibi cursu, luctando, hasta, disco, pugilatu, pila, saliendo sese exercebant magis quam scorto aut saviis: ibi suam aetatem extendebant, non in latebrosis locis. inde de hippodromo et palaestra ubi revenisses domum, cincticulo praecinctus in sella apud magistrum adsideres cum libro: cum legeres, si unam peccavisses syllabam, fieret corium tam maculosum, quam est nutricis pallium. Alii, Lyde, nunc sunt mores. Id equidem ego certo scio. nam olim populi prius honorem capiebat suffragio, quam magistro desinebat esse dicto oboediens; at nunc, prius quam septuennis est, si attingas eum manu, extemplo puer paedagogo tabula disrumpit caput!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.736, 10.221, 0.942)
(0.862, 9.558, 0.969)
(1.757, 0.569, 0.637, 0.363)
0.961
(0.244, 0.192, 0.787)
(0.166, 0.062, 0.727)


In [26]:
sentence = "Nunc experiar, sitne tibi cor acre in pectore. sequere! Quo sequar? quo ducis nunc me? Ad illam, quae tuom perdidit, tibi filium unice unicum. Heia, Lyde, leniter qui saeviunt, sapiunt magis. minus mirandumst, aetas si quid illorum facit, quam si non faciat. feci ego istaec in adulescentia. dum caveatur, praeter aequom ne quid sine. Non sino neque equidem illum me vivo corrumpi sinam. sed tu, qui pro tam corrupto dicis causam filio, eademne erat haec disciplina tibi, cum tu adulescens eras? ante solem nisi in veneras, haud mediocris poenas penderes. ibi cursu, luctando, saliendo sese exercebant magis quam aut: ibi suam aetatem extendebant, non in locis. inde de et ubi revenisses domum, in apud magistrum adsideres cum libro: cum legeres, si unam peccavisses fieret tam quam est nutricis Alii, Lyde, nunc sunt mores. Id equidem ego certo scio. nam olim prius honorem capiebat quam magistro desinebat esse dicto oboediens; at nunc, prius quam septuennis est, si attingas eum manu, extemplo puer paedagogo tabula caput!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.695, 8.902, 0.929)
(0.83, 8.045, 0.959)
(1.343, 0.745, 0.573, 0.427)
0.951
(0.183, 0.195, 1.067)
(0.171, 0.055, 0.757)


In [27]:
sentence = "Senatus consultis perfectis, L. Aemilius consul e curia in contionem processit orationemque talem habuit: „animadvertisse videor, Quirites, maiorem mihi sortito Macedoniam provinciam gratulationem factam quam cum aut conpulsus consulatus, aut quo die magistratum inii, neque id ob aliam causam, quam quia bello in Macedonia, quod diu trahitur, existimastis dignum maiestate populi Romani exitum per me imponi posse. Deos quoque huic favisse sorti spero eosdemque in rebus gerendis adfuturos esse. Haec partim ominari, partim sperare possum; illud adfirmare pro certo audeo me omni ope adnisurum esse, ne frustra vos hanc spem de me conceperitis. Quae ad bellum opus sunt, et senatus decrevit et, quoniam extemplo proficisci placet neque ego in mora sum, C. Licinius collega, vir egregius, aeque enixe parabit, ac si ipse id bellum gesturus esset; vos, quae scripsero senatui ac vobis rumores credulitate vestra alatis, quorum auctor nemo exstabit. Nam nunc quidem, quod volgo fieri, hoc praecipue bello, animadverti, nemo tam famae contemptor est, cuius non debilitari animus possit. In omnibus circulis atque etiam, si dis placet, in conviviis sunt, qui exercitus in Macedoniam ducant, ubi castra locanda sint, sciant, quae loca praesidiis occupanda, quando aut quo saltu intranda Macedonia, ubi horrea ponenda, qua terra mari subvehendi commeatus, quando cum hoste manus conserendae, quando quiesse sit melius."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.731, 10.539, 0.941)
(0.895, 10.319, 0.977)
(1.773, 0.564, 0.639, 0.361)
0.958
(0.293, 0.188, 0.639)
(0.101, 0.12, 0.457)


In [28]:
sentence = "Senatus consultis perfectis, L. Aemilius consul e curia in contionem processit orationemque talem habuit: „animadvertisse videor, Quirites, maiorem mihi sortito Macedoniam provinciam gratulationem factam quam cum aut conpulsus consulatus, aut quo die magistratum inii, neque id ob aliam causam, quam quia bello in Macedonia, quod diu trahitur, existimastis dignum maiestate populi Romani exitum per me imponi posse. Deos quoque huic favisse sorti spero eosdemque in rebus gerendis adfuturos esse. Haec partim, partim sperare possum; illud adfirmare pro certo audeo me omni ope adnisurum esse, ne frustra vos hanc spem de me conceperitis. Quae ad bellum opus sunt, et senatus decrevit et, quoniam extemplo proficisci placet neque ego in mora sum, C. Licinius collega, vir egregius, aeque parabit, ac si ipse id bellum gesturus esset; vos, quae scripsero senatui ac vobis rumores vestra alatis, quorum auctor nemo exstabit. Nam nunc quidem, quod volgo fieri, hoc praecipue bello, animadverti, nemo tam famae contemptor est, cuius non animus possit. In omnibus atque etiam, si dis placet, in conviviis sunt, qui exercitus in Macedoniam ducant, ubi castra locanda sint, sciant, quae loca praesidiis occupanda, quando aut quo saltu intranda Macedonia, ubi horrea ponenda, qua terra mari commeatus, quando cum hoste, quando quiesse sit melius."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.72, 10.182, 0.938)
(0.888, 9.928, 0.975)
(1.667, 0.6, 0.625, 0.375)
0.956
(0.284, 0.184, 0.649)
(0.105, 0.105, 0.5)


In [29]:
sentence = "Micipsa filius regnum solus obtinuit Mastanabale et Gulussa fratribus morbo absumptis. Is Adherbalem et Hiempsalem ex sese genuit Iugurthamque filium Mastanabalis fratris, quem Massinissa, quod ortus ex concubina erat, privatum dereliquerat, eodem cultu, quo liberos suos, domi habuit. Qui ubi primum adolevit, pollens viribus, decora facie, sed multo maxime ingenio validus, non se luxu neque inertiae corrumpendum dedit, sed, uti mos gentis illius est, equitare, iaculari; cursu cum aequalibus certare et, cum omnis gloria anteiret, omnibus tamen carus esse; ad hoc pleraque tempora in venando agere, leonem atque alias feras primus aut in primis ferire: plurimum facere, minimum ipse de se loqui. Quibus rebus Micipsa tametsi initio laetus fuerat, existimans virtutem Iugurthae regno suo gloriae fore, tamen, postquam hominem adulescentem exacta sua aetate et parvis liberis magis magisque crescere intellegit, vehementer eo negotio permotus multa cum animo suo volvebat: Terrebat eum natura mortalium avida imperi et praeceps ad explendam animi cupidinem, praeterea opportunitas suae liberorumque aetatis, quae etiam mediocris viros spe praedae transversos agit, ad hoc studia Numidarum in Iugurtham accensa, ex quibus, si talem virum dolis interfecisset, ne qua seditio aut bellum oriretur, anxius erat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.726, 9.899, 0.939)
(0.85, 9.583, 0.967)
(2.153, 0.465, 0.683, 0.317)
0.959
(0.339, 0.108, 0.317)
(0.076, 0.114, 0.4)


In [30]:
sentence = "Micipsa filius regnum solus obtinuit Mastanabale et Gulussa fratribus morbo absumptis. Is Adherbalem et Hiempsalem ex sese genuit Iugurthamque filium Mastanabalis fratris, quem Massinissa, quod ortus ex erat, privatum dereliquerat, eodem cultu, quo liberos suos, domi habuit. Qui ubi primum viribus, decora facie, sed multo maxime ingenio validus, non se neque inertiae corrumpendum dedit, sed, uti mos gentis illius est, cursu cum aequalibus certare et, cum omnis gloria anteiret, omnibus tamen carus esse; ad hoc pleraque tempora in agere, leonem atque alias feras primus aut in primis ferire: plurimum facere, minimum ipse de se loqui. Quibus rebus Micipsa tametsi initio laetus fuerat, existimans virtutem Iugurthae regno suo gloriae fore, tamen, postquam hominem adulescentem exacta sua aetate et parvis liberis magis magisque crescere intellegit, vehementer eo negotio permotus multa cum animo suo volvebat: Terrebat eum natura mortalium avida imperi et praeceps ad explendam animi cupidinem, praeterea opportunitas suae liberorumque aetatis, quae etiam mediocris viros spe praedae, ad hoc studia in Iugurtham accensa, ex quibus, si talem virum dolis interfecisset, ne qua seditio aut bellum oriretur, anxius erat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.716, 9.498, 0.935)
(0.846, 9.153, 0.965)
(1.983, 0.504, 0.665, 0.335)
0.956
(0.349, 0.097, 0.279)
(0.068, 0.102, 0.4)


In [31]:
sentence = "Dixi de quodam oratore saeculi nostri recto quidem et sano, sed parum grandi et ornato, ut opinor, apte: Nihil peccat, nisi quod nihil peccat. Debet enim orator erigi, attolli, interdum etiam effervescere, ecferri, ac saepe accedere ad praeceps; nam plerumque altis et excelsis adiacent abrupta. Tutius per plana, sed humilius et depressius iter; frequentior currentibus quam reptantibus lapsus, sed his non labentibus nulla, illis non nulla laus, etiamsi labantur. Nam ut quasdam artes, ita eloquentiam nihil magis quam ancipitia commendant. Vides qui per funem in summa nituntur, quantos soleant excitare clamores, cum iamiamque casuri videntur. Sunt enim maxime mirabilia, quae maxime insperata, maxime periculosa utque Graeci magis exprimunt. Ideo nequaquam par gubernatoris est virtus, cum placido et cum turbato mari vehitur: tunc admirante nullo inlaudatus, inglorius subit portum, at cum stridunt funes, curvatur arbor, gubernacula gemunt, tunc ille clarus et dis maris proximus. Cur haec? Quia visus es mihi in scriptis meis adnotasse quaedam ut tumida, quae ego sublimia, ut improba, quae ego audentia, ut nimia, quae ego plena arbitrabar. Plurimum autem refert, reprehendenda adnotes an insignia. Omnis enim advertit, quod eminet et exstat; sed acri intentione diiudicandum est, immodicum sit an grande, altum an enorme. Atque ut Homerum potissimum attingam: Quem tandem alterutram in partem potest fugere et totum illud. Sed opus est examine et libra, incredibilia sint haec et inania an magnifica, caelestia. Nec nunc ego me his similia aut dixisse aut posse dicere puto, sed hoc intellegi volo, laxandos esse eloquentiae frenos, nec angustissimo gyro ingeniorum impetus refringendos."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.689, 10.92, 0.933)
(0.878, 11.245, 0.974)
(1.885, 0.53, 0.653, 0.347)
0.955
(0.191, 0.183, 0.958)
(0.108, 0.12, 0.474)


In [32]:
sentence = "Dixi de quodam oratore saeculi nostri recto quidem et sano, sed parum grandi et, ut opinor, apte: Nihil peccat, nisi quod nihil peccat. Debet enim orator erigi, attolli, interdum etiam, efferri, ac saepe accedere ad praeceps; nam plerumque altis et excelsis adiacent. Tutius per plana, sed humilius et iter; frequentior currentibus quam lapsus, sed his non labentibus nulla, illis non nulla laus, etiamsi labantur. Nam ut quasdam artes, ita eloquentiam nihil magis quam ancipitia. Vides qui per funem in summa nituntur, quantos soleant excitare clamores, cum casuri videntur. Sunt enim maxime mirabilia, quae maxime, maxime periculosa utque Graeci magis exprimunt. Ideo par gubernatoris est virtus, cum placido et cum turbato mari vehitur: tunc admirante nullo inlaudatus, inglorius subit portum, at cum funes, curvatur arbor, gemunt, tunc ille clarus et dis maris proximus. Cur haec? Quia visus es mihi in scriptis meis quaedam ut tumida, quae ego sublimia, ut improba, quae ego, ut nimia, quae ego plena arbitrabar. Plurimum autem refert, reprehendenda adnotes an insignia. Omnis enim, quod et exstat; sed acri diiudicandum est, sit an grande, altum an. Atque ut Homerum potissimum attingam: Quem tandem alterutram in partem potest fugere et totum illud. Sed opus est examine et libra, incredibilia sint haec et inania an magnifica, caelestia. Nec nunc ego me his similia aut dixisse aut posse dicere puto, sed hoc intellegi volo, esse eloquentiae frenos, nec angustissimo ingeniorum impetus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.665, 10.089, 0.925)
(0.867, 10.369, 0.971)
(1.644, 0.608, 0.622, 0.378)
0.949
(0.182, 0.143, 0.786)
(0.1, 0.096, 0.511)


In [33]:
sentence = "Aulus milites mense Ianuario ex hibernis in expeditionem evocat, magnisque itineribus hieme aspera pervenit ad oppidum Suthul, ubi regis thesauri erant. Quod quamquam et saevitia temporis et opportunitate loci neque capi neque obsideri poterat, tamen cupidine caecus ob thesauros oppidi potiendi vineas agere, aggerem iacere aliaque, quae incepto usui forent, properare. At Iugurtha cognita vanitate atque imperitia legati subdole eius augere amentiam, missitare supplicantis legatos, ipse quasi vitabundus per saltuosa loca et tramites exercitum ductare. Denique Aulum perpulit, uti relicto Suthule in abditas regiones sese veluti cedentem insequeretur. Interea per homines callidos diu noctuque exercitum temptabat, centuriones ducesque turmarum, partim uti transfugerent, corrumpere, alii signo dato locum uti desererent. Quae postquam ex sententia instruit, intempesta nocte de improviso multitudine Numidarum Auli castra circumvenit. Milites Romani, perculsi tumultu insolito, arma capere alii, alii se abdere, pars territos confirmare, trepidare omnibus locis. Vis magna hostium, caelum nocte atque nubibus obscuratum, periculum anceps; postremo fugere an manere tutius foret, in incerto erat. Sed ex eo numero, quos paulo ante corruptos diximus, cohors una Ligurum cum paucis militibus transiere ad regem. Nostri foeda fuga, plerique abiectis armis, proximum collem occupaverunt."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.796, 10.852, 0.956)
(0.852, 10.404, 0.968)
(4.027, 0.248, 0.801, 0.199)
0.977
(0.364, 0.155, 0.426)
(0.08, 0.182, 0.306)


In [34]:
sentence = "Aulus milites mense Ianuario ex hibernis in expeditionem evocat, magnisque itineribus hieme aspera pervenit ad oppidum Suthul, ubi regis erant. Quod quamquam et saevitia temporis et opportunitate loci neque capi neque obsideri poterat, tamen cupidine caecus ob thesauros oppidi potiendi vineas agere, aggerem iacere aliaque, quae usui forent, properare. At Iugurtha cognita atque legati subdole eius augere legatos, ipse quasi per loca et exercitum. Denique Aulum uti relicto Suthule in abditas regiones sese veluti cedentem insequeretur. Interea per homines callidos diu noctuque exercitum temptabat, centuriones ducesque turmarum, partim uti transfugerent, corrumpere, alii signo dato locum uti desererent. Quae postquam ex sententia instruit, nocte de improviso multitudine Auli castra circumvenit. Milites Romani, perculsi tumultu arma capere alii, alii se abdere, pars territos confirmare, trepidare omnibus locis. Vis magna hostium, caelum nocte atque nubibus periculum anceps; postremo fugere an manere tutius foret, in incerto erat. Sed ex eo numero, quos paulo ante corruptos diximus, cohors una cum paucis militibus transiere ad regem. Nostri foeda fuga, plerique abiectis armis, proximum collem occupaverunt."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.781, 10.154, 0.952)
(0.841, 9.661, 0.965)
(3.568, 0.28, 0.781, 0.219)
0.974
(0.359, 0.159, 0.443)
(0.082, 0.165, 0.333)


In [35]:
sentence = "Consedere duces et vulgi stante corona surgit ad hos clipei dominus septemplicis Aiax intendensque manus agimus, pro Iuppiter! inquit ante rates causam, et mecum confertur Ulixes! at non Hectoreis dubitavit cedere flammis, quas ego sustinui, quas hac a classe fugavi. tutius est igitur fictis contendere verbis, quam pugnare manu, sed nec mihi dicere promptum nec facere est isti: quantumque ego Marte feroci inque acie valeo, tantum valet iste loquendo. nec memoranda tamen vobis mea facta, Pelasgi, esse reor: vidistis enim; sua narret Ulixes, quae sine teste gerit, quorum nox conscia sola est! Hector adest secumque deos in proelia ducit, quaque ruit, non tu tantum terreris, Ulixe, sed fortes etiam: tantum trahit ille timoris. hunc ego poscentem, cum quo concurreret, unus sustinui: sortemque meam vovistis, Achivi, et vestrae valuere preces. si quaeritis huius fortunam pugnae, non sum superatus ab illo. ecce ferunt Troes ferrumque ignesque Iovemque in Danaas classes: ubi nunc facundus Ulixes? nempe ego mille meo protexi pectore puppes, spem vestri reditus: date pro tot navibus arma! Quodsi vera licet mihi dicere, quaeritur istis quam mihi maior honos, coniunctaque gloria nostra est, atque Aiax armis, non Aiaci arma petuntur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.704, 9.674, 0.933)
(0.817, 9.176, 0.958)
(2.0, 0.5, 0.667, 0.333)
0.952
(0.279, 0.211, 0.755)
(0.148, 0.101, 0.596)


In [36]:
sentence = "Consedere duces et vulgi stante corona surgit ad hos dominus Aiax intendensque manus agimus, Iuppiter! inquit ante rates causam, et mecum confertur Ulixes! at non dubitavit cedere flammis, quas ego sustinui, quas hac a classe fugavi. tutius est igitur fictis contendere verbis, quam pugnare manu, sed nec mihi dicere  nec facere isti: quantumque ego feroci inque acie valeo, tantum valet iste loquendo. nec memoranda tamen vobis mea facta, esse reor: vidistis enim; sua narret Ulixes, quae sine teste gerit, quorum nox conscia sola est! Hector adest secumque deos in proelia ducit, quaque ruit, non tu tantum terreris, Ulixe, sed fortes etiam: tantum trahit ille timoris. hunc ego poscentem, cum quo concurreret, unus sustinui: sortemque meam et vestrae valuere preces. si quaeritis huius fortunam pugnae, non sum superatus ab illo. ecce ferunt ferrumque ignesque in classes: ubi nunc Ulixes? nempe ego mille meo protexi pectore spem vestri reditus: date pro tot navibus arma! Quodsi vera licet mihi dicere, quaeritur istis quam mihi maior honos, coniunctaque gloria nostra est, atque Aiax armis, non Aiaci arma petuntur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.695, 9.173, 0.93)
(0.805, 8.561, 0.954)
(1.852, 0.54, 0.649, 0.351)
0.947
(0.263, 0.211, 0.804)
(0.149, 0.103, 0.591)


In [37]:
sentence = "maxime Teucrorum ductor, quo sospite numquam res equidem Troiae victas aut regna fatebor, nobis ad belli auxilium pro nomine tanto exiguae vires; hinc Tusco claudimur amni, hinc Rutulus premit et murum circumsonat armis. sed tibi ego ingentis populos opulentaque regnis iungere castra paro, quam fors inopina salutem ostentat: fatis huc te poscentibus adfers. haud procul hinc saxo incolitur fundata vetusto urbis Agyllinae sedes, ubi Lydia quondam gens, bello praeclara, iugis insedit Etruscis. hanc multos florentem annos rex deinde superbo imperio et saevis tenuit Mezentius armis. quid memorem infandas caedes, quid facta tyranni effera? di capiti ipsius generique reservent! mortua quin etiam iungebat corpora vivis componens manibusque manus atque oribus ora, tormenti genus, et sanie taboque fluentis complexu in misero longa sic morte necabat. at fessi tandem cives infanda furentem armati circumsistunt ipsumque domumque, obtruncant socios, ignem ad fastigia iactant. ille inter caedem Rutulorum elapsus in agros confugere et Turni defendier hospitis armis. ergo omnis furiis surrexit Etruria iustis, regem ad supplicium praesenti Marte reposcunt. his ego te, Aenea, ductorem milibus addam. toto namque fremunt condensae litore puppes signaque ferre iubent, retinet longaevus haruspex fata canens: o Maeoniae delecta iuventus, flos veterum virtusque virum, quos iustus in hostem fert dolor et merita accendit Mezentius ira, nulli fas Italo tantam subiungere gentem: externos optate duces. tum Etrusca resedit hoc acies campo monitis exterrita divum."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.847, 12.618, 0.969)
(0.894, 12.0, 0.979)
(4.286, 0.233, 0.811, 0.189)
0.978
(0.387, 0.216, 0.558)
(0.131, 0.095, 0.58)


In [38]:
sentence = "maxime ductor, quo sospite numquam res equidem Troiae victas aut regna fatebor, nobis ad belli auxilium pro nomine tanto exiguae vires; hinc claudimur amni, hinc premit et murum armis. sed tibi ego ingentis populos opulentaque regnis iungere castra paro, quam fors salutem ostentat: fatis huc poscentibus. haud procul hinc saxo incolitur fundata vetusto urbis sedes, ubi quondam gens, bello praeclara, iugis. hanc multos florentem annos rex deinde superbo imperio et saevis tenuit Mezentius armis. quid memorem infandas caedes, quid facta tyranni effera? di capiti ipsius generique! mortua quin etiam corpora vivis manibusque manus atque oribus ora, tormenti genus, et sanie taboque fluentis complexu in misero longa sic morte necabat. at fessi tandem cives infanda furentem armati circumsistunt ipsumque domumque, obtruncant, ignem ad fastigia iactant. ille inter caedem elapsus in agros confugere et Turni defendier hospitis armis. ergo omnis surrexit iustis, regem ad supplicium praesenti Marte reposcunt. his ego te, Aenea, ductorem milibus addam. toto namque fremunt litore puppes signaque ferre iubent, retinet longaevus haruspex fata canens: o delecta iuventus, flos veterum virtusque virum, quos iustus in hostem fert dolor et merita accendit Mezentius ira, nulli fas tantam gentem: externos optate duces. tum resedit hoc acies campo monitis exterrita divum."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.844, 11.909, 0.968)
(0.892, 11.217, 0.978)
(3.854, 0.259, 0.794, 0.206)
0.975
(0.392, 0.201, 0.513)
(0.121, 0.091, 0.571)


In [39]:
sentence = "Nox erat et terris animalia somnus habebat: effigies sacrae divum Phrygiique penates, quos mecum a Troia mediisque ex ignibus urbis extuleram, visi ante oculos astare iacentis in somnis multo manifesti lumine, qua se plena per insertas fundebat luna fenestras; tum sic adfari et curas his demere dictis: quod tibi delato Ortygiam dicturus Apollo est, hic canit et tua nos en ultro ad limina mittit. nos te Dardania incensa tuaque arma secuti, nos tumidum sub te permensi classibus aequor, idem venturos tollemus in astra nepotes imperiumque urbi dabimus. tu moenia magnis magna para longumque fugae ne linque laborem. mutandae sedes. non haec tibi litora suasit Delius aut Cretae iussit considere Apollo. est locus, Hesperiam Grai cognomine dicunt, terra antiqua, potens armis atque ubere glaebae; Oenotri coluere viri; nunc fama minores Italiam dixisse ducis de nomine gentem. talibus attonitus visis et voce deorum (nec sopor illud erat, sed coram agnoscere vultus velatasque comas praesentiaque ora videbar; tum gelidus toto manabat corpore sudor) corripio e stratis corpus tendoque supinas ad caelum cum voce manus et munera libo intemerata focis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.812, 10.779, 0.96)
(0.898, 10.165, 0.978)
(2.667, 0.375, 0.727, 0.273)
0.973
(0.373, 0.141, 0.379)
(0.102, 0.102, 0.5)


In [40]:
sentence = "Nox erat et terris animalia somnus habebat: effigies sacrae divum Phrygiique penates, quos mecum a Troia mediisque ex ignibus urbis extuleram, visi ante oculos astare iacentis in somnis multo lumine, qua se plena per fundebat luna fenestras; tum sic adfari et curas his demere dictis: quod tibi dicturus Apollo est, hic canit et tua nos en ultro ad limina mittit. nos te incensa tuaque arma secuti, nos tumidum permensi classibus aequor, idem venturos tollemus in astra nepotes imperiumque urbi dabimus. tu moenia magnis magna para longumque fugae ne linque laborem. mutandae sedes. non haec tibi litora suasit Delius aut Cretae iussit Apollo. est locus, Hesperiam Grai cognomine dicunt, terra antiqua, potens armis atque ubere glaebae; coluere viri; nunc fama minores Italiam dixisse ducis de nomine gentem. talibus attonitus visis et voce deorum (nec sopor illud erat, sed coram agnoscere vultus velatasque comas praesentiaque ora videbar; tum gelidus toto manabat corpore sudor) corripio e stratis corpus tendoque supinas ad caelum cum voce manus et munera libo intemerata focis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.808, 10.447, 0.958)
(0.893, 9.818, 0.976)
(2.63, 0.38, 0.725, 0.275)
0.972
(0.375, 0.143, 0.381)
(0.108, 0.096, 0.529)


In [41]:
sentence = "At non in Pharia manes iacuere favilla nec cinis exiguus tantam conpescuit umbram; prosiluit busto semustaque membra relinquens degeneremque rogum sequitur convexa Tonantis. qua niger astriferis conectitur axibus aer, semidei manes habitant, quos ignea virtus innocuos vita patientes aetheris imi fecit. non illuc auro positi nec ture sepulti perveniunt. illic postquam se lumine vero implevit, stellasque vagas miratus et astra fixa polis, vidit quanta sub nocte iaceret nostra dies risitque sui ludibria trunci. hinc super Emathiae campos et signa cruenti Caesaris ac sparsas volitavit in aequore classes, et scelerum vindex in sancto pectore Bruti sedit et invicti posuit se mente Catonis. Ille, ubi pendebant casus dubiumque manebat quem dominum mundi facerent civilia bella, oderat et Magnum, quamvis comes isset in arma auspiciis raptus patriae ductuque senatus; at post Thessalicas clades iam pectore toto Pompeianus erat. patriam tutore carentem excepit, populi trepidantia membra refovit, ignavis manibus proiectos reddidit enses, nec regnum cupiens gessit civilia bella nec servire timens. nil causa fecit in armis ipse sua: totae post Magni funera partes libertatis erant. quas ne per litora fusas colligeret rapido victoria Caesaris actu, Corcyrae secreta petit ac mille carinis abstulit Emathiae secum fragmenta ruinae."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.812, 11.258, 0.961)
(0.902, 11.157, 0.979)
(3.923, 0.255, 0.797, 0.203)
0.969
(0.375, 0.198, 0.528)
(0.14, 0.098, 0.587)


In [42]:
sentence = "At non in manes iacuere favilla nec cinis exiguus tantam umbram; prosiluit membra relinquens degeneremque rogum sequitur Tonantis. qua niger astriferis conectitur axibus aer, semidei manes habitant, quos ignea virtus innocuos vita patientes aetheris imi fecit. non illuc auro positi nec ture sepulti perveniunt. illic postquam se lumine vero implevit, stellasque vagas miratus et astra fixa polis, vidit quanta sub nocte iaceret nostra dies risitque sui ludibria trunci. hinc super et signa cruenti Caesaris ac sparsas volitavit in aequore classes, et scelerum vindex in sancto pectore Bruti sedit et invicti posuit se mente Catonis. Ille, ubi pendebant casus dubiumque manebat quem dominum mundi facerent civilia bella, oderat et Magnum, quamvis comes isset in arma auspiciis raptus patriae ductuque senatus; at post iam pectore toto erat. patriam tutore carentem excepit, populi trepidantia membra refovit, ignavis manibus proiectos reddidit enses, nec regnum cupiens gessit civilia bella nec servire timens. nil causa fecit in armis ipse sua: totae post Magni funera partes libertatis erant. quas ne per litora fusas colligeret rapido victoria Caesaris actu, secreta petit ac mille carinis abstulit secum fragmenta."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.804, 10.763, 0.958)
(0.9, 10.649, 0.979)
(3.59, 0.279, 0.782, 0.218)
0.967
(0.356, 0.206, 0.578)
(0.15, 0.111, 0.574)


In [43]:
sentence = "Primum facinus novi principatus fuit Postumi Agrippae caedes, quem ignarum inerumumque quamvis firmatus animo centurio aegre confecit. nihil de ea re Tiberius apud senatum disseruit: patris iussa simulabat, quibus praescripsisset tribuno custodiae adposito, ne cunctaretur Agrippam morte adficere, quandoque ipse supremum diem explevisset. multa sine dubio saevaque Augustus de moribus adulescentis questus, ut exilium eius senatus consulto sanciretur perfecerat: ceterum in nullius umquam suorum necem duravit, neque mortem nepoti pro securitate privigni inlatam credibile erat. propius vero Tiberium ac Liviam, illum metu, hanc novercalibus odiis, suspecti et invisi iuvenis caedem festinavisse. nuntianti centurioni, ut mos militiae, factum esse quod imperasset, neque imperasse sese et rationem facti reddendam apud senatum respondit. quod postquam Sallustius Crispus particeps secretorum, is ad tribunum miserat codicillos, comperit, metuens ne reus subderetur, iuxta periculoso ficta seu vera promeret, monuit Liviam ne arcana domus, ne consilia amicorum, ministeria militum vulgarentur, neve Tiberius vim principatus resolveret cunta ad senatum vocando: eam condicionem esse imperandi, ut non aliter ratio constet quam si uni reddatur. At Romae ruere in servitium consules, patres, eques. quanto quis inlustrior, tanto magis falsi ac festinantes, vultuque composito, ne laeti excessu principis neu tristiores primordio, lacrimas gaudium, questus adulationem miscebant."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.774, 10.813, 0.952)
(0.852, 10.404, 0.968)
(3.239, 0.309, 0.764, 0.236)
0.971
(0.342, 0.153, 0.448)
(0.108, 0.097, 0.525)


In [44]:
sentence = "Primum facinus novi principatus fuit Postumi Agrippae caedes, quem ignarum inerumumque quamvis firmatus animo centurio aegre confecit. nihil de ea re Tiberius apud senatum disseruit: patris iussa simulabat, quibus praescripsisset tribuno custodiae adposito, ne cunctaretur Agrippam morte adficere, quandoque ipse supremum diem explevisset. multa sine dubio saevaque Augustus de moribus adulescentis questus, ut exilium eius senatus consulto sanciretur perfecerat: ceterum in nullius umquam suorum necem, neque mortem nepoti pro securitate privigni inlatam credibile erat. propius vero Tiberium ac Liviam, illum metu, hanc novercalibus odiis, suspecti et invisi iuvenis caedem festinavisse. nuntianti centurioni, ut mos militiae, factum esse quod imperasset, neque imperasse sese et rationem facti reddendam apud senatum respondit. quod postquam Sallustius Crispus particeps secretorum, is ad tribunum miserat codicillos, comperit, metuens ne reus subderetur, iuxta periculoso ficta seu vera promeret, monuit Liviam ne arcana domus, ne consilia amicorum, ministeria militum vulgarentur, neve Tiberius vim principatus resolveret cunta ad senatum vocando: eam condicionem esse imperandi, ut non aliter ratio constet quam si uni reddatur. At Romae ruere in servitium consules, patres, eques. quanto quis inlustrior, tanto magis falsi ac festinantes, vultuque composito, ne laeti excessu principis neu tristiores primordio, lacrimas gaudium, questus adulationem miscebant."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.773, 10.769, 0.951)
(0.851, 10.357, 0.968)
(3.217, 0.311, 0.763, 0.237)
0.971
(0.344, 0.154, 0.448)
(0.104, 0.098, 0.513)


In [45]:
sentence = "Perseus ad Pieriam silvam via militari frequenti agmine equitum et regio comitatu fugit. Simul in silvam ventum est, ubi plures diversae semitae erant, et nox adpropinquabat, cum perpaucis maxime fidis via devertit. Equites sine duce relicti alii alia in civitates suas dilapsi sunt; perpauci inde Pellam celerius quam ipse Perseus, quia recta et expedita via ierant, pervenerunt. Rex ad mediam ferme noctem errore et variis difficultatibus viae vexatus est; in regia Perseo, qui Pellae praeerant, Euctus Eulaeusque et regii pueri praesto erant. Contra ea amicorum, qui alii alio casu servati ex proelio Pellam venerant, cum saepe arcessiti essent, nemo ad eum venit. Tres erant tantum cum eo fugae comites, Euander Cretensis, Neo Boeotus, et Archidamus Aetolus. Cum iis, iam metuens, ne, qui venire ad se abnuerent, maius aliquid mox auderent, quarta vigilia profugit. Secuti eum sunt admodum quingenti Cretenses. Petebat Amphipolim; sed nocte a Pella exierat, properans ante lucem Axium amnem traicere, eum finem sequendi propter difficultatem transitus fore ratus Romanis. Consulem, cum se in castra victor recepisset, ne sincero gaudio frueretur, cura de minore filio stimulabat. P. Scipio is erat, Africanus et ipse postea deleta Carthagine appellatus, naturalis consulis Paulli filius, adoptione Africani nepos."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.692, 9.668, 0.93)
(0.819, 9.619, 0.959)
(2.421, 0.413, 0.708, 0.292)
0.94
(0.308, 0.133, 0.433)
(0.082, 0.088, 0.485)


In [46]:
sentence = "Perseus ad silvam via militari frequenti agmine equitum et regio comitatu fugit. Simul in silvam ventum est, ubi plures diversae semitae erant, et nox adpropinquabat, cum perpaucis maxime fidis via. Equites sine duce relicti alii alia in civitates suas dilapsi sunt; perpauci inde celerius quam ipse Perseus, quia recta et expedita via ierant, pervenerunt. Rex ad mediam ferme noctem errore et variis difficultatibus viae vexatus est; in regia Perseo, qui Pellae praeerant, Euctus Eulaeusque et regii praesto erant. Contra ea amicorum, qui alii alio casu servati ex proelio Pellam venerant, cum saepe arcessiti essent, nemo ad eum venit. Tres erant tantum cum eo fugae comites, Euander, Neo, et Archidamus. Cum iis, iam metuens, ne, qui venire ad se abnuerent, maius aliquid mox auderent, quarta vigilia profugit. Secuti eum sunt admodum quingenti Cretenses. Petebat; sed nocte a Pella exierat, properans ante lucem amnem traicere, eum finem sequendi propter difficultatem transitus fore ratus Romanis. Consulem, cum se in castra victor recepisset, ne gaudio frueretur, cura de minore filio. P. Scipio is erat, Africanus et ipse postea deleta Carthagine appellatus, naturalis consulis Paulli filius, adoptione Africani nepos."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.685, 9.289, 0.927)
(0.819, 9.229, 0.959)
(2.228, 0.449, 0.69, 0.31)
0.936
(0.321, 0.125, 0.39)
(0.082, 0.087, 0.484)


In [47]:
sentence = "Quaeret quispiam: Quid? Illi ipsi summi viri, quorum virtutes litteris proditae sunt, istane doctrina, quam tu effers laudibus, eruditi fuerunt? Difficile est hoc de omnibus confirmare, sed tamen est certe, quod respondeam: Ego multos homines excellenti animo ac virtute fuisse et sine doctrina naturae ipsius habitu prope divino per se ipsos et moderatos et gravis exstitisse fateor. etiam illud adiungo saepius ad laudem atque virtutem naturam sine doctrina quam sine natura valuisse doctrinam. Atque idem ego contendo, cum ad naturam eximiam atque illustrem accesserit ratio quaedam conformatioque doctrinae, tum illud nescio quid praeclarum ac singulare solere exsistere. Ex hoc esse hunc numero, quem patres nostri viderunt, divinum hominem Africanum; ex hoc fortissimum virum et illis temporibus doctissimum, M. Catonem illum senem: qui profecto si nihil ad percipiendam virtutem litteris adiuvarentur, numquam se ad earum studium contulissent. Quodsi non his tantus fructus ostenderetur et si ex his studiis delectatio sola peteretur, tamen, ut opinor, hanc animi adversionem humanissimam ac liberalissimam iudicaretis. Nam ceterae neque temporum sunt neque aetatum omnium neque locorum; haec studia adulescentiam alunt, senectutem oblectant, secundas res ornant, adversis perfugium ac solacium praebent, delectant domi, non impediunt foris. Quodsi ipsi haec neque attingere neque sensu nostro gustare possemus, tamen ea mirari deberemus, etiam cum in aliis videremus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.644, 9.291, 0.918)
(0.808, 9.209, 0.956)
(1.667, 0.6, 0.625, 0.375)
0.948
(0.25, 0.151, 0.604)
(0.12, 0.077, 0.61)


In [48]:
sentence = "Quaeret quispiam: Quid? Illi ipsi summi viri, quorum virtutes litteris proditae sunt, istane doctrina, quam tu effers laudibus, eruditi fuerunt? Difficile est hoc de omnibus confirmare, sed tamen est certe, quod respondeam: Ego multos homines excellenti animo ac virtute fuisse et sine doctrina naturae ipsius habitu prope divino per se ipsos et moderatos et gravis exstitisse fateor. etiam illud adiungo saepius ad laudem atque virtutem naturam sine doctrina quam sine natura valuisse doctrinam. Atque contendo, cum ad naturam atque illustrem accesserit ratio quaedam doctrinae, tum illud praeclarum ac singulare solere exsistere. Ex hoc esse hunc numero, quem patres nostri viderunt, divinum hominem; ex hoc fortissimum virum et illis temporibus doctissimum, illum senem: qui profecto si nihil ad percipiendam virtutem litteris adiuvarentur, numquam se ad earum studium contulissent. Quodsi non his tantus fructus ostenderetur et si ex his studiis sola peteretur, tamen, ut opinor, hanc humanissimam ac liberalissimam iudicaretis. Nam ceterae neque temporum sunt neque aetatum omnium neque locorum; haec studia adulescentiam alunt, senectutem, secundas res ornant, adversis ac praebent, delectant domi, non impediunt foris. Quodsi ipsi haec neque attingere neque sensu nostro possemus, tamen ea mirari deberemus, etiam cum in aliis videremus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.62, 8.588, 0.909)
(0.795, 8.598, 0.952)
(1.56, 0.641, 0.609, 0.391)
0.942
(0.231, 0.149, 0.644)
(0.12, 0.078, 0.605)


In [49]:
sentence = "Tarpeium nemus et Tarpeiae turpe sepulcrum fabor et antiqui limina capta Iouis. lucus erat felix hederoso conditus antro, multaque nativis obstrepit arbor aquis, Silvani ramosa domus, quo dulcis ab aestu fistula poturas ire iubebat ovis. hunc Tatius fontem vallo praecingit acerno, fidaque suggesta castra coronat humo. quid tum Roma fuit, tubicen vicina Curetis cum quateret lento murmure saxa Iovis? atque ubi nunc terris dicuntur iura subactis, stabant Romano pila Sabina Foro. murus erant montes: ubi nunc est curia saepta, bellicus ex illo fonte bibebat equus. hinc Tarpeia deae fontem libavit: at illi urgebat medium fictilis urna caput. vidit harenosis Tatium proludere campis pictaque per flavas arma leuare iubas: obstipuit regis facie et regalibus armis, interque oblitas excidit urna manus. annua pastorum convivia, lusus in urbe, cum pagana madent fercula divitiis, Romulus excubias decrevit in otia solvi atque intermissa castra silere tuba. hoc Tarpeia suum tempus rata convenit hostem: pacta ligat, pactis ipsa futura comes. omnia praebebant somnos: sed Iuppiter unus decrevit poenis invigilare suis. prodiderat portaeque fidem patriamque iacentem, nubendique petit, quem velit, ipsa diem. at Tatius, neque enim sceleri dedit hostis honorem, nube ait et regni scande cubile mei! dixit, et ingestis comitum super obruit armis. haec, virgo, officiis dos erat apta tuis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.819, 11.692, 0.962)
(0.888, 11.27, 0.977)
(3.744, 0.267, 0.789, 0.211)
0.972
(0.42, 0.161, 0.384)
(0.143, 0.099, 0.592)


In [50]:
sentence = "nemus et turpe sepulcrum fabor et antiqui limina capta Iouis. lucus erat felix hederoso conditus antro, multaque obstrepit arbor aquis, ramosa domus, quo dulcis ab aestu fistula poturas ire iubebat ovis. hunc Tatius fontem vallo praecingit, fidaque castra coronat humo. quid tum Roma fuit, vicina cum quateret lento murmure saxa Iovis? atque ubi nunc terris dicuntur iura subactis, stabant Romano pila Sabina Foro. murus erant montes: ubi nunc est curia saepta, bellicus ex illo fonte bibebat equus. hinc Tarpeia deae fontem libavit: at illi urgebat medium urna caput. vidit harenosis Tatium campis pictaque per flavas arma leuare iubas: obstipuit regis facie et regalibus armis, interque oblitas excidit urna manus. annua pastorum convivia, lusus in urbe, cum madent divitiis, Romulus excubias decrevit in otia solvi atque intermissa castra silere tuba. hoc Tarpeia suum tempus rata convenit hostem: pacta ligat, pactis ipsa futura comes. omnia praebebant somnos: sed Iuppiter unus decrevit poenis invigilare suis. prodiderat portaeque fidem patriamque iacentem, nubendique petit, quem velit, ipsa diem. at Tatius, neque enim sceleri dedit hostis honorem, nube ait et regni scande cubile mei! dixit, et ingestis comitum super obruit armis. haec, virgo, officiis dos erat apta tuis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.812, 11.258, 0.961)
(0.886, 10.814, 0.976)
(3.465, 0.289, 0.776, 0.224)
0.97
(0.425, 0.161, 0.378)
(0.146, 0.083, 0.636)


In [51]:
sentence = "Cadmus agit grates peregrinaeque oscula terrae figit et ignotos montes agrosque salutat. Sacra Iovi facturus erat: iubet ire ministros et petere e vivis libandas fontibus undas. silva vetus stabat nulla violata securi, et specus in media virgis ac vimine densus efficiens humilem lapidum conpagibus arcum uberibus fecundus aquis; ubi conditus antro Martius anguis erat, cristis praesignis et auro; igne micant oculi, corpus tumet omne venenis, tres vibrant linguae, triplici stant ordine dentes. quem postquam Tyria lucum de gente profecti infausto tetigere gradu, demissaque in undas urna dedit sonitum, longo caput extulit antro caeruleus serpens horrendaque sibila misit. effluxere urnae manibus sanguisque reliquit corpus et attonitos subitus tremor occupat artus. ille volubilibus squamosos nexibus orbes torquet et inmensos saltu sinuatur in arcus ac media plus parte leves erectus in auras despicit omne nemus tantoque est corpore, quanto, si totum spectes, geminas qui separat arctos. nec mora, Phoenicas, sive illi tela parabant sive fugam, sive ipse timor prohibebat utrumque, occupat: hos morsu, longis conplexibus illos, hos necat adflati funesta tabe veneni."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.828, 10.769, 0.963)
(0.914, 10.772, 0.982)
(4.633, 0.216, 0.822, 0.178)
0.967
(0.402, 0.142, 0.353)
(0.136, 0.089, 0.605)


In [52]:
sentence = "Cadmus agit peregrinaeque oscula terrae figit et ignotos montes agrosque salutat. Sacra Iovi facturus erat: iubet ire ministros et petere e vivis libandas fontibus undas. silva vetus stabat nulla violata securi, et specus in media virgis ac densus humilem lapidum arcum uberibus fecundus aquis; ubi conditus antro anguis erat, cristis praesignis et auro; igne micant oculi, corpus tumet omne venenis, tres vibrant linguae, triplici stant ordine dentes. quem postquam Tyria lucum de gente profecti infausto tetigere gradu, demissaque in undas dedit sonitum, longo caput extulit antro caeruleus serpens horrendaque sibila misit. effluxere urnae manibus sanguisque reliquit corpus et attonitos subitus tremor occupat artus. ille squamosos orbes torquet et inmensos saltu in arcus ac media plus parte leves erectus in auras despicit omne nemus tantoque est corpore, quanto, si totum spectes, geminas qui separat. nec mora, Phoenicas, sive illi tela parabant sive fugam, sive ipse timor prohibebat utrumque, occupat: hos morsu, longis conplexibus illos, hos necat adflati funesta tabe veneni."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.824, 10.389, 0.962)
(0.915, 10.389, 0.982)
(4.3, 0.233, 0.811, 0.189)
0.965
(0.384, 0.145, 0.377)
(0.138, 0.094, 0.595)


In [53]:
sentence = "Tarquinius iuvenis socios dapibusque meroque accipit; ex illis rege creatus ait: ecquid in officio torus est socialis? et ecquid coniugibus nostris mutua cura sumus? quisque suam laudat: studiis certamina crescunt, et fervet multo linguaque corque mero. surgit, cui dederat clarum Collatia nomen: non opus est verbis, credite rebus ait. nox superest: tollamur equis Urbemque petamus! dicta placent, frenis impediuntur equi. pertulerant dominos. regalia protinus illi tecta petunt: custos in fore nullus erat. ecce nurum regis fusis per colla coronis inveniunt posito pervigilare mero. inde cito passu petitur Lucretia, cuius ante torum calathi lanaque mollis erat. lumen ad exiguum famulae data pensa trahebant; inter quas tenui sic ait illa sono: mittenda est domino, nunc, nunc properate, puellae! quam primum nostra facta lacerna manu. quid tamen auditis, nam plura audire potestis? quantum de bello dicitur esse super? mens abit et morior, quotiens pugnantis imago me subit, et gelidum pectora frigus habet. desinit in lacrimas inceptaque fila remisit, in gremio voltum deposuitque suum. hoc ipsum decuit: lacrimae decuere pudicam, et facies animo dignaque parque fuit. pone metum, veni coniunx ait; illa revixit, deque viri collo dulce pependit onus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.773, 10.514, 0.951)
(0.856, 10.093, 0.969)
(3.022, 0.331, 0.751, 0.249)
0.949
(0.308, 0.238, 0.772)
(0.199, 0.075, 0.725)


In [54]:
sentence = "Tarquinius iuvenis socios dapibusque meroque accipit; ex illis rege creatus ait: in officio est et coniugibus nostris cura sumus? quisque suam laudat: studiis certamina crescunt, et fervet multo linguaque corque mero. surgit, cui dederat clarum nomen: non opus est verbis, credite rebus ait. nox superest: tollamur equis petamus! dicta placent, frenis equi. pertulerant dominos. protinus illi tecta petunt: custos in fore nullus erat. ecce regis fusis per colla coronis inveniunt posito pervigilare mero. inde cito passu petitur Lucretia, cuius ante lanaque mollis erat. lumen ad exiguum data inter quas tenui sic ait illa sono: mittenda est domino, nunc, nunc properate, puellae, nostra facta manu. quid tamen auditis, nam plura audire potestis? quantum de bello dicitur mens abit et morior, quotiens pugnantis imago me subit, et gelidum pectora frigus habet. desinit in lacrimas inceptaque remisit, in voltum deposuitque suum. hoc ipsum decuit: lacrimae decuere pudicam, et facies animo dignaque parque fuit. metum, veni coniunx ait; illa deque viri collo dulce pependit onus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.764, 9.694, 0.947)
(0.856, 9.298, 0.967)
(2.744, 0.364, 0.733, 0.267)
0.948
(0.28, 0.267, 0.956)
(0.211, 0.087, 0.708)


In [55]:
sentence = "Grave vulnus Macrinus noster accepit: amisit uxorem singularis exempli, etiam si olim fuisset. Vixit cum hac triginta novem annis sine iurgio, sine offensa. Quam illa reverentiam marito suo praestitit, cum ipsa summam mereretur! quot quantasque virtutes, ex diversis aetatibus sumptas, collegit et miscuit! Habet quidem Macrinus grande solacium, quod tantum bonum tam diu tenuit; sed hinc magis exacerbatur, quod amisit; nam fruendis voluptatibus crescit carendi dolor. Vale. Tristem et acerbum casum Helvidiarum sororum! Utraque a partu, utraque filiam enixa decessit. Afficior dolore, nec tamen supra modum doleo: ita mihi luctuosum videtur, quod puellas honestissimas in flore primo fecunditas abstulit. Angor infantium sorte, quae sunt parentibus statim et, dum nascuntur, orbatae, angor optimorum maritorum, angor etiam meo nomine. Nam patrem illarum defunctum quoque perseverantissime diligo, ut actione mea librisque testatum est; cui nunc unus ex tribus liberis superest, domumque pluribus adminiculis paulo ante fundatam desolatus sustinet. Magno tamen fomento dolor meus acquiescit, si hunc saltem fortem et incolumem, paremque illi patri, illi avo fortuna servaverit. Cuius ego pro salute, pro moribus hoc sum magis anxius, quod unicus factus est. Nosti in amore mollitiam animi mei, nosti metus; quo minus te mirari oportebit, quod plurimum timeam, de quo plurimum spero. Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.789, 11.129, 0.955)
(0.901, 10.695, 0.979)
(2.431, 0.411, 0.709, 0.291)
0.964
(0.26, 0.16, 0.615)
(0.156, 0.055, 0.738)


In [56]:
sentence = "Grave vulnus Macrinus noster accepit: amisit uxorem singularis exempli, etiam si olim fuisset. Vixit cum hac triginta novem annis sine iurgio, sine. Quam illa marito suo praestitit, cum ipsa summam mereretur! quot quantasque virtutes, ex diversis aetatibus sumptas, collegit et miscuit! Habet quidem Macrinus grande solacium, quod tantum bonum tam diu tenuit; sed hinc magis, quod amisit; nam fruendis voluptatibus crescit carendi dolor. Vale. Tristem et acerbum casum sororum! Utraque a partu, utraque filiam enixa decessit. Afficior dolore, nec tamen supra modum doleo: ita mihi videtur, quod puellas honestissimas in flore primo abstulit. Angor infantium sorte, quae sunt parentibus statim et, dum nascuntur, orbatae, angor optimorum maritorum, angor etiam meo nomine. Nam patrem illarum quoque diligo, ut actione mea librisque testatum est; cui nunc unus ex tribus liberis superest, domumque pluribus paulo ante sustinet. Magno tamen dolor meus, si hunc saltem fortem et incolumem, paremque illi patri, illi avo fortuna servaverit. Cuius ego pro salute, pro moribus hoc sum magis anxius, quod unicus factus est. Nosti in amore mollitiam animi mei, nosti metus; quo minus te mirari oportebit, quod plurimum timeam, de quo plurimum spero. Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.774, 10.559, 0.951)
(0.891, 10.076, 0.976)
(2.207, 0.453, 0.688, 0.312)
0.961
(0.253, 0.167, 0.66)
(0.156, 0.043, 0.784)


In [57]:
sentence = "Flebilis indignos, Elegia, solve capillos! A, nimis ex vero nunc tibi nomen erit! Ille tui vates operis, tua fama, Tibullus ardet in exstructo, corpus inane, rogo. Ecce, puer Veneris fert eversamque pharetram et fractos arcus et sine luce facem; adspice, demissis ut eat miserabilis alis pectoraque infesta tundat aperta manu! Excipiunt lacrimas sparsi per colla capilli, oraque singultu concutiente sonant. Fratris in Aeneae sic illum funere dicunt egressum tectis, pulcher Iule, tuis; at sacri vates et divum cura vocamur; sunt etiam, qui nos numen habere putent. Scilicet omne sacrum mors inportuna profanat, omnibus obscuras inicit illa manus! Cum rapiunt mala fata bonos, ignoscite fasso!, sollicitor nullos esse putare deos. Vive pius, moriere; pius cole sacra, colentem mors gravis a templis in cava busta trahet; carminibus confide bonis — iacet, ecce, Tibullus: vix manet e toto, parva quod urna capit! Tene, sacer vates, flammae rapuere rogales pectoribus pasci nec timuere tuis? Avertit vultus, Erycis quae possidet arces; sunt quoque, qui lacrimas continuisse negant. Si tamen e nobis aliquid nisi nomen et umbra restat, in Elysia valle Tibullus erit. Obvius huic venias hedera iuvenalia cinctus tempora cum Calvo, docte Catulle, tuo. His comes umbra tua est; siqua est modo corporis umbra, auxisti numeros, culte Tibulle, pios. Ossa quieta, precor, tuta requiescite in urna, et sit humus cineri non onerosa tuo!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.761, 11.243, 0.949)
(0.818, 10.854, 0.961)
(4.19, 0.239, 0.807, 0.193)
0.964
(0.33, 0.202, 0.611)
(0.174, 0.073, 0.704)


In [58]:
sentence = "solve capillos! A, nimis ex vero nunc tibi nomen erit! Ille tui vates operis, tua fama, Tibullus ardet in exstructo, corpus inane, rogo. Ecce, puer Veneris fert eversamque pharetram et fractos arcus et sine luce facem; adspice, demissis ut eat miserabilis alis pectoraque infesta tundat manu! Excipiunt lacrimas sparsi per colla capilli, oraque concutiente sonant. Fratris in Aeneae sic illum funere dicunt egressum tectis, pulcher tuis; at sacri vates et divum cura vocamur; sunt etiam, qui nos numen habere putent. Scilicet omne sacrum mors omnibus obscuras inicit illa manus! Cum rapiunt mala fata bonos ignoscite fasso! sollicitor nullos esse putare deos. Vive pius moriere; pius cole sacra colentem mors gravis a templis in cava trahet; carminibus confide bonis iacet, ecce, Tibullus: vix manet e toto, parva quod capit! Tene, sacer vates, flammae rapuere pectoribus pasci nec timuere tuis? Avertit vultus, quae possidet arces; sunt quoque, qui lacrimas continuisse negant. Si tamen e nobis aliquid nisi nomen et umbra restat, in valle Tibullus erit. Obvius huic venias hedera iuvenalia cinctus tempora cum Calvo, docte Catulle, tuo. His comes umbra tua est; siqua est modo corporis umbra, auxisti numeros, culte Tibulle, pios. Ossa quieta, precor, tuta requiescite in urna, et sit humus cineri non onerosa tuo!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.75, 10.712, 0.946)
(0.809, 10.292, 0.958)
(3.857, 0.259, 0.794, 0.206)
0.961
(0.328, 0.211, 0.642)
(0.181, 0.074, 0.712)


In [59]:
sentence = "Consul ubi ea parari cognovit, dispositis praesidiis, ut res atque tempus monebat, convocato senatu refert, quid de iis fieri placeat, qui in custodiam traditi erant. Sed eos paulo ante frequens senatus iudicaverat contra rem publicam fecisse. Tum D. Iunius Silanus primus sententiam rogatus, quod eo tempore consul designatus erat, de iis, qui in custodiis tenebantur, et praeterea de L. Cassio, P. Furio, P. Umbreno, Q. Annio, si deprehensi forent, supplicium sumundum decreverat; isque postea permotus oratione C. Caesaris pedibus in sententiam Ti. Neronis iturum se dixit, qui de ea re praesidiis additis referundum censuerat. Sed Caesar, ubi ad eum ventum est, rogatus sententiam a consule huiusce modi verba locutus est: Omnis homines, patres conscripti, qui de rebus dubiis consultant, ab odio, amicitia, ira atque misericordia vacuos esse decet. Haud facile animus verum providet, ubi illa officiunt, neque quisquam omnium lubidini simul et usui paruit. Ubi intenderis ingenium, valet; si lubido possidet, ea dominatur, animus nihil valet."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.705, 8.807, 0.931)
(0.833, 8.66, 0.961)
(2.25, 0.444, 0.692, 0.308)
0.946
(0.314, 0.199, 0.633)
(0.122, 0.109, 0.528)


In [60]:
sentence = "Consul ubi ea parari cognovit, dispositis praesidiis, ut res atque tempus monebat, convocato senatu refert, quid de iis fieri placeat, qui in custodiam traditi erant. Sed eos paulo ante frequens senatus iudicaverat contra rem publicam fecisse. Tum D. Iunius Silanus primus sententiam rogatus, quod eo tempore consul designatus erat, de iis, qui in custodiis tenebantur, et praeterea de L. Cassio, P. Furio, P. Umbreno, Q. Annio, si deprehensi forent, decreverat; isque postea permotus oratione C. Caesaris Ti. Neronis se dixit, qui de ea re praesidiis additis referundum censuerat. Sed Caesar, ubi ad eum ventum est, rogatus sententiam a consule huiusce modi verba locutus est: Omnis homines, patres conscripti, qui de rebus dubiis consultant, ab odio, amicitia, ira atque misericordia vacuos esse decet. Haud facile animus verum providet, ubi illa, neque quisquam omnium lubidini simul et usui paruit. Ubi intenderis ingenium, valet; si lubido possidet, ea dominatur, animus nihil valet."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.711, 8.684, 0.932)
(0.843, 8.515, 0.963)
(2.17, 0.461, 0.685, 0.315)
0.944
(0.295, 0.201, 0.682)
(0.121, 0.107, 0.529)


In [61]:
sentence = "prominet in pontum cuneatus acumine longo collis (utrumque latus circumfluit aequoris unda): huc ferus adscendit Cyclops mediusque resedit; lanigerae pecudes nullo ducente secutae. cui postquam pinus, baculi quae praebuit usum, ante pedes posita est antemnis apta ferendis sumptaque harundinibus conpacta est fistula centum, senserunt toti pastoria sibila montes, senserunt undae; latitans ego rupe meique Acidis in gremio residens procul auribus hausi talia dicta meis auditaque mente notavi: Candidior folio nivei Galatea ligustri, floridior pratis, longa procerior alno, splendidior vitro, tenero lascivior haedo, levior adsiduo detritis aequore conchis, solibus hibernis, aestiva gratior umbra. sunt mihi, pars montis, vivo pendentia saxo antra, quibus nec sol medio sentitur in aestu, nec sentitur hiems; sunt poma gravantia ramos, sunt auro similes longis in vitibus uvae, sunt et purpureae: tibi et has servamus et illas. nec tibi castaneae me coniuge, nec tibi deerunt arbutei fetus: omnis tibi serviet arbor. Iam modo caeruleo nitidum caput exsere ponto, iam, Galatea, veni, nec munera despice nostra! certe ego me novi liquidaeque in imagine vidi nuper aquae, placuitque mihi mea forma videnti. adspice, sim quantus: non est hoc corpore maior Iuppiter in caelo, nam vos narrare soletis nescio quem regnare Iovem."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.76, 10.537, 0.948)
(0.898, 10.887, 0.978)
(3.267, 0.306, 0.766, 0.234)
0.952
(0.335, 0.155, 0.462)
(0.109, 0.094, 0.538)


In [62]:
sentence = "in pontum acumine longo collis (utrumque latus aequoris unda): huc ferus adscendit Cyclops mediusque resedit; lanigerae pecudes nullo ducente secutae. cui postquam pinus, baculi quae praebuit usum, ante pedes posita est apta  sumptaque harundinibus est fistula centum, senserunt toti montes, senserunt undae; ego rupe meique in gremio residens procul auribus hausi talia dicta meis auditaque mente notavi: Candidior folio nivei Galatea, pratis, longa alno, splendidior, tenero lascivior haedo, levior adsiduo aequore conchis, solibus, aestiva gratior umbra. sunt mihi, pars montis, pendentia saxo, quibus nec sol medio sentitur in aestu, nec sentitur hiems; sunt poma gravantia ramos, sunt auro similes longis in vitibus uvae, sunt et purpureae: tibi et has servamus et illas. nec tibi castaneae me coniuge, nec tibi deerunt: omnis tibi serviet arbor. Iam modo caeruleo nitidum caput ponto, iam, Galatea, veni, nec munera despice nostra! certe ego me novi liquidaeque in imagine vidi nuper aquae, placuitque mihi mea forma videnti. adspice, sim quantus: non est hoc corpore maior Iuppiter in caelo, nam vos narrare soletis nescio quem regnare Iovem."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.731, 9.559, 0.939)
(0.881, 9.889, 0.974)
(2.8, 0.357, 0.737, 0.263)
0.945
(0.341, 0.145, 0.424)
(0.1, 0.076, 0.567)


In [63]:
sentence = "Quamquam huiusce rei potestas omnis in vobis sita est, iudices; totam rem publicam vos in hac causa tenetis, vos gubernatis. Si L. Catilina cum suo consilio nefariorum hominum quos secum eduxit hac de re posset iudicare, condemnaret L. Murenam, si interficere posset, occideret. Petunt enim rationes illius ut orbetur auxilio res publica, ut minuatur contra suum furorem imperatorum copia, ut maior facultas tribunis plebis detur depulso adversario seditionis ac discordiae concitandae. Idemne igitur delecti ex amplissimis ordinibus honestissimi atque sapientissimi viri iudicabunt quod ille importunissimus gladiator, hostis rei publicae iudicaret? Mihi credite, iudices, in hac causa non solum de L. Murenae verum etiam de vestra salute sententiam feretis. In discrimen extremum venimus; nihil est iam unde nos reficiamus aut ubi lapsi resistamus. Non solum minuenda non sunt auxilia quae habemus sed etiam nova, si fieri possit, comparanda. Hostis est enim non apud Anienem, quod bello Punico gravissimum visum est, sed in urbe, in foro; di immortales! sine gemitu hoc dici non potest, non nemo etiam in illo sacrario rei publicae, in ipsa, inquam, curia non nemo hostis est. Di faxint ut meus conlega, vir fortissimus, hoc Catilinae nefarium latrocinium armatus opprimat! ego togatus vobis bonisque omnibus adiutoribus hoc quod conceptum res publica periculum parturit consilio discutiam et comprimam."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.654, 9.43, 0.92)
(0.805, 9.278, 0.956)
(1.773, 0.564, 0.639, 0.361)
0.958
(0.286, 0.138, 0.483)
(0.12, 0.077, 0.61)


In [64]:
sentence = "Quamquam huiusce rei potestas omnis in vobis sita est, iudices; totam rem publicam vos in hac causa tenetis, vos gubernatis. Si L. Catilina cum suo consilio nefariorum hominum quos secum eduxit hac de re posset iudicare, condemnaret L. Murenam, si interficere posset, occideret. Petunt enim rationes illius ut orbetur auxilio res publica, ut minuatur contra suum furorem imperatorum copia, ut maior facultas tribunis plebis detur depulso adversario seditionis ac discordiae concitandae. Idemne igitur delecti ex amplissimis ordinibus honestissimi atque sapientissimi viri iudicabunt quod ille importunissimus gladiator, hostis rei publicae iudicaret? Mihi credite, iudices, in hac causa non solum de L. Murenae verum etiam de vestra salute. In discrimen extremum venimus; nihil est iam unde nos reficiamus aut ubi lapsi. Non solum minuenda non sunt auxilia quae habemus sed etiam nova, si fieri possit, comparanda. Hostis est enim non apud, quod bello Punico gravissimum visum est, sed in urbe, in foro; di immortales! sine gemitu hoc dici non potest, non nemo etiam in illo sacrario rei publicae, in ipsa, inquam, curia non nemo hostis est. Di faxint ut meus conlega, vir fortissimus, hoc Catilinae nefarium latrocinium armatus opprimat! ego togatus vobis bonisque omnibus adiutoribus hoc quod conceptum res publica periculum consilio et comprimam."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.644, 9.147, 0.917)
(0.795, 8.962, 0.953)
(1.693, 0.591, 0.629, 0.371)
0.956
(0.286, 0.128, 0.448)
(0.114, 0.079, 0.59)


In [65]:
sentence = "Per pol quam paucos reperias meretricibus fidelis evenire amatores, Syra. vel hic Pamphilus iurabat quotiens Bacchidi, quam sancte, uti quivis facile posset credere, numquam illa viva ducturum uxorem domum! em duxit. ergo propterea te sedulo et moneo et hortor ne quoiusquam misereat, quin spolies mutiles laceres quemque nacta sis. utine eximium neminem habeam? neminem: nam nemo illorum quisquam, scito, ad te venit quin ita paret sese abs te ut blanditiis suis quam minimo pretio suam voluptatem expleat. hiscin tu amabo non contra insidiabere? tamen pol eandem iniuriumst esse omnibus. iniurium autem est ulcisci advorsarios, aut qua via te captent eadem ipsos capi? eheu me miseram, quor non aut istaec mihi aetas et formast aut tibi haec sententia? Senex si quaeret me, modo isse dicito ad portum percontatum adventum Pamphili. audin quid dicam, Scirte? si quaeret me, uti tum dicas; si non quaeret, nullus dixeris, alias ut uti possim causa hac integra. sed videon ego Philotium? unde haec advenit? Philotis, salve multum. o salve, Parmeno. salve mecastor, Parmeno. et tu edepol, Syra. dic mi, ubi, Philotis, te oblectasti iam diu?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.698, 9.343, 0.931)
(0.82, 8.637, 0.958)
(1.632, 0.613, 0.62, 0.38)
0.952
(0.155, 0.238, 1.536)
(0.24, 0.073, 0.768)


In [66]:
sentence = "Per pol quam paucos reperias meretricibus fidelis evenire amatores, Syra. vel hic Pamphilus iurabat quotiens Bacchidi, quam sancte, uti quivis facile posset credere, numquam illa viva ducturum uxorem domum! em duxit. ergo propterea te sedulo et moneo et hortor ne quoiusquam misereat, quin spolies mutiles laceres quemque nacta sis. utine eximium neminem habeam? neminem: nam nemo illorum quisquam, scito, ad te venit quin ita paret sese abs te ut blanditiis suis quam minimo pretio suam voluptatem expleat. hiscin tu amabo non contra insidiabere? tamen pol eandem iniuriumst esse omnibus. iniurium autem est ulcisci advorsarios, aut qua via te captent eadem ipsos capi? eheu me miseram, quor non aut istaec mihi aetas et formast aut tibi haec sententia? Senex si quaeret me, modo isse dicito ad portum percontatum adventum Pamphili. audin quid dicam, Scirte? si quaeret me, uti tum dicas; si non quaeret, dixeris, alias ut uti possim causa hac integra. sed videon ego Philotium? unde haec advenit? Philotis, salve multum. o salve, Parmeno. salve, Parmeno. et tu, Syra. dic mi, ubi, Philotis, te oblectasti iam diu?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.693, 9.196, 0.929)
(0.815, 8.468, 0.956)
(1.588, 0.63, 0.614, 0.386)
0.951
(0.157, 0.23, 1.464)
(0.244, 0.074, 0.768)


In [67]:
sentence = "Quibus rebus cognitis Caesar apud milites contionatur. Omnium temporum iniurias inimicorum in se commemorat; a quibus deductum ac depravatum Pompeium queritur invidia atque obtrectatione laudis suae, cuius ipse honori et dignitati semper faverit adiutorque fuerit. Novum in re publica introductum exemplum queritur, ut tribunicia intercessio armis votaretur atque opprimeretur, quae superioribus annis sine armis esset restituta. Sullam nudata omnibus rebus tribunicia potestate tamen intercessionem liberam reliquisse. Pompeium, qui amissa restituisse videatur, bona etiam, quae ante habuerint, ademisse. Quotienscumque sit decretum, darent operam magistratus, ne quid res publica detrimenti caperet (qua voce et quo senatus consulto populus Romanus ad arma sit vocatus), factum in perniciosis legibus, in vi tribunicia, in secessione populi templis locisque editioribus occupatis: atque haec superioris aetatis exempla expiata Saturnini atque Gracchorum casibus docet; quarum rerum illo tempore nihil factum, ne cogitatum quidem. nulla lex promulgata, non cum populo agi coeptum, nulla secessio facta. Hortatur, cuius imperatoris ductu novem annis rem publicam felicissime gesserint plurimaque proelia secunda fecerint, omnem Galliam Germaniamque pacaverint, ut eius existimationem dignitatemque ab inimicis defendant. Conclamant legionis tredecem, quae aderat, milites, hanc enim initio tumultus evocaverat, reliquae nondum convenerant, sese paratos esse imperatoris sui tribunorumque plebis iniurias defendere."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.686, 9.549, 0.928)
(0.761, 9.063, 0.945)
(2.731, 0.366, 0.732, 0.268)
0.941
(0.323, 0.185, 0.571)
(0.108, 0.139, 0.438)


In [68]:
sentence = "Quibus rebus cognitis Caesar apud milites contionatur. Omnium temporum iniurias inimicorum in se commemorat; a quibus deductum ac depravatum Pompeium queritur invidia atque laudis suae, cuius ipse honori et dignitati semper faverit adiutorque fuerit. Novum in re publica introductum exemplum queritur, ut tribunicia intercessio armis atque opprimeretur, quae superioribus annis sine armis esset restituta. Sullam nudata omnibus rebus tribunicia potestate tamen intercessionem liberam reliquisse. Pompeium, qui amissa restituisse videatur, etiam, quae ante habuerint, ademisse. Quotienscumque sit decretum, darent operam magistratus, ne quid res publica detrimenti caperet (qua voce et quo senatus consulto populus Romanus ad arma sit vocatus), factum in perniciosis legibus, in vi tribunicia, in secessione populi templis locisque editioribus occupatis: atque haec superioris aetatis exempla expiata Saturnini atque Gracchorum casibus docet; quarum rerum illo tempore nihil factum, ne cogitatum quidem. nulla lex promulgata, non cum populo agi coeptum, nulla secessio facta. Hortatur, cuius imperatoris novem annis rem publicam felicissime gesserint plurimaque proelia secunda fecerint, omnem Galliam Germaniamque pacaverint, ut eius existimationem dignitatemque ab inimicis defendant. Conclamant legionis tredecem, quae aderat, milites, hanc enim initio tumultus evocaverat, reliquae nondum convenerant, sese paratos esse imperatoris sui tribunorumque plebis iniurias defendere."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.679, 9.359, 0.926)
(0.754, 8.853, 0.943)
(2.654, 0.377, 0.726, 0.274)
0.94
(0.325, 0.178, 0.548)
(0.105, 0.142, 0.426)


In [69]:
sentence = "Intumuit Iuno, postquam inter sidera paelex fulsit, et ad canam descendit in aequora Tethyn Oceanumque senem, quorum reverentia movit saepe deos, causamque viae scitantibus infit: quaeritis, aetheriis quare regina deorum sedibus huc adsim? pro me tenet altera caelum! o ego quantum egi! quam vasta potentia nostra est. esse hominem vetui: facta est dea! sic ego poenas sontibus inpono, sic est mea magna potestas! vindicet antiquam faciem vultusque ferinos detrahat, Argolica quod in ante Phoronide fecit cur non et pulsa ducit Iunone meoque collocat in thalamo socerumque Lycaona sumit?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.852, 7.995, 0.964)
(0.952, 7.559, 0.988)
(2.52, 0.397, 0.716, 0.284)
0.949
(0.295, 0.182, 0.615)
(0.18, 0.034, 0.842)


In [70]:
sentence = "Intumuit Iuno, postquam inter sidera paelex fulsit, et ad canam descendit in aequora Tethyn Oceanumque senem, quorum reverentia movit saepe deos, causamque viae infit: quaeritis, aetheriis quare regina deorum sedibus huc adsim? pro me tenet altera caelum! o ego quantum egi! quam vasta potentia nostra est. esse hominem vetui: facta est dea! sic ego poenas inpono, sic est mea magna potestas! vindicet antiquam faciem vultusque ferinos detrahat, quod in ante fecit cur non et pulsa Iunone meoque collocat in thalamo socerumque sumit?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.841, 7.62, 0.961)
(0.947, 7.152, 0.987)
(2.28, 0.439, 0.695, 0.305)
0.945
(0.268, 0.183, 0.682)
(0.181, 0.024, 0.882)


In [71]:
sentence = "Aeneas scopulum interea conscendit, et omnem prospectum late pelago petit, Anthea si quem iactatum vento videat Phrygiasque biremis, aut Capyn, aut celsis in puppibus arma Caici. Navem in conspectu nullam, tris litore cervos prospicit errantis; hos tota armenta sequuntur a tergo, et longum per vallis pascitur agmen. Constitit hic, arcumque manu celerisque sagittas corripuit, fidus quae tela gerebat Achates; ductoresque ipsos primum, capita alta ferentis cornibus arboreis, sternit, tum volgus, et omnem miscet agens telis nemora inter frondea turbam; nec prius absistit, quam septem ingentia victor corpora fundat humi, et numerum cum navibus aequet. Hinc portum petit, et socios partitur in omnes. Vina bonus quae deinde cadis onerarat Acestes litore Trinacrio dederatque abeuntibus heros, dividit, et dictis maerentia pectora mulcet: O socii, neque enim ignari sumus ante malorum, O passi graviora, dabit deus his quoque finem. Vos et Scyllaeam rabiem penitusque sonantis accestis scopulos, vos et Cyclopea saxa experti: revocate animos, maestumque timorem mittite: forsan et haec olim meminisse iuvabit. Per varios casus, per tot discrimina rerum tendimus in Latium; sedes ubi fata quietas ostendunt; illic fas regna resurgere Troiae. Durate, et vosmet rebus servate secundis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.833, 11.365, 0.965)
(0.931, 11.211, 0.986)
(3.537, 0.283, 0.78, 0.22)
0.947
(0.369, 0.176, 0.478)
(0.161, 0.07, 0.698)


In [72]:
sentence = "Aeneas scopulum interea conscendit, et omnem prospectum late pelago petit, si quem iactatum vento videat, aut, aut celsis in puppibus arma. Navem in conspectu nullam, tris litore cervos prospicit errantis; hos tota armenta sequuntur a tergo, et longum per vallis pascitur agmen. Constitit hic, arcumque manu celerisque sagittas corripuit, fidus quae tela gerebat; ductoresque ipsos primum, capita alta ferentis cornibus arboreis, tum, et omnem miscet agens telis nemora inter turbam; nec prius absistit, quam septem ingentia victor corpora fundat humi, et numerum cum navibus aequet. Hinc portum petit, et socios partitur in omnes. Vina bonus quae deinde cadis onerarat litore dederatque abeuntibus heros, dividit, et dictis maerentia pectora mulcet: O socii, neque enim ignari sumus ante malorum, O passi graviora, dabit deus his quoque finem. Vos et rabiem sonantis accestis scopulos, vos et Cyclopea saxa experti: revocate animos, maestumque timorem mittite: et haec olim meminisse iuvabit. Per varios casus, per tot discrimina rerum tendimus in Latium; sedes ubi fata quietas ostendunt; illic fas regna resurgere Troiae. Durate, et vosmet rebus servate secundis."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.82, 10.751, 0.961)
(0.924, 10.572, 0.984)
(3.195, 0.313, 0.762, 0.238)
0.942
(0.358, 0.185, 0.516)
(0.169, 0.081, 0.674)


In [73]:
sentence = "Bello deinde Aborigines Troianique simul petiti. Turnus rex Rutulorum, cui pacta Lavinia ante adventum Aeneae fuerat, praelatum sibi advenam aegre patiens simul Aeneae Latinoque bellum intulerat. Neutra acies laeta ex eo certamine abiit: victi Rutuli: victores Aborigines Troianique ducem Latinum amisere. Inde Turnus Rutulique diffisi rebus ad florentes opes Etruscorum Mezentiumque regem eorum confugiunt, qui Caere opulento tum oppido imperitans, iam inde ab initio minime laetus novae origine urbis et tum nimio plus quam satis tutum esset accolis rem Troianam crescere ratus, haud gravatim socia arma Rutulis iunxit. Aeneas adversus tanti belli terrorem ut animos Aboriginum sibi conciliaret nec sub eodem iure solum sed etiam nomine omnes essent, Latinos utramque gentem appellavit; nec deinde Aborigines Troianis studio ac fide erga regem Aeneam cessere."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.789, 8.746, 0.951)
(0.809, 7.839, 0.953)
(3.241, 0.309, 0.764, 0.236)
0.965
(0.32, 0.152, 0.475)
(0.048, 0.105, 0.316)


In [74]:
sentence = "Bello deinde simul petiti. Turnus rex, cui pacta Lavinia ante adventum Aeneae fuerat, praelatum sibi advenam aegre patiens simul Aeneae Latinoque bellum intulerat. Neutra acies laeta ex eo certamine abiit: victi Rutuli: victores Aborigines Troianique ducem Latinum amisere. Inde Turnus Rutulique diffisi rebus ad florentes opes regem eorum confugiunt, qui opulento tum oppido imperitans, iam inde ab initio minime laetus novae origine urbis et tum quam satis tutum esset accolis rem Troianam crescere ratus, haud socia arma Rutulis iunxit. Aeneas adversus tanti belli terrorem ut animos Aboriginum sibi conciliaret nec sub eodem iure solum sed etiam nomine omnes essent, utramque gentem appellavit; nec deinde Aborigines Troianis studio ac fide erga regem Aeneam cessere."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.796, 8.466, 0.952)
(0.821, 7.529, 0.956)
(2.897, 0.345, 0.743, 0.257)
0.962
(0.304, 0.157, 0.514)
(0.061, 0.105, 0.368)


In [75]:
sentence = "Multas per gentes et multa per aequora vectus advenio has miseras, frater, ad inferias, ut te postremo donarem munere mortis et mutam nequiquam alloquerer cinerem. quandoquidem fortuna mihi tete abstulit ipsum. heu miser indigne frater adempte mihi, nunc tamen interea haec, prisco quae more parentum tradita sunt tristi munere ad inferias, accipe fraterno multum manantia fletu, atque in perpetuum, frater, ave atque vale. tempore quo primum vestis mihi tradita pura est, iucundum cum aetas florida ver ageret, multa satis lusi: non est dea nescia nostri, quae dulcem curis miscet amaritiem."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.8, 7.589, 0.95)
(0.875, 7.0, 0.968)
(2.462, 0.406, 0.711, 0.289)
0.958
(0.297, 0.143, 0.481)
(0.11, 0.055, 0.667)


In [76]:
sentence = "Multas per gentes et multa per aequora vectus advenio has miseras, frater, ad inferias, ut te postremo donarem munere mortis et mutam nequiquam alloquerer cinerem. quandoquidem fortuna mihi tete abstulit ipsum. heu miser indigne frater adempte mihi, nunc tamen interea haec, prisco quae more parentum tradita sunt tristi munere ad inferias, accipe fraterno multum manantia fletu, atque in perpetuum, frater, ave atque vale. tempore quo primum mihi tradita est, iucundum cum aetas florida ver ageret, multa satis lusi: non est dea nescia nostri, quae dulcem curis miscet."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.793, 7.398, 0.948)
(0.869, 6.786, 0.966)
(2.346, 0.426, 0.701, 0.299)
0.956
(0.284, 0.148, 0.52)
(0.114, 0.057, 0.667)


In [77]:
sentence = "Littera quaecumque est toto tibi lecta libello, est mihi sollicito tempore facta viae. aut haec me, gelido tremerem cum mense Decembri, scribentem mediis Hadria vidit aquis; aut, postquam bimarem cursu superavimus Isthmon, alteraque est nostrae sumpta carina fugae, quod facerem versus inter fera murmura ponti, Cycladas Aegaeas obstipuisse puto. ipse ego nunc miror tantis animique marisque fluctibus ingenium non cecidisse meum. saepe maris pars intus erat; tamen ipse trementi carmina ducebam qualiacumque manu. nunc quoque contenti stridunt Aquilone rudentes, inque modum cumuli concava surgit aqua. ipse gubernator tollens ad sidera palmas exposcit votis, inmemor artis, opem. quocumque aspicio, nihil est nisi mortis imago, quam dubia timeo mente timensque precor. nam simul insidiis hominum pelagique laboro, et faciunt geminos ensis et unda metus. cumque sit hibernis agitatum flatibus aequor, pectora sunt ipso turbidiora mari.quo magis his debes ignoscere, candide lector, si spe sunt, ut sunt, inferiora tua. non haec in nostris, ut quondam, scripsimus hortis, nec, consuete, meum, lectule, corpus habes. iactor in indomito brumali luce profundo ipsaque caeruleis charta feritur aquis. improba pugnat hiems indignaturque quod ausim scribere se rigidas incutiente minas. vincat hiems hominem! sed eodem tempore, quaeso, ipse modum statuam carminis, illa sui."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.779, 10.885, 0.953)
(0.88, 10.49, 0.974)
(2.679, 0.373, 0.728, 0.272)
0.953
(0.35, 0.157, 0.449)
(0.133, 0.071, 0.65)


In [78]:
sentence = "Littera quaecumque est toto tibi lecta libello, est mihi sollicito tempore facta viae. aut haec me, gelido tremerem cum mense Decembri, scribentem mediis vidit aquis; aut, postquam cursu superavimus, alteraque est nostrae sumpta carina fugae, quod facerem versus inter fera murmura ponti, Aegaeas obstipuisse puto. ipse ego nunc miror tantis animique marisque fluctibus ingenium non cecidisse meum. saepe maris pars intus erat; tamen ipse trementi carmina ducebam qualiacumque manu. nunc quoque stridunt Aquilone rudentes, inque modum surgit aqua. ipse gubernator tollens ad sidera palmas exposcit votis, inmemor artis, opem. quocumque aspicio, nihil est nisi mortis imago, quam dubia timeo mente timensque precor. nam simul insidiis hominum pelagique laboro, et faciunt geminos ensis et unda metus. cumque sit hibernis agitatum aequor, pectora sunt ipso mari.quo magis his debes ignoscere, candide lector, si spe sunt, ut sunt, inferiora tua. non haec in nostris, ut quondam, scripsimus hortis, nec, consuete, meum, lectule, corpus habes. iactor in indomito luce ipsaque caeruleis charta feritur aquis. improba pugnat hiems indignaturque quod ausim scribere se rigidas incutiente minas. vincat hiems hominem! sed eodem tempore, quaeso, ipse modum statuam carminis, illa sui."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.766, 10.395, 0.949)
(0.87, 9.96, 0.971)
(2.472, 0.405, 0.712, 0.288)
0.95
(0.333, 0.167, 0.5)
(0.141, 0.07, 0.667)


In [79]:
sentence = "Roma si vestrum est opus Iliaeque litus Etruscum tenuere turmae, iussa pars mutare lares et urbem sospite cursu, cui per ardentem sine fraude Troiam castus Aeneas patriae superstes liberum munivit iter, daturus plura relictis: di, probos mores docili iuventae, di, senectuti placidae quietem, Romulae genti date remque prolemque et decus omne. Quaeque vos bobus veneratur albis clarus Anchisae Venerisque sanguis, impetret, bellante prior, iacentem lenis in hostem. iam mari terraque manus potentis Medus Albanasque timet securis, iam Scythae responsa petunt, superbi nuper et Indi. iam Fides et Pax et Honos Pudorque priscus et neglecta redire Virtus audet adparetque beata pleno Copia cornu."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.912, 9.208, 0.98)
(0.977, 9.058, 0.995)
(5.375, 0.186, 0.843, 0.157)
0.952
(0.382, 0.196, 0.513)
(0.088, 0.118, 0.429)


In [80]:
sentence = "Roma si vestrum est opus litus Etruscum tenuere turmae, iussa pars mutare lares et urbem cursu, cui per ardentem sine fraude Troiam castus Aeneas patriae superstes liberum munivit iter, daturus plura relictis: di, probos mores docili iuventae, di, senectuti placidae quietem, Romulae genti date remque prolemque et decus omne. Quaeque vos bobus albis clarus Anchisae Venerisque sanguis, impetret, bellante prior, iacentem lenis in hostem. iam mari terraque manus potentis timet securis, iam Scythae, superbi nuper et Indi. iam Fides et Pax et Honos Pudorque priscus et neglecta redire Virtus audet adparetque beata pleno cornu."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.904, 8.767, 0.978)
(0.974, 8.605, 0.994)
(4.875, 0.205, 0.83, 0.17)
0.947
(0.383, 0.181, 0.472)
(0.074, 0.117, 0.389)


In [81]:
sentence = "Sic eorum potestas et auctoritas totius orbis ambitus occupavit, sic imperium suum ultra solis vias et ipsius oceani limites propagavit, dum exercent in armis virtutem religiosam, dum urbem muniunt sacrorum religionibus, castis virginibus, multis honoribus ac nominibus sacerdotum, dum obsessi et citra solum Capitolium capti colunt deos, quos alius iam sprevisset iratos, et per Gallorum acies mirantium superstitionis audaciam pergunt telis inermes, sed cultu religionis armati, dum captis in hostilibus moenibus adhuc ferociente victoria numina victa venerantur, dum undique hospites deos quaerunt et suos faciunt, dum aras extruunt etiam ignotis numinibus et Manibus. Sic, dum universarum gentium sacra suscipiunt, etiam regna meruerunt. Hinc perpetuus venerationis tenor mansit, qui longa aetate non infringitur, sed augetur: quippe antiquitas caerimoniis atque fanis tantum sanctitatis tribuere consuevit quantum adstruxerit vetustatis. Nec tamen temere (ausim enim interim et ipse concedere et sic melius errare) maiores nostri aut observandis auguriis aut extis consulendis aut instituendis sacris aut delubris dedicandis operam navaverunt. Specta de libris memoriam; iam eos deprehendes initiasse ritus omnium religionum, vel ut remuneraretur divina indulgentia, vel ut averteretur imminens ira aut iam tumens et saeviens placaretur. Testis Mater Idaea, quae adventu suo et probavit matronae castitatem et urbem metu hostili liberavit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.777, 10.901, 0.952)
(0.93, 11.122, 0.985)
(2.648, 0.378, 0.726, 0.274)
0.946
(0.298, 0.212, 0.712)
(0.137, 0.096, 0.587)


In [82]:
sentence = "Sic eorum potestas et auctoritas totius orbis ambitus occupavit, sic imperium suum ultra solis vias et ipsius oceani limites, dum exercent in armis virtutem religiosam, dum urbem muniunt sacrorum religionibus, castis virginibus, multis honoribus ac nominibus sacerdotum, dum obsessi et solum capti colunt deos, quos alius iam sprevisset iratos, et per Gallorum acies mirantium superstitionis audaciam pergunt telis inermes, sed cultu religionis armati, dum captis in hostilibus moenibus adhuc victoria numina victa venerantur, dum undique hospites deos quaerunt et suos faciunt, dum aras extruunt etiam ignotis numinibus et Manibus. Sic, dum universarum gentium sacra suscipiunt, etiam regna meruerunt. Hinc perpetuus venerationis mansit, qui longa aetate non infringitur, sed augetur: quippe antiquitas caerimoniis atque tantum sanctitatis tribuere consuevit quantum adstruxerit vetustatis. Nec tamen temere (ausim enim interim et ipse concedere et sic melius errare) maiores nostri aut observandis auguriis aut consulendis aut instituendis sacris aut delubris dedicandis. Specta memoriam; iam eos deprehendes initiasse ritus omnium religionum, vel ut divina indulgentia, vel ut averteretur imminens ira aut iam tumens et saeviens placaretur. Testis, quae adventu suo et probavit matronae castitatem et urbem metu hostili liberavit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.76, 10.275, 0.947)
(0.923, 10.525, 0.984)
(2.453, 0.408, 0.71, 0.29)
0.94
(0.272, 0.201, 0.74)
(0.131, 0.098, 0.571)


In [83]:
sentence = "Iamque fere mediam caeli nox umida metam contigerat, placida laxabant membra quiete sub remis fusi per dura sedilia nautae, cum levis aetheriis delapsus Somnus ab astris aera dimovit tenebrosum et dispulit umbras, te, Palinure, petens, tibi somnia tristia portans insonti; puppique deus consedit in alta Phorbanti similis funditque has ore loquelas: Iaside Palinure, ferunt ipsa aequora classem, aequatae spirant aurae, datur hora quieti. pone caput fessosque oculos furare labori. Ipse ego paulisper pro te tua munera inibo. Cui vix attolens Palinurus lumina fatur: Mene salis placidi vultum fluctusque quietos ignorare iubes? Mene huic confidere monstro? talia dicta dabat, clavum affixus et haerens nusquam amittebat oculosque sub astra tenebat. Ecce deus ramum Lethaeo rore madentem vique soporatum Stygia super utraque quassat tempora, cunctantique natantia lumina solvit. vix primos inopina quies laxaverat artus, et super incumbens cum puppis parte revulsa cumque gubernaclo liquidas proiecit in undas praecipitem ac socios nequiquam saepe vocantem; ipse volans tenuis se sustulit ales ad auras. Currit iter tutum non setius aequore classis. iamque adeo scopulos Sirenum advecta subibat, difficilis quondam multorumque ossibus albos, cum pater amisso fluitantem errare magistro sensit, et ipse ratem nocturnis rexit in undis multa gemens casuque animum concussus amici: O nimium caelo et pelago confise sereno, nudus in ignota, Palinure, iacebis harena."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.804, 11.621, 0.959)
(0.863, 11.415, 0.971)
(5.147, 0.194, 0.837, 0.163)
0.971
(0.376, 0.181, 0.481)
(0.124, 0.12, 0.51)


In [84]:
sentence = "Iamque fere mediam caeli nox umida metam contigerat, placida laxabant membra quiete sub fusi per dura nautae, cum levis aetheriis delapsus Somnus ab astris aera tenebrosum et umbras, te, Palinure, petens, tibi somnia tristia portans, puppique deus consedit in alta similis funditque has ore: Palinure, ferunt ipsa aequora classem, aequatae spirant aurae, datur hora quieti. pone caput fessosque oculos labori. Ipse ego paulisper pro te tua munera inibo. Cui vix attolens Palinurus lumina fatur: Mene salis placidi vultum fluctusque quietos ignorare iubes? Mene huic confidere monstro? talia dicta dabat, affixus et haerens nusquam amittebat oculosque sub astra tenebat. Ecce deus ramum rore madentem vique super utraque quassat, cunctantique natantia lumina solvit. vix primos inopina quies laxaverat artus, et super incumbens cum puppis parte revulsa cumque liquidas proiecit in undas praecipitem ac socios nequiquam saepe vocantem; ipse volans tenuis se sustulit ad auras. Currit iter tutum setius aequore classis. iamque adeo scopulos advecta subibat, difficilis quondam multorumque ossibus albos, cum pater amisso errare magistro sensit, et ipse ratem nocturnis rexit in undis multa gemens casuque animum concussus amici: O nimium caelo et pelago confise sereno, nudus in ignota, Palinure, iacebis harena."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.784, 10.81, 0.954)
(0.847, 10.615, 0.967)
(4.758, 0.21, 0.826, 0.174)
0.968
(0.363, 0.195, 0.536)
(0.132, 0.1, 0.568)


In [85]:
sentence = "M. Cicero sub adventum triumvirorum urbe cesserat; primo in Tusculanum fugerat, inde transversis itineribus in Formianum, ut a Caieta navem conscensurus, proficiscitur. Unde aliquoties in altum provectum cum modo venti adversi retulissent, modo ipse iactationem navis caeco volvente fluctu pati non posset, taedium tandem eum et fugae et vitae cepit regressusque ad superiorem villam, quae paulo plus mille passibus a mari abest, moriar inquit in patria saepe servata! Satis constat servos fortiter fideliterque paratos fuisse ad dimicandum; ipsum deponi lecticam et quietos pati, quod sors iniqua cogeret, iussisse. Prominenti ex lectica praebentique inmotam cervicem caput praecisum est. Nec satis stolidae crudelitati militum fuit: manus quoque, scripsisse in Antonium exprobrantes, praeciderunt. ita relatum caput ad Antonium iussuque eius inter duas manus in rostris positum, ubi ille consul, ubi saepe consularis, ubi eo ipso anno adversus Antonium quanta nulla umquam humana vox cum admiratione eloquentiae auditus fuerat. Vixit tres et sexaginta annos, ut, si vis afuisset, ne inmatura quidem mors videri possit. ingenium et operibus et praemiis operum felix; ipse fortunae diu prosperae, sed in longo tenore felicitatis magnis interim ictus vulneribus, exilio, ruina partium, pro quibus steterat, filiae exitu tam tristi atque acerbo, omnium adversorum nihil, ut viro dignum erat, tulit praeter mortem. Si quis tamen virtutibus vitia pensarit, vir magnus ac memorabilis fuit et in cuius laudes exsequendas Cicerone laudatore opus fuerit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.757, 11.275, 0.948)
(0.897, 11.165, 0.978)
(2.313, 0.432, 0.698, 0.302)
0.962
(0.277, 0.152, 0.548)
(0.081, 0.095, 0.462)


In [86]:
sentence = "M. Cicero sub adventum triumvirorum urbe cesserat; primo in fugerat, inde itineribus in ut a navem conscensurus, proficiscitur. Unde in provectum cum modo venti adversi retulissent, modo ipse navis volvente fluctu pati non posset, taedium tandem eum et fugae et vitae cepit regressusque ad superiorem villam, quae paulo plus mille passibus a mari abest, moriar inquit in patria saepe servata! Satis constat servos fortiter fideliterque paratos fuisse ad dimicandum; ipsum deponi et quietos pati, quod sors iniqua cogeret, iussisse. ex lectica praebentique inmotam cervicem caput praecisum est. Nec satis crudelitati militum fuit: manus quoque, scripsisse in. ita relatum caput ad Antonium iussuque eius inter duas manus in rostris positum, ubi ille consul, ubi saepe consularis, ubi eo ipso anno adversus Antonium quanta nulla umquam humana vox cum admiratione eloquentiae auditus fuerat. Vixit tres et sexaginta annos, ut, si vis afuisset, ne inmatura quidem mors videri possit. ingenium et operibus et praemiis operum felix; ipse fortunae diu prosperae, sed in longo felicitatis magnis interim ictus vulneribus, exilio, ruina partium, pro quibus steterat, filiae exitu tam tristi atque acerbo, omnium nihil, ut viro dignum erat, tulit praeter mortem. Si quis tamen virtutibus vitia vir magnus ac memorabilis fuit et in cuius laudes exsequendas Cicerone laudatore opus fuerit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.751, 10.756, 0.946)
(0.906, 10.641, 0.98)
(2.06, 0.486, 0.673, 0.327)
0.958
(0.28, 0.159, 0.569)
(0.078, 0.093, 0.457)


In [87]:
sentence = "Postquam Caesar dicundi finem fecit, ceteri verbo alius alii varie adsentiebantur. At M. Porcius Cato rogatus sententiam huiusce modi orationem habuit: Longe alia mihi mens est, patres conscripti, cum res atque pericula nostra considero et cum sententias nonnullorum ipse mecum reputo. Illi mihi disseruisse videntur de poena eorum, qui patriae, parentibus, aris atque focis suis bellum paravere; res autem monet cavere ab illis magis quam, quid in illos statuamus, consultare. Nam cetera maleficia tum persequare, ubi facta sunt; hoc, nisi provideris, ne accidat, ubi evenit, frustra iudicia inplores: capta urbe nihil fit reliqui victis. Sed, per deos inmortalis, vos ego appello, qui semper domos, villas, signa, tabulas vostras pluris quam rem publicam fecistis: si ista, cuiuscumque modi sunt, quae amplexamini, retinere, si voluptatibus vostris otium praebere voltis, expergiscimini aliquando et capessite rem publicam! Non agitur de vectigalibus neque de sociorum iniuriis: libertas et anima nostra in dubio est. Saepenumero, patres conscripti, multa verba in hoc ordine feci, saepe de luxuria atque avaritia nostrorum civium questus sum multosque mortalis ea causa advorsos habeo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.733, 9.607, 0.94)
(0.831, 9.022, 0.961)
(2.185, 0.458, 0.686, 0.314)
0.965
(0.249, 0.173, 0.698)
(0.128, 0.076, 0.629)


In [88]:
sentence = "Postquam Caesar dicundi finem fecit, ceteri verbo alius alii varie adsentiebantur. At M. Porcius Cato rogatus sententiam huiusce modi orationem habuit: Longe alia mihi mens est, patres conscripti, cum res atque pericula nostra considero et cum sententias nonnullorum ipse mecum reputo. Illi mihi videntur de poena eorum, qui patriae, parentibus, atque suis bellum paravere; res autem monet cavere ab illis magis quam, quid in illos statuamus, consultare. Nam cetera maleficia tum persequare, ubi facta sunt; hoc, nisi provideris, ne accidat, ubi evenit, frustra iudicia: capta urbe nihil fit reliqui victis. Sed, per deos inmortalis, vos ego appello, qui semper domos, villas, vostras pluris quam rem publicam fecistis: si ista, cuiuscumque modi sunt, quae retinere, si voluptatibus vostris otium praebere voltis, aliquando et capessite rem publicam! Non agitur de vectigalibus neque de sociorum iniuriis: libertas et anima nostra in dubio est. Saepenumero, patres conscripti, multa verba in hoc ordine feci, saepe de luxuria atque avaritia nostrorum civium questus sum multosque mortalis ea causa advorsos habeo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.72, 9.214, 0.935)
(0.818, 8.581, 0.957)
(2.037, 0.491, 0.671, 0.329)
0.963
(0.232, 0.177, 0.763)
(0.134, 0.073, 0.647)


In [89]:
sentence = "Hinc Cereris ludi: non est opus indice causae; sponte deae munus promeritumque patet. panis erat primis virides mortalibus herbae, quas tellus nullo sollicitante dabat; et modo carpebant vivax e caespite gramen, nunc epulae tenera fronde cacumen erant. postmodo glans nota est: bene erat iam glande reperta, duraque magnificas quercus habebat opes. prima Ceres homine ad meliora alimenta vocato mutavit glandes utiliore cibo. illa iugo tauros collum praebere coegit: tum primum soles eruta vidit humus. aes erat in pretio, Chalybeia massa latebat: eheu, perpetuo debuit illa tegi. pace Ceres laeta est; et vos orate, coloni, perpetuam pacem pacificumque ducem. farra deae micaeque licet salientis honorem detis et in veteres turea grana focos; et, si tura aberunt, unctas accendite taedas: parva bonae Cereri, sint modo casta, placent. a bove succincti cultros removete ministri: bos aret; ignavam sacrificate suem. apta iugo cervix non est ferienda securi: vivat et in dura saepe laboret humo. Exigit ipse locus raptus ut virginis edam: plura recognosces, pauca docendus eris. terra tribus scopulis vastum procurrit in aequor Trinacris, a positu nomen adepta loci, grata domus Cereri: multas ea possidet urbes, in quibus est culto fertilis Henna solo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.799, 10.984, 0.957)
(0.9, 11.023, 0.979)
(3.846, 0.26, 0.794, 0.206)
0.934
(0.402, 0.169, 0.421)
(0.132, 0.09, 0.595)


In [90]:
sentence = "Hinc Cereris ludi: non est opus indice causae; sponte deae munus patet. panis erat primis virides mortalibus herbae, quas tellus nullo sollicitante dabat; et modo carpebant vivax e caespite gramen, nunc epulae tenera fronde cacumen erant. glans nota est: bene erat iam glande reperta, duraque magnificas quercus habebat opes. prima Ceres homine ad meliora alimenta vocato mutavit glandes utiliore cibo. illa iugo tauros collum praebere coegit: tum primum soles eruta vidit humus. aes erat in pretio, latebat: eheu, perpetuo debuit illa tegi. pace Ceres laeta est; et vos orate, coloni, perpetuam pacem pacificumque ducem. farra deae licet honorem detis et in veteres grana focos; et, si tura aberunt, unctas accendite taedas: parva bonae Cereri, sint modo casta, placent. a bove succincti cultros removete ministri: bos aret; ignavam sacrificate suem. apta iugo cervix non est ferienda securi: vivat et in dura saepe laboret humo. Exigit ipse locus raptus ut virginis edam: plura recognosces, pauca docendus eris. terra tribus scopulis vastum procurrit in aequor a nomen adepta loci, grata domus Cereri: multas ea possidet urbes, in quibus est culto fertilis Henna solo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.789, 10.584, 0.954)
(0.894, 10.611, 0.977)
(3.615, 0.277, 0.783, 0.217)
0.93
(0.387, 0.166, 0.429)
(0.144, 0.089, 0.619)


In [91]:
sentence = "Neque vero tam remisso ac languido animo quisquam omnium fuit, qui ea nocte conquieverit. tanta erat summae rerum exspectatio, ut alius in aliam partem mente atque animo traheretur, quid ipsis Corfiniensibus, quid Domitio, quid Lentulo, quid reliquis accideret, qui quosque eventus exciperent. Quarta vigilia circiter Lentulus Spinther de muro cum vigiliis custodibusque nostris conloquitur: velle, si sibi fiat potestas, Caesarem convenire. facta potestate ex oppido mittitur, neque ab eo prius Domitiani milites discedunt, quam in conspectum Caesaris deducatur. cum eo de salute sua agit, orat atque obsecrat, ut sibi parcat, veteremque amicitiam commemorat Caesarisque in se beneficia exponit, quae erant maxima: quod per eum in collegium pontificum venerat, quod provinciam Hispaniam ex praetura habuerat, quod in petitione consulatus erat sublevatus. cuius orationem Caesar interpellat: se non maleficii causa ex provincia egressum, sed uti se a contumeliis inimicorum defenderet, ut tribunos plebis in ea re ex civitate expulsos in suam dignitatem restitueret, et se et populum Romanum factione paucorum oppressum in libertatem vindicaret. cuius oratione confirmatus Lentulus, ut in oppidum reverti liceat, petit: quod de sua salute inpetraverit, fore etiam reliquis ad suam spem solacio; adeo esse perterritos nonnullos, ut suae vitae durius consulere cogantur. facta potestate discedit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.66, 9.262, 0.921)
(0.866, 9.442, 0.97)
(1.526, 0.655, 0.604, 0.396)
0.943
(0.274, 0.162, 0.593)
(0.132, 0.076, 0.634)


In [92]:
sentence = "Neque vero tam remisso ac languido animo quisquam omnium fuit, qui ea nocte conquieverit. tanta erat summae rerum exspectatio, ut alius in aliam partem mente atque animo traheretur, quid ipsis, quid Domitio, quid Lentulo, quid reliquis accideret, qui quosque eventus exciperent. Quarta vigilia circiter Lentulus Spinther de muro cum vigiliis custodibusque nostris conloquitur: velle, si sibi fiat potestas, Caesarem convenire. facta potestate ex oppido mittitur, neque ab eo prius Domitiani milites discedunt, quam in conspectum Caesaris deducatur. cum eo de salute sua agit, orat atque obsecrat, ut sibi parcat, veteremque amicitiam commemorat Caesarisque in se beneficia exponit, quae erant maxima: quod per eum in collegium pontificum venerat, quod provinciam ex praetura habuerat, quod in petitione consulatus erat sublevatus. cuius orationem Caesar interpellat: se non maleficii causa ex provincia egressum, sed uti se a contumeliis inimicorum defenderet, ut tribunos plebis in ea re ex civitate expulsos in suam dignitatem restitueret, et se et populum Romanum factione paucorum oppressum in libertatem vindicaret. cuius oratione confirmatus Lentulus, ut in oppidum reverti liceat, petit: quod de sua salute inpetraverit, fore etiam reliquis ad suam spem solacio; adeo esse perterritos nonnullos, ut suae vitae durius consulere cogantur. facta potestate discedit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.656, 9.166, 0.92)
(0.863, 9.337, 0.969)
(1.5, 0.667, 0.6, 0.4)
0.942
(0.272, 0.164, 0.604)
(0.133, 0.077, 0.634)


In [93]:
sentence = "Sed memoria mea ingenti virtute, divorsis moribus fuere viri duo, M. Cato et C. Caesar. Quos quoniam res obtulerat, silentio praeterire non fuit consilium, quin utriusque naturam et mores, quantum ingenio possum, aperirem. Igitur iis genus, aetas, eloquentia prope aequalia fuere, magnitudo animi par, item gloria, sed alia alii. Caesar beneficiis ac munificentia magnus habebatur, integritate vitae Cato. Ille mansuetudine et misericordia clarus factus, huic severitas dignitatem addiderat. Caesar dando, sublevando, ignoscundo, Cato nihil largiundo gloriam adeptus est. In altero miseris perfugium erat, in altero malis pernicies. Illius facilitas, huius constantia laudabatur. Postremo Caesar in animum induxerat laborare, vigilare; negotiis amicorum intentus sua neglegere, nihil denegare, quod dono dignum esset; sibi magnum imperium, exercitum, bellum novum exoptabat, ubi virtus enitescere posset. At Catoni studium modestiae, decoris, sed maxume severitatis erat; non divitiis cum divite neque factione cum factioso, sed cum strenuo virtute, cum modesto pudore, cum innocente abstinentia certabat; esse quam videri bonus malebat: ita, quo minus petebat gloriam, eo magis illum sequebatur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.767, 9.791, 0.948)
(0.852, 9.416, 0.967)
(2.976, 0.336, 0.748, 0.252)
0.945
(0.333, 0.117, 0.352)
(0.074, 0.098, 0.429)


In [94]:
sentence = "Sed memoria mea ingenti virtute, divorsis moribus fuere viri duo, M. Cato et C. Caesar. Quos quoniam res obtulerat, silentio praeterire non fuit consilium, quin utriusque naturam et mores, quantum ingenio possum, aperirem. Igitur iis genus, aetas, eloquentia prope aequalia fuere, magnitudo animi par, item gloria, sed alia alii. Caesar beneficiis ac magnus habebatur, vitae Cato. Ille et misericordia clarus factus, huic severitas dignitatem addiderat. Caesar dando, ignoscundo, Cato nihil largiundo gloriam adeptus est. In altero miseris perfugium erat, in altero malis pernicies. Illius facilitas, huius constantia laudabatur. Postremo Caesar in animum induxerat laborare, vigilare; negotiis amicorum intentus sua neglegere, nihil denegare, quod dono dignum esset; sibi magnum imperium, exercitum, bellum novum exoptabat, ubi virtus posset. At Catoni studium modestiae, decoris, sed maxume severitatis erat; non divitiis cum divite neque factione cum, sed cum strenuo virtute, cum modesto pudore, cum innocente certabat; esse quam videri bonus malebat: ita, quo minus petebat gloriam, eo magis illum sequebatur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.756, 9.448, 0.945)
(0.843, 9.045, 0.964)
(2.805, 0.357, 0.737, 0.263)
0.942
(0.314, 0.128, 0.408)
(0.083, 0.09, 0.481)


In [95]:
sentence = "Tertius aequoreis inclusum Piscibus annum finierat Titan, omnemque refugerat Orpheus femineam Venerem, seu quod male cesserat illi, sive fidem dederat; multas tamen ardor habebat iungere se vati, multae doluere repulsae. ille etiam Thracum populis fuit auctor amorem in teneros transferre mares citraque iuventam aetatis breve ver et primos carpere flores. Collis erat collemque super planissima campi area, quam viridem faciebant graminis herbae: umbra loco deerat; qua postquam parte resedit dis genitus vates et fila sonantia movit, umbra loco venit: non Chaonis afuit arbor, non nemus Heliadum, non frondibus aesculus altis, nec tiliae molles, nec fagus et innuba laurus, et coryli fragiles et fraxinus utilis hastis enodisque abies curvataque glandibus ilex et platanus genialis acerque coloribus inpar amnicolaeque simul salices et aquatica lotos perpetuoque virens buxum tenuesque myricae et bicolor myrtus et bacis caerula tinus. vos quoque, flexipedes hederae, venistis et una pampineae vites et amictae vitibus ulmi ornique et piceae pomoque onerata rubenti arbutus et lentae, victoris praemia, palmae et succincta comas hirsutaque vertice pinus, grata deum matri, siquidem Cybeleius Attis exuit hac hominem truncoque induruit illo. Adfuit huic turbae metas imitata cupressus, nunc arbor, puer ante."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.85, 11.627, 0.969)
(0.953, 11.633, 0.99)
(3.921, 0.255, 0.797, 0.203)
0.92
(0.481, 0.091, 0.189)
(0.075, 0.08, 0.483)


In [96]:
sentence = "Tertius aequoreis inclusum annum finierat, omnemque refugerat Orpheus femineam Venerem, seu quod male cesserat illi, sive fidem dederat; multas tamen ardor habebat iungere se vati, multae doluere repulsae. ille etiam populis amorem in teneros transferre mares iuventam aetatis breve ver et primos carpere flores. Collis erat collemque super planissima campi area, quam viridem faciebant graminis herbae: umbra loco deerat; qua postquam parte resedit dis genitus vates et fila sonantia movit, umbra loco venit: non afuit arbor, non nemus, non frondibus altis, nec molles, nec fagus et laurus, et coryli fragiles et fraxinus utilis hastis abies curvataque glandibus ilex et platanus coloribus simul salices et  perpetuoque virens buxum tenuesque myricae et myrtus et bacis caerula. vos quoque, hederae, venistis et una pampineae vites et vitibus ulmi ornique et piceae pomoque onerata rubenti et, victoris praemia, palmae et succincta hirsutaque vertice pinus, grata deum matri, siquidem exuit hac hominem truncoque illo. Adfuit huic turbae imitata cupressus, nunc arbor, puer ante."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.829, 10.422, 0.963)
(0.942, 10.364, 0.988)
(3.27, 0.306, 0.766, 0.234)
0.904
(0.443, 0.108, 0.243)
(0.083, 0.089, 0.481)


In [97]:
sentence = "Scipio postquam in Siciliam venit, voluntarios milites ordinavit centuriavitque. ex iis trecentos iuvenes, florentes aetate et virium robore insignes, inermes circa se habebat, ignorantes quem ad usum neque centuriati neque armati servarentur. tum ex totius Siciliae iuniorum numero principes genere et fortuna trecentos equites qui secum in Africam traicerent legit, diemque iis qua equis armisque instructi atque ornati adessent edixit. gravis ea militia, procul domo, terra marique multos labores magna pericula allatura videbatur; neque ipsos modo sed parentes cognatosque eorum ea cura angebat. ubi dies quae dicta erat advenit, arma equosque ostenderunt. tum Scipio renuntiari sibi dixit quosdam equites Siculorum tamquam gravem et duram horrere eam militiam: si qui ita animati essent, malle eos sibi iam tum fateri quam postmodo querentes segnes atque inutiles milites rei publicae esse; expromerent quid sentirent; cum bona venia se auditurum."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.75, 8.746, 0.941)
(0.9, 8.538, 0.977)
(1.957, 0.511, 0.662, 0.338)
0.948
(0.341, 0.145, 0.426)
(0.118, 0.103, 0.533)


In [98]:
sentence = "Scipio postquam in Siciliam venit, voluntarios milites. ex iis trecentos iuvenes, florentes aetate et virium robore insignes, inermes circa se habebat, ignorantes quem ad usum neque neque armati. tum ex totius Siciliae iuniorum numero trecentos equites qui secum in Africam legit, diemque iis qua equis armisque instructi atque ornati adessent edixit. gravis ea militia, procul domo, terra marique multos labores magna pericula allatura videbatur; neque ipsos modo sed parentes cognatosque eorum ea cura. ubi dies quae dicta erat advenit, arma equosque ostenderunt. tum Scipio sibi dixit quosdam equites Siculorum gravem et duram eam militiam: si qui ita essent, malle eos sibi iam tum quam querentes atque inutiles milites rei publicae esse; quid sentirent; se auditurum."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.713, 7.647, 0.929)
(0.873, 7.358, 0.968)
(1.614, 0.62, 0.617, 0.383)
0.935
(0.325, 0.145, 0.447)
(0.087, 0.078, 0.526)


In [99]:
sentence = "Et cum diu nunc in veneratione, nunc in desiderio regis haesissent, in ipsos versa miseratio est. Macedonia profecti ultra Euphraten in mediis hostibus novum imperium aspernantibus destitutos se esse cernebant: sine certo regis herede, sine herede regni publicas vires ad se quemque tracturum. Bella deinde civilia, quae secuta sunt, mentibus augurabantur: iterum non de regno Asiae, sed de rege ipsis sanguinem esse fundendum; novis vulneribus veteres rumpendas cicatrices; senes, debiles, modo petita missione a iusto rege, nunc morituros pro potentia forsitan satellitis alicuius ignobilis. Has cogitationes volventibus nox supervenit terroremque auxit. Milites in armis vigilabant. Babylonii alius e muris, alius culmine sui quisque tecti prospectabant, quasi certiora visuri. Nec quisquam lumina audebat accendere; et, quia oculorum cessabat usus, fremitus vocesque auribus captabant; ac plerumque, vano metu territi per obscuras semitas alius alii occursantes invicem suspecti ac solliciti ferebantur. Persae comis suo more detonsis in lugubri veste cum coniugibus ac liberis, non ut victorem et modo ut hostem, sed ut gentis suae iustissimum regem vero desiderio lugebant. Adsueti sub rege vivere non alium, qui imperaret ipsis, digniorem fuisse confitebantur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.747, 9.969, 0.944)
(0.858, 9.672, 0.968)
(2.49, 0.402, 0.713, 0.287)
0.958
(0.341, 0.123, 0.361)
(0.084, 0.101, 0.455)


In [100]:
sentence = "Et cum diu nunc in veneratione, nunc in desiderio regis haesissent, in ipsos versa miseratio est. profecti ultra Euphraten in mediis hostibus novum imperium aspernantibus se esse cernebant: sine certo regis herede, sine herede regni publicas vires ad se quemque tracturum. Bella deinde civilia, quae secuta sunt, mentibus augurabantur: iterum non de regno Asiae, sed de rege ipsis sanguinem esse fundendum; novis vulneribus veteres rumpendas; senes, debiles, modo petita missione a iusto rege, nunc morituros pro potentia forsitan alicuius ignobilis. Has cogitationes volventibus nox supervenit terroremque auxit. Milites in armis vigilabant. alius e muris, alius sui quisque tecti prospectabant, quasi certiora visuri. Nec quisquam lumina audebat accendere; et, quia oculorum usus, fremitus vocesque auribus captabant; ac plerumque, vano metu territi per obscuras alius alii occursantes invicem suspecti ac solliciti ferebantur. comis suo more detonsis in lugubri veste cum coniugibus ac liberis, non ut victorem et ut hostem, sed ut gentis suae iustissimum regem vero desiderio lugebant. Adsueti sub rege vivere non alium, qui imperaret ipsis, digniorem fuisse confitebantur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.738, 9.567, 0.941)
(0.847, 9.206, 0.965)
(2.36, 0.424, 0.702, 0.298)
0.955
(0.321, 0.119, 0.37)
(0.083, 0.101, 0.452)


In [101]:
sentence = "Hoc tibi soli putas accidisse et admiraris quasi rem novam quod peregrinatione tam longa et tot locorum varietatibus non discussisti tristitiam gravitatemque mentis? Animum debes mutare, non caelum. Licet vastum traieceris mare, licet, ut ait Vergilius noster, terraeque urbesque recedant, sequentur te quocumque perveneris vitia. Hoc idem querenti cuidam Socrates ait, quid miraris nihil tibi peregrinationes prodesse, cum te circumferas? premit te eadem causa quae expulit. Quid terrarum iuvare novitas potest? quid cognitio urbium aut locorum? in irritum cedit ista iactatio. Quaeris quare te fuga ista non adiuvet? tecum fugis. Onus animi deponendum est: non ante tibi ullus placebit locus. Vadis huc illuc ut excutias insidens pondus quod ipsa iactatione incommodius fit, sicut in navi onera immota minus urgent, inaequaliter convoluta citius eam partem in quam incubuere demergunt. Quidquid facis, contra te facis et motu ipso noces tibi; aegrum enim concutis. At cum istuc exemeris malum, omnis mutatio loci iucunda fiet; Cum hac persuasione vivendum est: non sum uni angulo natus, patria mea totus hic mundus est. Quod si liqueret tibi, non admirareris nil adiuvari te regionum varietatibus in quas subinde priorum taedio migras; prima enim quaeque placuisset si omnem tuam crederes."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.734, 10.176, 0.941)
(0.885, 10.135, 0.975)
(2.148, 0.466, 0.682, 0.318)
0.93
(0.24, 0.177, 0.739)
(0.146, 0.062, 0.7)


In [102]:
sentence = "Hoc tibi soli putas accidisse et admiraris quasi rem novam quod tam longa et tot locorum varietatibus non tristitiam gravitatemque mentis? Animum debes mutare, non caelum. Licet vastum traieceris mare, licet, ut ait Vergilius noster, terraeque urbesque recedant, sequentur te quocumque perveneris vitia. Hoc idem querenti cuidam Socrates ait, quid miraris nihil tibi prodesse, cum te? premit te eadem causa quae expulit. Quid terrarum iuvare novitas potest? quid cognitio urbium aut locorum? ista. Quaeris quare te fuga ista non adiuvet? tecum fugis. Onus animi deponendum est: non ante tibi ullus placebit locus. Vadis huc illuc ut excutias pondus quod ipsa incommodius fit, sicut in navi onera immota minus urgent, citius eam partem in quam incubuere demergunt. Quidquid facis, contra te facis et motu ipso noces tibi; aegrum enim concutis. At cum istuc exemeris malum, omnis mutatio loci iucunda fiet; Cum hac vivendum est: non sum uni angulo natus, patria mea totus hic mundus est. Quod si liqueret tibi, non admirareris nil adiuvari te regionum varietatibus in quas priorum taedio migras; enim placuisset si omnem tuam crederes."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.722, 9.573, 0.937)
(0.88, 9.522, 0.973)
(1.983, 0.504, 0.665, 0.335)
0.923
(0.227, 0.176, 0.775)
(0.147, 0.062, 0.703)


In [103]:
sentence = "Quis fuit, horrendos primus qui protulit enses? Quam ferus et vere ferreus ille fuit! Tum caedes hominum generi, tum proelia nata, Tum brevior dirae mortis aperta via est. An nihil ille miser meruit, nos ad mala nostra Vertimus, in saevas quod dedit ille feras? Divitis hoc vitium est auri, nec bella fuerunt, Faginus adstabat cum scyphus ante dapes. Non arces, non vallus erat, somnumque petebat Securus sparsas dux gregis inter oves. Tunc mihi vita foret, volgi nec tristia nossem Arma nec audissem corde micante tubam; Nunc ad bella trahor, et iam quis forsitan hostis Haesura in nostro tela gerit latere. Quam potius laudandus hic est, quem prole parata Occupat in parva pigra senecta casa. Ipse suas sectatur oves, at filius agnos, Et calidam fesso conparat uxor aquam. Sic ego sim, liceatque caput candescere canis, Temporis et prisci facta referre senem. Interea pax arva colat. pax candida primum Duxit araturos sub iuga curva boves, Pax aluit vites et sucos condidit uvae, Funderet ut nato testa paterna merum, Pace bidens vomerque nitent, at tristia duri Militis in tenebris occupat arma situs."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.81, 10.838, 0.959)
(0.925, 10.712, 0.984)
(2.978, 0.336, 0.749, 0.251)
0.95
(0.369, 0.162, 0.439)
(0.117, 0.078, 0.6)


In [104]:
sentence = "Quis fuit, horrendos primus qui protulit enses? Quam ferus et vere ferreus ille fuit! Tum caedes hominum generi, tum proelia nata, Tum brevior dirae mortis aperta via est. An nihil ille miser meruit, nos ad mala nostra Vertimus, in saevas quod dedit ille feras? Divitis hoc vitium est auri, nec bella fuerunt, adstabat cum scyphus ante dapes. Non arces, non vallus erat, somnumque petebat Securus sparsas dux gregis inter oves. Tunc mihi vita foret, volgi nec tristia nossem Arma nec audissem corde micante tubam; Nunc ad bella trahor, et iam quis forsitan hostis Haesura in nostro tela gerit latere. Quam potius laudandus hic est, quem prole parata Occupat in parva pigra senecta casa. Ipse suas oves, at filius agnos, Et calidam fesso conparat uxor aquam. Sic ego sim, liceatque caput canis, Temporis et prisci facta referre senem. Interea pax arva colat. pax candida primum Duxit araturos sub iuga curva boves, Pax aluit vites et sucos condidit uvae, Funderet ut nato testa paterna merum, Pace vomerque nitent, at tristia duri Militis in tenebris occupat arma."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.805, 10.613, 0.958)
(0.922, 10.477, 0.983)
(2.867, 0.349, 0.741, 0.259)
0.948
(0.374, 0.155, 0.415)
(0.121, 0.069, 0.636)


In [105]:
sentence = "Sequitur clades, forte an dolo principis incertum (nam utrumque auctores prodidere), sed omnibus, quae huic urbi per violentiam ignium acciderunt, gravior atque atrocior. initium in ea parte circi ortum, quae Palatino Caelioque montibus contigua est, ubi per tabernas, quibus id mercimonium inerat, quo flamma alitur, simul coeptus ignis et statim validus ac vento citus longitudinem circi conripuit. neque enim domus munimentis saeptae vel templa muris cincta aut quid aliud morae interiacebat. impetu pervagatum incendium plana primum, deinde in edita adsurgens et rursus inferiora populando anteiit remedia velocitate mali et obnoxia urbe artis itineribus hucque et illuc flexis atque enormibus vicis, qualis vetus Roma fuit. Eo in tempore Nero Antii agens non ante in urbem regressus est, quam domui eius ignis propinquaret. neque tamen sisti potuit, quin et Palatium et domus et cuncta circum haurirentur. sed solacium populo exturbato ac profugo campum Martis ac monumenta Agrippae, hortos quin etiam suos patefacit et subitaria aedificia exstruxit, quae multitudinem inopem acciperent; subvectaque utensilia ab Ostia et propinquis municipiis. quae quamquam popularia in inritum cadebant, quia pervaserat rumor ipso tempore flagrantis urbis inisse eum domesticam scaenam et cecinisse Troianum excidium, praesentia mala vetustis cladibus adsimulantem. Sexto demum die finis incendio factus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.792, 11.115, 0.956)
(0.914, 10.818, 0.982)
(2.456, 0.407, 0.711, 0.289)
0.947
(0.374, 0.101, 0.27)
(0.076, 0.096, 0.441)


In [106]:
sentence = "Sequitur clades, forte an dolo principis incertum (nam utrumque auctores prodidere), sed omnibus, quae huic urbi per violentiam ignium acciderunt, gravior atque atrocior. initium in ea parte circi ortum, quae montibus est, ubi per tabernas, quibus id inerat, quo flamma alitur, simul coeptus ignis et statim validus ac vento citus longitudinem circi conripuit. neque enim domus munimentis saeptae vel templa muris cincta aut quid aliud morae. impetu incendium plana primum, deinde in edita adsurgens et rursus inferiora populando anteiit velocitate mali et obnoxia urbe artis itineribus hucque et illuc flexis atque, qualis vetus Roma fuit. Eo in tempore Nero agens non ante in urbem regressus est, quam domui eius ignis propinquaret. neque tamen sisti potuit, quin et et domus et cuncta circum haurirentur. sed solacium populo exturbato ac profugo campum Martis ac, hortos quin etiam suos patefacit et aedificia exstruxit, quae multitudinem inopem acciperent; ab Ostia et propinquis municipiis. quae quamquam, quia pervaserat rumor ipso tempore flagrantis urbis inisse eum domesticam scaenam et cecinisse Troianum excidium, praesentia mala vetustis cladibus adsimulantem. Sexto demum die finis incendio factus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.774, 10.298, 0.951)
(0.901, 9.909, 0.978)
(2.161, 0.463, 0.684, 0.316)
0.94
(0.362, 0.107, 0.297)
(0.074, 0.102, 0.419)


In [107]:
sentence = "Nulla est enim natio, quam pertimescamus, nullus rex, qui bellum populo Romano facere possit. Omnia sunt externa unius virtute terra marique pacata; domesticum bellum manet, intus insidiae sunt, intus inclusum periculum est, intus est hostis. Cum luxuria nobis, cum amentia, cum scelere certandum est. Huic ego me bello ducem profiteor, Quirites; suscipio inimicitias hominum perditorum; quae sanari poterunt, quacumque ratione sanabo, quae resecanda erunt, non patiar ad perniciem civitatis manere. Proinde aut exeant aut quiescant aut, si et in urbe et in eadem mente permanent, ea, quae merentur, expectent. At etiam sunt, qui dicant, Quirites, a me eiectum esse Catilinam. Quod ego si verbo assequi possem, istos ipsos eicerem, qui haec loquuntur. Homo enim videlicet timidus aut etiam permodestus vocem consulis ferre non potuit; simul atque ire in exilium iussus est, paruit. Quin hesterno die, cum domi meae paene interfectus essem, senatum in aedem Iovis Statoris convocavi, rem omnem ad patres conscriptos detuli. Quo cum Catilina venisset, quis eum senator appellavit, quis salutavit, quis denique ita aspexit ut perditum civem ac non potius ut inportunissimum hostem? Quin etiam principes eius ordinis partem illam subselliorum, ad quam ille accesserat, nudam atque inanem reliquerunt. Hic ego vehemens ille consul, qui verbo civis in exilium eicio, quaesivi a Catilina, in nocturno conventu apud M. Laecam fuisset necne."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.664, 9.707, 0.924)
(0.87, 9.96, 0.971)
(1.578, 0.634, 0.612, 0.388)
0.937
(0.238, 0.201, 0.843)
(0.145, 0.07, 0.674)


In [108]:
sentence = "Nulla est enim natio, quam pertimescamus, nullus rex, qui bellum populo Romano facere possit. Omnia sunt externa unius virtute terra marique pacata; domesticum bellum manet, intus insidiae sunt, intus inclusum periculum est, intus est hostis. Cum luxuria nobis, cum amentia, cum scelere certandum est. Huic ego me bello ducem profiteor, Quirites; suscipio inimicitias hominum perditorum; quae sanari poterunt, quacumque ratione sanabo, quae resecanda erunt, non patiar ad perniciem civitatis manere. Proinde aut exeant aut quiescant aut, si et in urbe et in eadem mente permanent, ea, quae merentur, expectent. At etiam sunt, qui dicant, Quirites, a me eiectum esse Catilinam. Quod ego si verbo assequi possem, istos ipsos eicerem, qui haec loquuntur. Homo enim timidus aut etiam permodestus vocem consulis ferre non potuit; simul atque ire in exilium iussus est, paruit. Quin hesterno die, cum domi meae paene interfectus essem, senatum in aedem Iovis Statoris convocavi, rem omnem ad patres conscriptos detuli. Quo cum Catilina venisset, quis eum senator appellavit, quis salutavit, quis denique ita aspexit ut perditum civem ac non potius ut inportunissimum hostem? Quin etiam principes eius ordinis partem illam subselliorum, ad quam ille accesserat, nudam atque inanem reliquerunt. Hic ego vehemens ille consul, qui verbo civis in exilium eicio, quaesivi a Catilina, in nocturno conventu apud fuisset necne."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.659, 9.569, 0.922)
(0.867, 9.811, 0.971)
(1.542, 0.648, 0.607, 0.393)
0.935
(0.238, 0.2, 0.84)
(0.148, 0.071, 0.674)


In [109]:
sentence = "Igitur domi militiaeque boni mores colebantur; concordia maxuma, minuma avaritia erat; ius bonumque apud eos non legibus magis quam natura valebat. Iurgia, discordias, simultates cum hostibus exercebant, cives cum civibus de virtute certabant. In suppliciis deorum magnifici, domi parci, in amicos fideles erant. Duabus his artibus, audacia in bello, ubi pax evenerat, aequitate, seque remque publicam curabant. Quarum rerum ego maxuma documenta haec habeo, quod in bello saepius vindicatum est in eos, qui contra imperium in hostem pugnaverant quique tardius revocati proelio excesserant, quam qui signa relinquere aut pulsi loco cedere ausi erant; in pace vero, quod beneficiis magis quam metu imperium agitabant et accepta iniuria ignoscere quam persequi malebant. Sed ubi labore atque iustitia res publica crevit, reges magni bello domiti, nationes ferae et populi ingentes vi subacti, Carthago, aemula imperi Romani, ab stirpe interiit, cuncta maria terraeque patebant, saevire fortuna ac miscere omnia coepit. Qui labores, pericula, dubias atque asperas res facile toleraverant, iis otium divitiaeque optanda alias, oneri miseriaeque fuere. Igitur primo pecuniae, deinde imperi cupido crevit: ea quasi materies omnium malorum fuere. Namque avaritia fidem, probitatem ceterasque artis bonas subvortit; Ambitio multos mortalis falsos fieri subegit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.737, 10.157, 0.942)
(0.833, 10.0, 0.963)
(3.13, 0.319, 0.758, 0.242)
0.963
(0.374, 0.121, 0.324)
(0.105, 0.074, 0.588)


In [110]:
sentence = "Igitur domi militiaeque boni mores colebantur; concordia maxuma, minuma avaritia erat; ius bonumque apud eos non legibus magis quam natura valebat. Iurgia, discordias, simultates cum hostibus exercebant, cives cum civibus de virtute certabant. In deorum magnifici, domi parci, in amicos fideles erant. Duabus his artibus, audacia in bello, ubi pax evenerat, aequitate, seque remque publicam curabant. Quarum rerum ego maxuma documenta haec habeo, quod in bello saepius vindicatum est in eos, qui contra imperium in hostem pugnaverant quique tardius revocati proelio excesserant, quam qui signa relinquere aut pulsi loco cedere ausi erant; in pace vero, quod beneficiis magis quam metu imperium et accepta iniuria ignoscere quam persequi malebant. Sed ubi labore atque iustitia res publica crevit, reges magni bello domiti, nationes ferae et populi ingentes vi subacti, Carthago, imperi Romani, ab stirpe interiit, cuncta maria terraeque patebant, saevire fortuna ac miscere omnia coepit. Qui labores, pericula, dubias atque asperas res facile toleraverant, iis otium divitiaeque oneri miseriaeque fuere. Igitur primo pecuniae, deinde imperi cupido crevit: ea quasi materies omnium malorum fuere. Namque avaritia fidem, ceterasque artis bonas; Ambitio multos mortalis falsos fieri subegit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.727, 9.832, 0.939)
(0.825, 9.654, 0.961)
(2.978, 0.336, 0.749, 0.251)
0.961
(0.366, 0.115, 0.313)
(0.098, 0.071, 0.581)


In [111]:
sentence = "ecquis, io silvae, crudelius inquit amavit? posse putes tangi: minimum est, quod amantibus obstat. quisquis es, huc exi! quid me, puer unice, fallis quove petitus abis? certe nec forma nec aetas est mea, quam fugias, et amarunt me quoque nymphae! spem mihi nescio quam vultu promittis amico, cumque ego porrexi tibi bracchia, porrigis ultro, cum risi, adrides; lacrimas quoque saepe notavi me lacrimante tuas; nutu quoque signa remittis et, quantum motu formosi suspicor oris, verba refers aures non pervenientia nostras! iste ego sum: sensi, nec me mea fallit imago; uror amore mei: flammas moveoque feroque. quid faciam? roger anne rogem? quid deinde rogabo? quod cupio mecum est: inopem me copia fecit. o utinam a nostro secedere corpore possem! votum in amante novum, vellem, quod amamus, abesset. iamque dolor vires adimit, nec tempora vitae longa meae superant, primoque exstinguor in aevo. nec mihi mors gravis est posituro morte dolores, hic, qui diligitur, vellem diuturnior esset; nunc duo concordes anima moriemur in una. Dixit et ad faciem rediit male sanus eandem et lacrimis turbavit aquas, obscuraque moto reddita forma lacu est; quam cum vidisset abire, quo refugis? remane nec me, crudelis, amantem desere! clamavit; liceat, quod tangere non est, adspicere et misero praebere alimenta furori!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.67, 9.545, 0.925)
(0.82, 9.669, 0.96)
(2.172, 0.46, 0.685, 0.315)
0.937
(0.217, 0.281, 1.295)
(0.222, 0.084, 0.726)


In [112]:
sentence = "ecquis, silvae, crudelius inquit amavit? posse putes tangi: minimum est, quod amantibus obstat. quisquis es, huc exi! quid me, puer unice, fallis quove petitus abis? certe nec forma nec aetas est mea, quam fugias, et amarunt me quoque nymphae! spem mihi vultu promittis amico, cumque ego porrexi tibi bracchia, porrigis, cum risi, adrides; lacrimas quoque saepe notavi me lacrimante tuas; nutu quoque signa remittis et, quantum motu formosi suspicor oris, verba refers aures non pervenientia nostras! iste ego sum: sensi, nec me mea fallit imago; uror amore mei: flammas moveoque feroque. quid faciam? roger anne rogem? quid deinde rogabo? quod cupio mecum est: inopem me copia fecit. o utinam a nostro secedere corpore possem! votum amante novum, vellem, quod amamus, abesset. iamque dolor vires adimit, nec tempora vitae longa meae superant, primoque exstinguor in aevo. nec mihi mors gravis est posituro morte dolores, hic, qui diligitur, vellem diuturnior esset; nunc duo concordes anima moriemur in una. Dixit et ad faciem rediit male sanus eandem et lacrimis turbavit aquas, obscuraque moto reddita forma lacu est; quam cum vidisset abire, quo refugis? remane nec me, crudelis, amantem desere! clamavit; liceat, quod tangere non est, adspicere et misero praebere alimenta furori!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.672, 9.452, 0.925)
(0.816, 9.518, 0.959)
(2.194, 0.456, 0.687, 0.313)
0.936
(0.223, 0.284, 1.273)
(0.222, 0.086, 0.721)


In [113]:
sentence = "Haec cum Philosophia dignitate vultus et oris gravitate servata leniter suaviterque cecinisset, tum ego nondum penitus insiti maeroris oblitus intentionem dicere adhuc aliquid parantis abrupi et: o, inquam, veri praevia luminis, quae usque adhuc tua fudit oratio cum sui speculatione divina tum tuis rationibus invicta patuerunt, eaque mihi etsi ob iniuriae dolorem nuper oblita non tamen antehac prorsus ignorata dixisti. Sed ea ipsa est vel maxima nostri causa maeroris quod, cum rerum bonus rector exsistat, vel esse omnino mala possint vel impunita praetereant; quod solum quanta dignum sit admiratione profecto consideras."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.824, 7.862, 0.957)
(0.908, 7.318, 0.977)
(2.5, 0.4, 0.714, 0.286)
0.96
(0.275, 0.165, 0.6)
(0.11, 0.088, 0.556)


In [114]:
sentence = "At huic aliud maius adiungitur; nam imperante florenteque nequitia virtus non solum praemiis caret, uerum etiam sceleratorum pedibus subiecta calcatur et in locum facinorum supplicia luit. Quae fieri in regno scientis omnia, potentis omnia, sed bona tantummodo volentis dei nemo satis potest nec admirari nec conqueri. Tum illa: Et esset, inquit, infiniti stuporis omnibusque horribilius monstris si, uti tu aestimas, in tanti velut patris familias dispositissima domo vilia vasa colerentur, pretiosa sordescerent. Sed non ita est; nam si ea quae paulo ante conclusa sunt inconvulsa servantur, ipso de cuius nunc regno loquimur auctore cognosces semper quidem potentes esse bonos, malos vero abiectos semper atque imbecilles, nec sine poena umquam esse vitia nec sine praemio virtutes, bonis felicia malis semper infortunata contingere multaque id genus, quae sopitis querelis firma te soliditate corroborent."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.794, 9.087, 0.953)
(0.891, 8.549, 0.975)
(2.359, 0.424, 0.702, 0.298)
0.963
(0.212, 0.152, 0.714)
(0.115, 0.122, 0.484)


In [115]:
sentence = "Sub idem tempus Achaia atque Asia falso exterritae velut Nero adventaret, vario super exitu eius rumore eoque pluribus vivere eum fingentibus credentibusque. ceterorum casus conatusque in contextu operis dicemus: tunc servus e Ponto sive, ut alii tradidere, libertinus ex Italia, citharae et cantus peritus, unde illi super similitudinem oris propior ad fallendum fides, adiunctis desertoribus, quos inopia vagos ingentibus promissis corruperat, mare ingreditur; ac vi tempestatum Cythnum insulam detrusus et militum quosdam ex Oriente commeantium adscivit vel abnuentis interfici iussit, et spoliatis negotiatoribus mancipiorum valentissimum quemque armavit. inde late terror: multi ad celebritatem nominis erecti rerum novarum cupidine et odio praesentium."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.901, 9.055, 0.977)
(0.975, 8.721, 0.994)
(3.81, 0.263, 0.792, 0.208)
0.965
(0.382, 0.167, 0.436)
(0.069, 0.158, 0.304)


In [116]:
sentence = "Sub idem tempus Achaia atque Asia falso exterritae velut Nero adventaret, vario super exitu eius rumore eoque pluribus vivere eum fingentibus credentibusque. ceterorum casus conatusque in contextu operis dicemus: tunc servus e sive, ut alii tradidere, libertinus ex Italia, citharae et cantus peritus, unde illi super similitudinem oris propior ad fallendum fides, adiunctis desertoribus, quos inopia vagos ingentibus promissis corruperat, mare ingreditur; ac vi tempestatum insulam et militum quosdam ex Oriente adscivit vel abnuentis interfici iussit, et spoliatis negotiatoribus mancipiorum valentissimum quemque armavit. inde late terror: multi ad nominis rerum novarum cupidine et odio praesentium."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.895, 8.721, 0.976)
(0.973, 8.37, 0.994)
(3.524, 0.284, 0.779, 0.221)
0.962
(0.365, 0.167, 0.457)
(0.074, 0.147, 0.333)


In [117]:
sentence = "Caesar sollicito per vasta silentia gressu vix famulis audenda parat, cunctisque relictis sola placet Fortuna comes. tentoria postquam egressus vigilum somno cedentia membra transsiluit questus tacite, quod fallere posset, litora curva legit, primisque invenit in undis rupibus exesis haerentem fune carinam. rectorem dominumque ratis secura tenebat haud procul inde domus, non ullo robore fulta sed sterili iunco cannaque intexta palustri et latus inversa nudum munita phaselo. haec Caesar bis terque manu quassantia tectum limina commovit. molli consurgit Amyclas quem dabat alga toro. quisnam mea naufragus inquit tecta petit, aut quem nostrae fortuna coegit auxilium sperare casae? sic fatus ab alto aggere iam tepidae sublato fune favillae scintillam tenuem commotos pavit in ignes, securus belli: praedam civilibus armis scit non esse casas. o vitae tuta facultas pauperis angustique lares! o munera nondum intellecta deum! quibus hoc contingere templis aut potuit muris, nullo trepidare tumultu Caesarea pulsante manu?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.89, 10.759, 0.977)
(0.942, 10.364, 0.988)
(4.84, 0.207, 0.829, 0.171)
0.975
(0.37, 0.233, 0.63)
(0.116, 0.123, 0.486)


In [118]:
sentence = "Caesar sollicito per vasta silentia gressu vix famulis audenda parat, cunctisque relictis sola placet Fortuna comes. postquam egressus vigilum somno cedentia membra transsiluit questus tacite, quod fallere posset, litora curva legit, primisque invenit in undis rupibus exesis haerentem fune carinam. rectorem dominumque ratis secura tenebat haud procul inde domus, non ullo robore fulta sed sterili cannaque intexta et latus inversa nudum munita. haec Caesar bis terque manu quassantia tectum limina commovit. molli consurgit Amyclas quem dabat alga toro. quisnam mea naufragus inquit tecta petit, aut quem nostrae fortuna coegit auxilium sperare casae? sic fatus ab alto aggere iam tepidae sublato tenuem commotos pavit in ignes, securus belli: praedam civilibus armis scit non esse casas. o vitae tuta facultas pauperis angustique lares! o munera nondum intellecta deum! quibus hoc contingere templis aut potuit muris, nullo trepidare tumultu Caesarea pulsante manu?"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.892, 10.518, 0.977)
(0.947, 10.115, 0.989)
(4.56, 0.219, 0.82, 0.18)
0.974
(0.36, 0.245, 0.68)
(0.122, 0.129, 0.486)


In [119]:
sentence = "Philosophia iacuit usque ad hanc aetatem nec ullum habuit lumen litterarum Latinarum; quae inlustranda et excitanda nobis est, ut, si occupati profuimus aliquid civibus nostris, prosimus etiam, si possumus, otiosi. in quo eo magis nobis est elaborandum, quod multi iam esse libri Latini dicuntur scripti inconsiderate ab optimis illis quidem viris, sed non satis eruditis. fieri autem potest, ut recte quis sentiat et id quod sentit polite eloqui non possit; sed mandare quemquam litteris cogitationes suas, qui eas nec disponere nec inlustrare possit nec delectatione aliqua allicere lectorem, hominis est intemperanter abutentis et otio et litteris. itaque suos libros ipsi legunt cum suis, nec quisquam attingit praeter eos, qui eandem licentiam scribendi sibi permitti volunt. quare si aliquid oratoriae laudis nostra attulimus industria, multo studiosius philosophiae fontis aperiemus, e quibus etiam illa manabant. Sed ut Aristoteles, vir summo ingenio, scientia, copia, cum motus esset Isocratis rhetoris gloria, dicere docere etiam coepit adulescentes et prudentiam cum eloquentia iungere, sic nobis placet nec pristinum dicendi studium deponere et in hac maiore et uberiore arte versari. hanc enim perfectam philosophiam semper iudicavi, quae de maximis quaestionibus copiose posset ornateque dicere; in quam exercitationem ita nos studiose dedimus, ut iam etiam scholas Graecorum more habere auderemus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.668, 9.499, 0.924)
(0.874, 9.207, 0.971)
(1.22, 0.82, 0.55, 0.45)
0.954
(0.183, 0.223, 1.216)
(0.109, 0.124, 0.468)


In [120]:
sentence = "Philosophia iacuit usque ad hanc aetatem nec ullum habuit litterarum Latinarum; quae inlustranda et excitanda nobis est, ut, si occupati profuimus aliquid civibus nostris, prosimus etiam, si possumus, otiosi. in quo eo magis nobis est elaborandum, quod multi iam esse libri Latini dicuntur scripti inconsiderate ab optimis illis quidem viris, sed non satis eruditis. fieri autem potest, ut recte quis sentiat et id quod sentit eloqui non possit; sed mandare quemquam litteris cogitationes suas, qui eas nec disponere nec inlustrare possit nec delectatione aliqua lectorem, hominis est intemperanter abutentis et otio et litteris. itaque suos libros ipsi legunt cum suis, nec quisquam praeter eos, qui eandem licentiam scribendi sibi permitti volunt. quare si aliquid oratoriae laudis nostra industria, multo studiosius philosophiae fontis aperiemus, e quibus etiam illa manabant. Sed ut Aristoteles, vir summo ingenio, scientia, copia, cum motus esset rhetoris gloria, dicere docere etiam coepit adulescentes et cum eloquentia iungere, sic nobis placet nec pristinum dicendi studium et in hac maiore et uberiore arte versari. hanc enim perfectam philosophiam semper iudicavi, quae de maximis quaestionibus copiose posset dicere; in quam exercitationem ita nos studiose dedimus, ut iam etiam Graecorum more auderemus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.654, 9.045, 0.919)
(0.87, 8.7, 0.97)
(1.099, 0.91, 0.524, 0.476)
0.951
(0.173, 0.215, 1.242)
(0.11, 0.11, 0.5)


In [121]:
sentence = "Persevera ut coepisti et quantum potes propera, quo diutius frui emendato animo et composito possis. Frueris quidem etiam dum emendas, etiam dum componis: alia tamen illa voluptas est quae percipitur ex contemplatione mentis ab omni labe purae et splendidae. Tenes utique memoria quantum senseris gaudium cum praetexta posita sumpsisti virilem togam et in forum deductus es: maius expecta cum puerilem animum deposueris et te in viros philosophia transscripserit. Adhuc enim non pueritia sed, quod est gravius, puerilitas remanet; et hoc quidem peior est, quod auctoritatem habemus senum, vitia puerorum, nec puerorum tantum sed infantum: illi levia, hi falsa formidant, nos utraque. Profice modo: intelleges quaedam ideo minus timenda quia multum metus afferunt. Nullum malum magnum quod extremum est. Mors ad te venit: timenda erat si tecum esse posset: necesse est aut non perveniat aut transeat. Difficile est inquis animum perducere ad contemptionem animae. Non vides quam ex frivolis causis contemnatur? Alius ante amicae fores laqueo pependit, alius se praecipitavit e tecto ne dominum stomachantem diutius audiret, alius ne reduceretur e fuga ferrum adegit in viscera: non putas virtutem hoc effecturam quod efficit nimia formido? Nulli potest secura vita contingere qui de producenda nimis cogitat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.727, 10.123, 0.939)
(0.886, 9.828, 0.975)
(1.732, 0.577, 0.634, 0.366)
0.946
(0.201, 0.211, 1.051)
(0.186, 0.067, 0.735)


In [122]:
sentence = "Persevera ut coepisti et quantum potes propera, quo diutius frui emendato animo et composito possis. Frueris quidem etiam dum emendas, etiam dum componis: alia tamen illa voluptas est quae percipitur ex contemplatione mentis ab omni purae et splendidae. Tenes utique memoria quantum senseris gaudium cum posita sumpsisti virilem togam et in forum deductus es: maius expecta cum puerilem animum deposueris et te in viros philosophia. Adhuc enim non pueritia sed, quod est gravius, remanet; et hoc quidem peior est, quod auctoritatem habemus senum, vitia puerorum, nec puerorum tantum sed infantum: illi levia, hi falsa formidant, nos utraque. Profice modo: intelleges quaedam ideo minus timenda quia multum metus afferunt. Nullum malum magnum quod extremum est. Mors ad te venit: timenda erat si tecum esse posset: necesse est aut non perveniat aut transeat. Difficile est inquis animum perducere ad contemptionem animae. Non vides quam ex causis contemnatur? Alius ante amicae fores laqueo pependit, alius se praecipitavit e tecto ne dominum diutius audiret, alius ne reduceretur e fuga ferrum adegit in viscera: non putas virtutem hoc effecturam quod efficit nimia formido? Nulli potest secura vita contingere qui de nimis cogitat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.717, 9.799, 0.936)
(0.879, 9.47, 0.973)
(1.634, 0.612, 0.62, 0.38)
0.943
(0.188, 0.21, 1.114)
(0.187, 0.059, 0.761)


In [123]:
sentence = "Haec, sicuti exposui, ita gesta sunt, iudices. Insidiator superatus est, vi victa vis, vel potius oppressa virtute audacia est. Nihil dico quid res publica consecuta sit, nihil quid vos, nihil quid omnes boni: nihil sane id prosit Miloni, qui hoc fato natus est, ut ne se quidem servare potuerit, quin una rem publicam vosque servaret. Si id iure fieri non potuit, nihil habeo quod defendam. Sin hoc et ratio doctis, et necessitas barbaris, et mos gentibus, et feris etiam beluis natura ipsa praescripsit, ut omnem semper vim, quacumque ope possent, a corpore, a capite, a vita sua propulsarent, non potestis hoc facinus improbum iudicare, quin simul iudicetis omnibus, qui in latrones inciderint, aut illorum telis aut vestris sententiis esse pereundum. Quod si ita putasset, certe optabilius Miloni fuit dare iugulum P. Clodio, non semel ab illo neque tum primum petitum, quam iugulari a vobis, quia se non iugulandum illi tradidisset. Sin hoc nemo vestrum ita sentit, non illud iam in iudicium venit, occisusne sit (quod fatemur), sed iure an iniuria, quod multis in causis saepe quaesitum est. Insidias factas esse constat, et id est quod senatus contra rem publicam factum iudicavit: ab utro factae sint incertum est."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.645, 9.048, 0.917)
(0.841, 8.937, 0.963)
(1.345, 0.743, 0.574, 0.426)
0.911
(0.194, 0.184, 0.947)
(0.122, 0.086, 0.585)


In [124]:
sentence = "Haec, sicuti exposui, ita gesta sunt, iudices. superatus est, vi victa vis, vel potius oppressa virtute audacia est. Nihil dico quid res publica consecuta sit, nihil quid vos, nihil quid omnes boni: id prosit Miloni, qui hoc fato natus est, ut ne se quidem servare potuerit, quin una rem publicam vosque servaret. Si id iure fieri non potuit, nihil habeo quod defendam. Sin hoc et ratio doctis, et necessitas barbaris, et mos gentibus, et etiam beluis natura ipsa praescripsit, ut omnem semper vim, quacumque ope possent, a corpore, a capite, a vita sua propulsarent, non potestis hoc facinus improbum iudicare, quin simul iudicetis omnibus, qui in latrones inciderint, aut illorum telis aut vestris esse pereundum. Quod si ita putasset, certe optabilius Miloni fuit dare iugulum P. Clodio, non semel ab illo neque tum primum petitum, quam iugulari a vobis, quia se non iugulandum illi tradidisset. Sin hoc nemo vestrum ita sentit, non illud iam in iudicium venit, occisusne sit (quod fatemur), sed iure an iniuria, quod multis in causis saepe quaesitum est. Insidias factas esse constat, et id est quod senatus contra rem publicam factum iudicavit: ab utro factae sint incertum est."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.641, 8.877, 0.915)
(0.843, 8.756, 0.963)
(1.286, 0.778, 0.562, 0.438)
0.909
(0.188, 0.177, 0.944)
(0.125, 0.089, 0.585)


In [125]:
sentence = "Idem nobis per Romanos quoque auctores ordo ducendus est. Itaque ut apud illos Homerus, sic apud nos Vergilius auspicatissimum dederit exordium, omnium eius generis poetarum Graecorum nostrorumque haud dubie proximus. utar enim verbis isdem quae ex Afro Domitio iuvenis excepi, qui mihi interroganti quem Homero crederet maxime accedere secundus inquit est Vergilius, propior tamen primo quam tertio. Et hercule ut illi naturae caelesti atque inmortali cesserimus, ita curae et diligentiae vel ideo in hoc plus est, quod ei fuit magis laborandum, et quantum eminentibus vincimur, fortasse aequalitate pensamus. Ceteri omnes longe sequentur. Nam Macer et Lucretius legendi quidem, sed non ut phrasin, id est corpus eloquentiae, faciant, elegantes in sua quisque materia, sed alter humilis, alter difficilis. Atacinus Varro in iis per quae nomen est adsecutus interpres operis alieni, non spernendus quidem, verum ad augendam facultatem dicendi parum locuples. Ennium sicut sacros vetustate lucos adoremus, in quibus grandia et antiqua robora iam non tantam habent speciem quantam religionem. Propiores alii atque ad hoc de quo loquimur magis utiles. Lascivus quidem in herois quoque Ovidius et nimium amator ingenii sui, laudandus tamen partibus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.758, 10.229, 0.947)
(0.947, 10.066, 0.988)
(1.638, 0.611, 0.621, 0.379)
0.965
(0.236, 0.115, 0.488)
(0.071, 0.077, 0.481)


In [126]:
sentence = "Idem nobis per Romanos quoque auctores ordo ducendus est. Itaque ut apud illos Homerus, sic apud nos Vergilius dederit exordium, omnium eius generis poetarum Graecorum nostrorumque haud dubie proximus. utar enim verbis isdem quae ex iuvenis, qui mihi interroganti quem Homero crederet maxime accedere secundus inquit est Vergilius, propior tamen primo quam tertio. Et ut illi naturae caelesti atque inmortali cesserimus, ita curae et diligentiae vel ideo in hoc plus est, quod ei fuit magis laborandum, et quantum vincimur, fortasse. Ceteri omnes longe sequentur. Nam Macer et Lucretius legendi quidem, sed non ut, id est corpus eloquentiae, faciant, elegantes in sua quisque materia, sed alter humilis, alter difficilis. Atacinus Varro in iis per quae nomen est adsecutus interpres operis alieni, non spernendus quidem, verum ad augendam facultatem dicendi parum locuples. Ennium sicut sacros vetustate lucos adoremus, in quibus grandia et antiqua iam non tantam habent quantam religionem. Propiores alii atque ad hoc de quo loquimur magis utiles. Lascivus quidem in quoque Ovidius et nimium amator ingenii sui, laudandus tamen partibus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.741, 9.664, 0.942)
(0.941, 9.453, 0.987)
(1.464, 0.683, 0.594, 0.406)
0.962
(0.212, 0.106, 0.5)
(0.065, 0.071, 0.478)


In [127]:
sentence = "Ipse tridente suo terram percussit, at illa intremuit motuque vias patefecit aquarum. exspatiata ruunt per apertos flumina campos cumque satis arbusta simul pecudesque virosque tectaque cumque suis rapiunt penetralia sacris. si qua domus mansit potuitque resistere tanto indeiecta malo, culmen tamen altior huius unda tegit, pressaeque latent sub gurgite turres. iamque mare et tellus nullum discrimen habebant: omnia pontus erant, derant quoque litora ponto. Occupat hic collem, cumba sedet alter adunca et ducit remos illic, ubi nuper arabat: ille supra segetes aut mersae culmina villae navigat, hic summa piscem deprendit in ulmo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.902, 8.653, 0.977)
(0.957, 7.945, 0.99)
(3.0, 0.333, 0.75, 0.25)
0.971
(0.293, 0.228, 0.778)
(0.196, 0.065, 0.75)


In [128]:
sentence = "Ipse tridente suo terram percussit, at illa motuque vias patefecit aquarum. ruunt per apertos flumina campos cumque satis arbusta simul pecudesque virosque tectaque cumque suis rapiunt sacris. si qua domus mansit potuitque resistere tanto malo, culmen tamen altior huius unda tegit, pressaeque latent sub gurgite turres. iamque mare et tellus nullum discrimen habebant: omnia pontus erant, derant quoque litora ponto. Occupat hic collem, sedet alter  et ducit remos illic, ubi nuper arabat: ille supra segetes aut mersae culmina villae navigat, hic summa piscem in ulmo."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.894, 8.243, 0.975)
(0.952, 7.493, 0.988)
(2.696, 0.371, 0.729, 0.271)
0.968
(0.329, 0.235, 0.714)
(0.188, 0.059, 0.762)


In [129]:
sentence = "Habet populus Romanus, ad quos gubernacula rei publicae deferat; qui ubicumque terrarum sunt, ibi omne est rei publicae praesidium vel potius ipsa res publica, quae se adhuc tantum modo ulta est, nondum reciperavit. Habet quidem certe res publica adulescentis nobilissimos paratos defensores. Quam volent illi cedant otio consulentes, tamen a re publica revocabuntur. Et nomen pacis dulce est et ipsa res salutaris, sed inter pacem et servitutem plurimum interest. Pax est tranquilla libertas, servitus postremum malorum omnium non modo bello, sed morte etiam repellendum. Quodsi se ipsos illi nostri liberatores e conspectu nostro abstulerunt, at exemplum facti reliquerunt. Illi, quod nemo fecerat, fecerunt. Tarquinium Brutus bello est persecutus, qui tum rex fuit, cum esse Romae licebat; Sp. Cassius, Sp. Maelius, M. Manlius propter suspicionem regni adpetendi sunt necati; hi primum cum gladiis non in regnum adpetentem, sed in regnantem impetum fecerunt. Quod cum ipsum factum per se praeclarum est atque divinum, tum eitum ad imitandum est, praesertim cum illi eam gloriam consecuti sint, quae vix caelo capi posse videatur. Etsi enim satis in ipsa conscientia pulcherrimi facti fructus erat, tamen mortali immortalitatem non arbitror esse contemnendam."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.667, 9.092, 0.922)
(0.818, 9.0, 0.958)
(1.862, 0.537, 0.651, 0.349)
0.906
(0.28, 0.14, 0.5)
(0.091, 0.097, 0.486)


In [130]:
sentence = "quaeque viae tibi causa? quid hac ait arce petisti, progenies, Phaethon, haud infitianda parenti? ille refert: o lux inmensi publica mundi, Phoebe pater, si das usum mihi nominis huius, nec falsa Clymene culpam sub imagine celat, pignora da, genitor, per quae tua vera propago credar, et hunc animis errorem detrahe nostris! dixerat, at genitor circum caput omne micantes deposuit radios propiusque accedere iussit amplexuque dato nec tu meus esse negari dignus es, et Clymene veros ait edidit ortus, quoque minus dubites, quodvis pete munus, ut illud me tribuente feras! promissi testis adesto dis iuranda palus, oculis incognita nostris! vix bene desierat, currus rogat ille paternos inque diem alipedum ius et moderamen equorum. Paenituit iurasse patrem: qui terque quaterque concutiens inlustre caput temeraria dixit vox mea facta tua est; utinam promissa liceret non dare! confiteor, solum hoc tibi, nate, negarem. dissuadere licet: non est tua tuta voluntas! magna petis, Phaethon, et quae nec viribus istis munera conveniant nec tam puerilibus annis: sors tua mortalis, non est mortale, quod optas."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.726, 9.413, 0.938)
(0.816, 9.123, 0.958)
(2.907, 0.344, 0.744, 0.256)
0.964
(0.262, 0.22, 0.841)
(0.162, 0.084, 0.659)


In [131]:
sentence = "quaeque viae tibi causa? quid hac ait arce petisti, Phaethon, haud parenti? ille refert: o lux publica mundi, Phoebe pater, si das usum mihi nominis huius, nec falsa Clymene culpam sub imagine celat, da, genitor, per quae tua vera credar, et hunc animis errorem detrahe nostris! dixerat, at genitor circum caput omne deposuit propiusque accedere iussit dato nec tu meus esse negari dignus es, et Clymene veros ait edidit ortus, quoque minus dubites, quodvis pete munus, ut illud me tribuente feras! promissi testis adesto dis iuranda palus, oculis incognita nostris! vix bene desierat, currus rogat ille paternos inque diem ius et moderamen equorum. Paenituit iurasse patrem: qui terque quaterque concutiens inlustre caput temeraria dixit vox mea facta tua est; utinam promissa liceret non dare! confiteor, solum hoc tibi, nate, negarem. dissuadere licet: non est tua tuta voluntas! magna petis, Phaethon, et quae nec viribus istis munera conveniant nec tam puerilibus annis: sors tua mortalis, non est mortale, quod optas."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.711, 8.961, 0.933)
(0.802, 8.635, 0.954)
(2.698, 0.371, 0.73, 0.27)
0.961
(0.252, 0.22, 0.875)
(0.17, 0.075, 0.692)


In [132]:
sentence = "Germanico bello confecto multis de causis Caesar statuit sibi Rhenum esse transeundum; quarum illa fuit iustissima quod, cum videret Germanos tam facile impelli ut in Galliam venirent, suis quoque rebus eos timere voluit, cum intellegerent et posse et audere populi Romani exercitum Rhenum transire. Accessit etiam quod illa pars equitatus Usipetum et Tencterorum, quam supra commemoravi praedandi frumentandi causa Mosam transisse neque proelio interfuisse, post fugam suorum se trans Rhenum in fines Sugambrorum receperat seque cum his coniunxerat. Ad quos cum Caesar nuntios misisset, qui postularent eos qui sibi Galliaeque bellum intulissent sibi dederent, responderunt: populi Romani imperium Rhenum finire; si se invito Germanos in Galliam transire non aequum existimaret, cur sui quicquam esse imperii aut potestatis trans Rhenum postularet? Ubii autem, qui uni ex Transrhenanis ad Caesarem legatos miserant, amicitiam fecerant, obsides dederant, magnopere orabant ut sibi auxilium ferret, quod graviter ab Suebis premerentur; vel, si id facere occupationibus rei publicae prohiberetur, exercitum modo Rhenum transportaret: id sibi ad auxilium spemque reliqui temporis satis futurum. Tantum esse nomen atque opinionem eius exercitus Ariovisto pulso et hoc novissimo proelio facto etiam ad ultimas Germanorum nationes, uti opinione et amicitia populi Romani tuti esse possint. Navium magnam copiam ad transportandum exercitum pollicebantur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.627, 8.887, 0.912)
(0.748, 8.43, 0.94)
(1.716, 0.583, 0.632, 0.368)
0.951
(0.294, 0.189, 0.644)
(0.124, 0.104, 0.543)


In [133]:
sentence = "Germanico bello confecto multis de causis Caesar statuit sibi Rhenum esse transeundum; quarum illa fuit iustissima quod, cum videret Germanos tam facile impelli ut in Galliam venirent, suis quoque rebus eos timere voluit, cum intellegerent et posse et audere populi Romani exercitum Rhenum transire. Accessit etiam quod illa pars equitatus et, quam supra commemoravi praedandi frumentandi causa transisse neque proelio interfuisse, post fugam suorum se trans Rhenum in fines receperat seque cum his coniunxerat. Ad quos cum Caesar nuntios misisset, qui postularent eos qui sibi Galliaeque bellum intulissent sibi dederent, responderunt: populi Romani imperium Rhenum finire; si se invito Germanos in Galliam transire non aequum existimaret, cur sui quicquam esse imperii aut potestatis trans Rhenum postularet? autem, qui uni ex ad Caesarem legatos miserant, amicitiam fecerant, obsides dederant, magnopere orabant ut sibi auxilium ferret, quod graviter ab premerentur; vel, si id facere occupationibus rei publicae prohiberetur, exercitum modo Rhenum transportaret: id sibi ad auxilium spemque reliqui temporis satis futurum. Tantum esse nomen atque opinionem eius exercitus pulso et hoc novissimo proelio facto etiam ad ultimas Germanorum nationes, uti opinione et amicitia populi Romani tuti esse possint. Navium magnam copiam ad transportandum exercitum pollicebantur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.611, 8.494, 0.907)
(0.731, 7.975, 0.934)
(1.608, 0.622, 0.617, 0.383)
0.948
(0.285, 0.192, 0.673)
(0.13, 0.109, 0.543)


In [134]:
sentence = "quid est obsecro? nam mihi nunc nil rerum omniumst quod malim quam me hoc falso suspicarier. hoc primum, ut ne quid huius rerum ignores: anus, quae est dicta mater esse ei antehac, non fuit; ea obiit mortem. hoc ipsa in itinere alterae dum narrat forte audivi. quaenamst altera? mane: hoc quod coepi primum enarrem, Clitipho: post istuc veniam. propera. iam primum omnium, ubi ventum ad aedis est, Dromo pultat fores; anus quaedam prodit; haec ubi aperuit ostium, continuo hic se coniecit intro, ego consequor; anus foribus obdit pessulum, ad lanam redit. hic sciri potuit aut nusquam alibi, Clinia, quo studio vitam suam te absente exegerit, ubi de inproviso est interventum mulieri. nam ea res dedit tum existumandi copiam cotidianae vitae consuetudinem, quae quoiusque ingenium ut sit declarat maxume. texentem telam studiose ipsam offendimus, mediocriter vestitam veste lugubri (eius anuis causa opinor quae erat mortua) sine auro; tum ornatam ita uti quae ornantur sibi, nulla mala re esse expolitam muliebri; capillus pexus prolixus circum caput reiectus neglegenter; pax."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.725, 9.363, 0.937)
(0.885, 9.021, 0.974)
(1.651, 0.606, 0.623, 0.377)
0.945
(0.246, 0.192, 0.78)
(0.162, 0.072, 0.692)


In [135]:
sentence = "quid est obsecro? nam mihi nunc nil rerum omniumst quod malim quam me hoc falso suspicarier. hoc primum, ut ne quid huius rerum ignores: anus, quae est dicta mater esse ei antehac, non fuit; ea obiit mortem. hoc ipsa in itinere alterae dum narrat forte audivi. quaenamst altera? mane: hoc quod coepi primum enarrem, Clitipho: post istuc veniam. propera. iam primum omnium, ubi ventum ad aedis est, Dromo fores; anus quaedam prodit; haec ubi aperuit ostium, continuo hic se coniecit intro, ego consequor; anus foribus, ad lanam redit. hic sciri potuit aut nusquam alibi, Clinia, quo studio vitam suam te absente exegerit, ubi de inproviso est interventum mulieri. nam ea res dedit tum existumandi copiam cotidianae vitae consuetudinem, quae quoiusque ingenium ut sit declarat maxume. texentem telam studiose ipsam offendimus, mediocriter vestitam veste (eius anuis causa opinor quae erat mortua) sine auro; tum ornatam ita uti quae ornantur sibi, nulla mala re esse muliebri; capillus circum caput reiectus neglegenter."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.711, 8.961, 0.933)
(0.875, 8.573, 0.971)
(1.524, 0.656, 0.604, 0.396)
0.941
(0.233, 0.195, 0.838)
(0.157, 0.063, 0.714)


In [136]:
sentence = "Recordare igitur illum, M. Antoni, diem, quo dictaturam sustulisti; pone ante oculos laetitiam senatus populique Romani, confer cum hac nundinatione tua tuorumque; tum intelleges, quantum inter lucrum et laudem intersit. Sed nimirum, ut quidam morbo aliquo et sensus stupore suavitatem cibi non sentiunt, sic libidinosi, avari, facinerosi verae laudis gustatum non habent. Sed si te laus allicere ad recte faciendum non potest, ne metus quidem a foedissimis factis potest avocare? Iudicia non metuis, si propter innocentiam, laudo, sin propter vim, non intellegis, qui isto modo iudicia non timeat, ei quid timendum sit? Quodsi non metuis viros fortis egregiosque civis, quod a corpore tuo prohibentur armis, tui te, mihi crede, diutius non ferent. Quae est autem vita dies et noctes timere a suis? Nisi vero aut maioribus habes beneficiis obligatos, quam ille quosdam habuit ex iis, a quibus est interfectus, aut tu es ulla re cum eo comparandus. Fuit in illo ingenium, ratio, memoria, litterae, cura, cogitatio, diligentia; res bello gesserat, quamvis rei publicae calamitosas, at tamen magnas; multos annos regnare meditatus, magno labore, magnis periculis, quod cogitarat effecerat; muneribus, monumentis, congiariis, epulis multitudinem imperitam delenierat; suos praemiis, adversarios clementiae specie devinxerat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.733, 10.13, 0.941)
(0.828, 9.369, 0.961)
(2.032, 0.492, 0.67, 0.33)
0.95
(0.293, 0.178, 0.607)
(0.141, 0.073, 0.659)


In [137]:
sentence = "Recordare igitur illum, M. Antoni, diem, quo sustulisti; pone ante oculos laetitiam senatus populique Romani, confer cum hac tua tuorumque; tum intelleges, quantum inter lucrum et laudem intersit. Sed nimirum, ut quidam morbo aliquo et sensus suavitatem cibi non sentiunt, sic libidinosi, avari, verae laudis non habent. Sed si te laus ad recte faciendum non potest, ne metus quidem a foedissimis factis potest avocare? Iudicia non metuis, si propter innocentiam, laudo, sin propter vim, non intellegis, qui isto modo iudicia non timeat, ei quid timendum sit? Quodsi non metuis viros fortis egregiosque civis, quod a corpore tuo prohibentur armis, tui te, mihi crede, diutius non ferent. Quae est autem vita dies et noctes timere a suis? Nisi vero aut maioribus habes beneficiis, quam ille quosdam habuit ex iis, a quibus est interfectus, aut tu es ulla re cum eo comparandus. Fuit in illo ingenium, ratio, memoria, litterae, cura, cogitatio, diligentia; res bello gesserat, quamvis rei publicae calamitosas, at tamen magnas; multos annos regnare meditatus, magno labore, magnis periculis, quod cogitarat effecerat; muneribus, monumentis, epulis multitudinem imperitam; suos praemiis, adversarios clementiae specie devinxerat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.72, 9.71, 0.937)
(0.815, 8.892, 0.957)
(1.889, 0.529, 0.654, 0.346)
0.947
(0.282, 0.171, 0.608)
(0.143, 0.055, 0.722)


In [138]:
sentence = "Angit me Fanniae valetudo. Contraxit hanc, dum assidet Iuniae virgini, sponte primum, est enim affinis, deinde etiam ex auctoritate pontificum. Nam virgines, cum vi morbi atrio Vestae coguntur excedere, matronarum curae custodiaeque mandantur. Quo munere Fannia dum sedulo fungitur, hoc discrimine implicita est. Insident febres, tussis increscit; summa macies, summa defectio; animus tantum et spiritus viget Helvidio marito, Thrasea patre dignissimus; reliqua labuntur meque non metu tantum, verum etiam dolore conficiunt. Doleo enim feminam maximam eripi oculis civitatis, nescio, an aliquid simile visuris. Quae castitas illi, quae sanctitas, quanta gravitas, quanta constantia! Bis maritum secuta in exsilium est, tertio ipsa propter maritum relegata. Eadem quam iucunda, quam comis, quam denique, quod paucis datum est, non minus amabilis quam veneranda! Eritne, quam postea uxoribus nostris ostentare possimus? Erit, a qua viri quoque fortitudinis exempla sumamus, quam sic cernentes audientesque miremur, ut illas, quae leguntur? Ac mihi domus ipsa nutare videtur, licet adhuc posteros habeat. Me quidem illud etiam affligit et torquet, quod matrem eius rursus videor amittere. Quam haec sic auferet secum, meque et novo pariter et rescisso vulnere afficiet. Utramque colui, utramque dilexi. Habuerunt officia mea in secundis, habuerunt in adversis. Non feci tamen paria; atque eo magis hanc cupio servari, ut mihi solvendi tempora supersint. In his eram curis, cum scriberem ad te; quas si deus aliquis in gaudium verterit, de metu non querar. Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.765, 11.508, 0.951)
(0.928, 11.437, 0.985)
(2.054, 0.487, 0.673, 0.327)
0.967
(0.231, 0.17, 0.736)
(0.164, 0.08, 0.673)


In [139]:
sentence = "Angit me Fanniae valetudo. hanc, dum assidet Iuniae virgini, sponte primum est enim affinis, deinde etiam ex auctoritate pontificum. Nam virgines, cum vi morbi atrio Vestae coguntur excedere, matronarum curae custodiaeque mandantur. Quo munere Fannia dum sedulo fungitur, hoc discrimine implicita est. Insident; summa macies, summa; animus tantum et spiritus viget marito, patre dignissimus; reliqua labuntur meque non metu tantum, verum etiam dolore conficiunt. Doleo enim feminam maximam eripi oculis civitatis, nescio, an aliquid simile visuris. Quae castitas illi, quae quanta gravitas, quanta constantia! Bis maritum secuta in exsilium est, tertio ipsa propter maritum. Eadem quam iucunda, quam comis, quam denique, quod paucis datum est, non minus amabilis quam veneranda! Eritne, quam postea uxoribus nostris ostentare possimus? Erit, a qua viri quoque fortitudinis exempla sumamus, quam sic cernentes audientesque miremur, ut illas, quae leguntur? Ac mihi domus ipsa videtur, licet adhuc posteros habeat. Me quidem illud etiam affligit et torquet, quod matrem eius rursus videor amittere. Quam haec sic auferet secum, meque et novo pariter et rescisso vulnere afficiet. Utramque colui, utramque dilexi. Habuerunt officia mea in secundis, habuerunt in adversis. Non feci tamen; atque eo magis hanc cupio servari, ut mihi solvendi tempora supersint. In his eram curis, cum scriberem ad te; quas si deus aliquis in gaudium verterit, de metu non querar. Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.753, 11.048, 0.947)
(0.922, 10.948, 0.984)
(1.905, 0.525, 0.656, 0.344)
0.964
(0.226, 0.18, 0.796)
(0.163, 0.074, 0.686)


In [140]:
sentence = "Ordior a cultu; cultis bene Liber ab uvis provenit, et culto stat seges alta solo. Forma dei munus: forma quota quaeque superbit? Pars vestrum tali munere magna caret. Cura dabit faciem; facies neglecta peribit, Idaliae similis sit licet illa deae. Corpora si veteres non sic coluere puellae, nec veteres cultos sic habuere viros; Si fuit Andromache tunicas induta valentes, quid mirum? Duri militis uxor erat. Scilicet Aiaci, coniunx, ornata venires, cui tegimen septem terga fuere boum? Simplicitas rudis ante fuit: nunc aurea Roma est, et domiti magnas possidet orbis opes. Aspice, quae nunc sunt, Capitolia, quaeque fuerunt: alterius dices illa fuisse Iovis. Curia, concilio quae nunc dignissima tanto, de stipula Tatio regna tenente fuit. Quae nunc sub Phoebo ducibusque Palatia fulgent, quid nisi araturis pascua bubus erant? Prisca iuvent alios, ego me nunc denique natum gratulor: haec aetas moribus apta meis, non quia nunc terrae lentum subducitur aurum lectaque diverso litore concha venit; nec quia decrescunt effosso marmore montes, nec quia caeruleae mole fugantur aquae; sed quia cultus adest nec nostros mansit in annos rusticitas, priscis illa superstes avis. Munditiis capimur: non sint sine lege capilli! Admotae formam dantque negantque manus. Nec genus ornatus unum est: quod quamque decebit, eligat, et speculum consulat ante suum!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.746, 10.686, 0.945)
(0.879, 10.732, 0.974)
(2.661, 0.376, 0.727, 0.273)
0.93
(0.322, 0.156, 0.485)
(0.156, 0.059, 0.727)


In [141]:
sentence = "Ordior a cultu; cultis bene ab uvis provenit, et culto stat seges alta solo. Forma dei munus: forma Pars vestrum tali munere magna caret. Cura dabit faciem; facies neglecta peribit, similis sit licet illa. Corpora si veteres non sic coluere puellae, nec veteres cultos sic habuere viros; Si fuit tunicas induta, quid mirum? Duri militis uxor erat. Scilicet, coniunx, ornata venires, cui tegimen septem terga fuere boum? rudis ante fuit: nunc aurea Roma est, et domiti magnas possidet orbis opes. Aspice, quae nunc sunt, Capitolia, quaeque fuerunt: alterius dices illa fuisse Iovis. Curia, concilio quae nunc dignissima tanto, de regna tenente fuit. Quae nunc sub ducibusque fulgent, quid nisi araturis bubus erant? Prisca iuvent alios, ego me nunc denique natum gratulor: haec aetas moribus apta meis, non quia nunc terrae lentum subducitur aurum lectaque diverso litore concha venit; nec quia effosso marmore montes, nec quia caeruleae mole fugantur aquae; sed quia cultus adest nec nostros mansit in annos priscis illa avis. capimur: non sint sine lege capilli! Admotae formam dantque negantque manus. Nec genus ornatus unum est: quod quamque decebit, eligat, et consulat ante suum!"
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.724, 9.852, 0.938)
(0.862, 9.823, 0.969)
(2.364, 0.423, 0.703, 0.297)
0.92
(0.292, 0.157, 0.537)
(0.151, 0.059, 0.718)


In [142]:
sentence = "Philosophum audio et quidem quintum iam diem habeo, ex quo in scholam eo et ab octava disputantem audio. Bona inquis aetate! Quidni bona? Quid autem stultius est quam, quia diu non didiceris, non discere? Quid ergo? Idem faciam, quod trossuli et iuvenes? Bene mecum agitur, si hoc unum senectutem meam dedecet: Omnis aetatis homines haec schola admittit. In hoc senescamus, ut iuvenes sequamur? In theatrum senex ibo et in circum deferar et nullum par sine me depugnabit ad philosophum ire erubescam? Tamdiu discendum est, quamdiu nescias; si proverbio credimus, quamdiu vivas. Ego tamen illic aliquid et doceo. Quaeris, quid doceam? Etiam seni esse discendum. Pudet autem me generis humani, quotiens scholam intravi. Praeter ipsum theatrum Neapolitanorum, ut scis, transeundum est Metronactis petenti domum. Illud quidem fartum est, et ingenti studio, quis sit pythaules bonus, iudicatur; habet tubicen quoque Graecus concursum. At in illo loco, in quo vir bonus quaeritur, paucissimi sedent, et hi plerisque videntur nihil boni negotii habere, quod agant; inepti et inertes vocantur. Mihi contingat iste derisus: Aequo animo audienda sunt imperitorum convicia et ad honesta vadenti contemnendus est ipse contemptus. Perge, Lucili, et propera, ne tibi accidat, quod mihi, ut senex discas; immo ideo magis propera, quoniam id nunc aggressus es, quod perdiscere vix senex possis. Quantum inquis proficiam? Quantum temptaveris! Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.674, 9.889, 0.927)
(0.797, 9.533, 0.954)
(1.986, 0.503, 0.665, 0.335)
0.942
(0.185, 0.218, 1.175)
(0.201, 0.061, 0.768)


In [143]:
sentence = "Philosophum audio et quidem quintum iam diem habeo, ex quo in scholam eo et ab octava disputantem audio. Bona inquis aetate! bona? Quid autem stultius est quam, quia diu non didiceris, non discere? Quid ergo? Idem faciam, quod et iuvenes? si hoc unum senectutem meam: Omnis aetatis homines haec schola admittit. In hoc ut iuvenes sequamur? In theatrum senex ibo et in circum deferar et nullum par sine me ad philosophum ire? Tamdiu discendum est, quamdiu nescias; si credimus, quamdiu vivas. Ego tamen illic aliquid et doceo. Quaeris, quid doceam? Etiam seni esse discendum. Pudet autem me generis humani, quotiens scholam intravi. Praeter ipsum theatrum, ut scis, transeundum est petenti domum. Illud quidem est, et ingenti studio, quis sit bonus, iudicatur; habet quoque Graecus concursum. At in illo loco, in quo vir bonus quaeritur, paucissimi sedent, et hi plerisque videntur nihil boni negotii habere, quod agant; inepti et inertes vocantur. Mihi contingat iste: Aequo animo audienda sunt imperitorum et ad honesta vadenti contemnendus est ipse. Perge, Lucili, et propera, ne tibi accidat, quod mihi, ut senex discas; immo ideo magis propera, quoniam id nunc aggressus es, quod vix senex possis. Quantum inquis proficiam? Quantum temptaveris! Vale."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.658, 9.214, 0.921)
(0.784, 8.765, 0.95)
(1.761, 0.568, 0.638, 0.362)
0.935
(0.178, 0.198, 1.114)
(0.204, 0.056, 0.784)


In [144]:
sentence = "At me tum primum saevus circumstetit horror. Obstipui; subiit cari genitoris imago, ut regem aequaevum crudeli vulnere vidi vitam exhalantem, subiit deserta Creusa et direpta domus et parvi casus Iuli. Atque ubi iam patriae perventum ad limina sedis antiquasque domos, genitor, quem tollere in altos optabam primum montis primumque petebam, abnegat excisa vitam producere Troia exsiliumque pati. Vos o, quibus integer aevi sanguis, ait, solidaeque suo stant robore vires, vos agitate fugam! Me si caelicolae voluissent ducere vitam, has mihi servassent sedes. Satis una superque vidimus excidia et captae superavimus urbi. Talia perstabat memorans fixusque manebat. Nos contra effusi lacrimis coniunxque Creusa Ascaniusque omnisque domus, ne vertere secum cuncta pater fatoque urgenti incumbere vellet. Abnegat inceptoque et sedibus haeret in isdem. Mene efferre pedem, genitor, te posse relicto sperasti tantumque nefas patrio excidit ore? Si nihil ex tanta superis placet urbe relinqui, et sedet hoc animo perituraeque addere Troiae teque tuosque iuvat, patet isti ianua leto: Arma, viri, ferte arma! Vocat lux ultima victos. Reddite me Danais! Sinite instaurata revisam proelia! Numquam omnes hodie moriemur inulti.  Hinc ferro accingor rursus clipeoque sinistram insertabam aptans meque extra tecta ferebam. Ecce autem complexa pedes in limine coniunx haerebat, parvumque patri tendebat Iulum: Si periturus abis, et nos rape in omnia tecum! Sin aliquam expertus sumptis spem ponis in armis, hanc primum tutare domum! Cui parvus Iulus, cui pater et coniunx quondam tua dicta relinquor? Talia vociferans gemitu tectum omne replebat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.756, 11.668, 0.949)
(0.805, 11.243, 0.959)
(4.535, 0.221, 0.819, 0.181)
0.969
(0.301, 0.234, 0.778)
(0.155, 0.143, 0.521)


In [145]:
sentence = "At me tum primum saevus circumstetit horror. Obstipui; subiit cari genitoris imago, ut regem crudeli vulnere vidi vitam, subiit deserta et direpta domus et parvi casus Iuli. Atque ubi iam patriae perventum ad limina sedis antiquasque domos, genitor, quem tollere in altos optabam primum montis primumque petebam, vitam producere Troia exsiliumque pati. Vos o, quibus integer aevi sanguis, ait, solidaeque suo stant robore vires, vos agitate fugam! Me si voluissent ducere vitam, has mihi servassent sedes. una vidimus excidia et captae urbi. Talia perstabat memorans fixusque manebat. Nos contra effusi lacrimis coniunxque Creusa Ascaniusque omnisque domus, ne vertere secum cuncta pater fatoque urgenti vellet. inceptoque et sedibus haeret in isdem. Mene efferre pedem, genitor, te posse relicto sperasti tantumque nefas patrio excidit ore? Si nihil ex tanta superis placet urbe relinqui, et sedet hoc animo perituraeque addere Troiae teque tuosque iuvat, patet ianua leto: Arma, viri, ferte arma! Vocat lux ultima victos. Reddite me Danais! Sinite instaurata proelia! Numquam omnes hodie moriemur inulti.  Hinc ferro rursus clipeoque sinistram meque extra tecta ferebam. Ecce autem complexa pedes in limine coniunx haerebat, parvumque patri tendebat Iulum: Si periturus abis, et nos rape in omnia tecum! Sin aliquam expertus sumptis spem ponis in armis, hanc primum tutare domum! Cui parvus Iulus, cui pater et coniunx quondam tua dicta relinquor? Talia gemitu tectum omne replebat."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.742, 11.032, 0.945)
(0.792, 10.568, 0.955)
(4.14, 0.242, 0.805, 0.195)
0.966
(0.309, 0.229, 0.739)
(0.15, 0.132, 0.532)


In [146]:
sentence = "Tulit enim Romana regia sceleris tragici exemplum, ut taedio regum maturior veniret libertas: L. Tarquinius fratrem habuerat Arruntem Tarquinium, mitis ingenii iuvenem. His duobus duae Tulliae regis filiae nupserant, et ipsae longe dispares moribus. Forte ita inciderat, ne duo violenta ingenia matrimonio iungerentur. Angebatur ferox Tullia nihil materiae in viro neque ad cupiditatem neque ad audaciam esse; tota in alterum aversa Tarquinium eum mirari, eum virum dicere ac regio sanguine ortum. Contrahit celeriter similitudo eos, ut fere fit: Malum malo aptissimum; sed initium turbandi omnia a femina ortum est. Ea nullis verborum contumeliis parcere de viro ad fratrem, de sorore ad virum. Celeriter adulescentem suae temeritatis implet; L. Tarquinius et Tullia minor prope continuatis funeribus cum domos vacuas novo matrimonio fecissent, iunguntur nuptiis. Iam ab scelere ad aliud spectare mulier scelus. Nec nocte nec interdiu virum conquiescere pati, ne gratuita praeterita parricidia essent: Si tu is es, cui nuptam esse me arbitror, et virum et regem appello. Quin accingeris? Di te penates patriique et patris imago et domus regia et in domo regale solium et nomen Tarquinium creat vocatque regem. Aut si ad haec parum est animi, quid frustraris civitatem? Devolvere retro ad stirpem! His muliebribus instinctus furiis Tarquinius circumire et prensare minorum maxime gentium patres. Postremo, ut iam agendae rei tempus visum est, stipatus agmine armatorum in forum inrupit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.7, 10.383, 0.934)
(0.801, 10.167, 0.956)
(2.729, 0.366, 0.732, 0.268)
0.95
(0.312, 0.118, 0.377)
(0.091, 0.086, 0.513)


In [147]:
sentence = "Tulit enim Romana regia sceleris tragici exemplum, ut taedio regum maturior veniret libertas: L. Tarquinius fratrem habuerat Arruntem Tarquinium, mitis ingenii iuvenem. His duobus duae Tulliae regis filiae nupserant, et ipsae longe dispares moribus. Forte ita inciderat, ne duo violenta ingenia matrimonio iungerentur. Angebatur ferox Tullia nihil in viro neque ad cupiditatem neque ad audaciam esse; tota in alterum aversa eum mirari, eum virum dicere ac regio sanguine ortum. Contrahit celeriter similitudo eos, ut fere fit: Malum malo aptissimum; sed initium turbandi omnia a femina ortum est. Ea nullis verborum contumeliis parcere de viro ad fratrem, de sorore ad virum. Celeriter adulescentem suae temeritatis; L. Tarquinius et Tullia minor prope continuatis funeribus cum domos vacuas novo matrimonio fecissent, iunguntur nuptiis. Iam ab scelere ad aliud spectare mulier scelus. Nec nocte nec interdiu virum conquiescere pati, ne praeterita essent: Si tu is es, cui nuptam esse me arbitror, et virum et regem appello. Quin accingeris? Di te penates patriique et patris imago et domus regia et in domo regale solium et nomen Tarquinium creat vocatque regem. Aut si ad haec parum est animi, quid civitatem? retro ad stirpem! His muliebribus furiis Tarquinius minorum maxime gentium patres. Postremo, ut iam agendae rei tempus visum est, agmine armatorum in forum inrupit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.692, 9.985, 0.931)
(0.793, 9.716, 0.954)
(2.586, 0.387, 0.721, 0.279)
0.954
(0.316, 0.115, 0.364)
(0.091, 0.072, 0.559)


In [7]:
sentence = "quattuor principes ferro interempti: trina bella civilia, plura externa ac plerumque permixta: prosperae in Oriente, adversae in Occidente res: turbatum Illyricum, Galliae nutantes, perdomita Britannia et statim omissa: coortae in nos Sarmatarum ac Sueborum gentes, nobilitatus cladibus mutuis Dacus, mota prope etiam Parthorum arma falsi Neronis ludibrio. iam vero Italia novis cladibus vel post longam saeculorum seriem repetitis adflicta. haustae aut obrutae urbes, fecundissima Campaniae ora; et urbs incendiis vastata, consumptis, antiquissimis delubris, ipso Capitolio civium manibus incenso. pollutae caerimoniae, magna adulteria: plenum exiliimare, infecti caedibus scopuli. atrocius in urbe saevitum: nobilitas, opes, omissi gestique honores pro crimine et ob virtutes certissimum exitium. nec minus praemia delatorum invisa quam scelera, cum alii sacerdotia et consulatus ut spolia adepti, procurationes alii et interiorem potentiam, agerent verterent cuncta odio et terrore. corrupti in dominos servi, in patronos liberti; et quibus deerat inimicus per amicos oppressi. Non tamen adeo virtutum sterile saeculum ut non et bona exempla prodiderit. comitatae profugos liberos matres, secutae maritos in exilia coniuges: propinqui audentes, constantes generi, contumax etiam adversus tormenta servorum fides; supremae clarorum virorum necessitates fortiter toleratae et laudatis antiquorum mortibus pares exitus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.866, 11.805, 0.972)
(0.947, 11.594, 0.989)
(4.167, 0.24, 0.806, 0.194)
0.953
(0.413, 0.106, 0.256)
(0.022, 0.167, 0.114)


In [8]:
sentence = "quattuor principes ferro interempti: trina bella civilia, plura externa ac plerumque permixta: prosperae in Oriente, adversae in Occidente res: turbatum Illyricum, Galliae nutantes, perdomita Britannia et statim omissa: coortae in nos Sarmatarum ac Sueborum gentes, cladibus mutuis, mota prope etiam Parthorum arma falsi Neronis ludibrio. iam vero Italia novis cladibus vel post longam saeculorum seriem repetitis adflicta. haustae aut obrutae urbes, fecundissima Campaniae ora; et urbs incendiis vastata, consumptis, antiquissimis delubris, ipso Capitolio civium manibus incenso. pollutae caerimoniae, magna adulteria: plenum exiliimare, infecti caedibus scopuli. atrocius in urbe saevitum: nobilitas, opes, omissi gestique honores pro crimine et ob virtutes certissimum exitium. nec minus praemia invisa quam scelera, cum alii sacerdotia et consulatus ut adepti, alii et, agerent verterent cuncta odio et terrore. corrupti in dominos servi, in patronos liberti; et quibus deerat inimicus per amicos oppressi. Non tamen adeo virtutum saeculum ut non et bona exempla. profugos liberos matres, secutae maritos in exilia coniuges: propinqui audentes, constantes generi, contumax etiam adversus tormenta servorum fides; clarorum virorum necessitates fortiter toleratae et laudatis antiquorum mortibus pares exitus."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.857, 11.339, 0.97)
(0.942, 11.111, 0.988)
(3.861, 0.259, 0.794, 0.206)
0.95
(0.41, 0.101, 0.247)
(0.017, 0.166, 0.094)


In [9]:
sentence = "Proinde videamus, quos Romanorum mores et quam ob causam Deus verus ad augendum imperium adiuvare dignatus est, in cuius potestate sunt etiam regna terrena. Veteres igitur primique Romani, quantum eorum docet et commendat historia, quamvis ut aliae gentes excepta una populi Hebraeorum deos falsos colerent et non Deo victimas, sed daemoniis immolarent, tamen ‚laudis avidi, pecuiniae liberales erant, gloriam ingentem, divitias honestas volebant‘; hanc ardentissime dilexerunt, propter hanc vivere voluerunt, pro hac emori non dubitaverunt; ceteras cupiditates huius unius ingenti cupiditate presserunt. Ipsam denique patriam suam, quoniam servire videbatur inglorius, dominari vero atque imperare gloriosum, prius omni studio liberam, deinde dominam esse concupiverunt. Hinc est quod regalem dominationem non ferentes ‚annua imperia binosque imperatores sibi fecerunt, qui consules appellati sunt a consulendo, non reges aut domini a regnando atque dominando‘; cum et reges utique a regendo dicti melius videantur, ut regnum a regibus, reges autem, ut dictum est, a regendo; sed fastus regius non disciplina putata est regentis vel benivolentia consulentis, sed superbia dominantis. Expulso itaque rege Tarquinio et consulibus institutis secutum est, quod idem auctor in Romanorum laudibus posuit, quod ‚civitas incredibile memoratu est adepta libertate quantum brevi creverit; tanta cupido gloriae incesserat.‘ Ista ergo laudis aviditas et cupido gloriae multa illa miranda fecit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.712, 10.197, 0.936)
(0.842, 9.924, 0.965)
(2.106, 0.475, 0.678, 0.322)
0.943
(0.219, 0.141, 0.643)
(0.096, 0.135, 0.417)


In [10]:
sentence = "Proinde videamus, quos Romanorum mores et quam ob causam Deus verus ad augendum imperium adiuvare dignatus est, in cuius potestate sunt etiam regna terrena. Veteres igitur primique Romani, quantum eorum docet et commendat historia, quamvis ut aliae gentes excepta una populi Hebraeorum deos falsos colerent et non Deo victimas, sed daemoniis immolarent, tamen ‚laudis avidi, pecuiniae liberales erant, gloriam ingentem, divitias honestas volebant‘; hanc ardentissime dilexerunt, propter hanc vivere voluerunt, pro hac non dubitaverunt; ceteras cupiditates huius unius ingenti cupiditate. Ipsam denique patriam suam, quoniam servire videbatur, dominari vero atque imperare gloriosum, prius omni studio liberam, deinde dominam esse concupiverunt. Hinc est quod regalem dominationem non ferentes ‚annua imperia binosque imperatores sibi fecerunt, qui consules appellati sunt a consulendo, non reges aut domini a regnando atque dominando‘; cum et reges utique a regendo dicti melius videantur, ut regnum a regibus, reges autem, ut dictum est, a regendo; sed regius non putata est regentis vel benivolentia consulentis, sed superbia dominantis. Expulso itaque rege Tarquinio et consulibus institutis secutum est, quod idem auctor in Romanorum laudibus posuit, quod ‚civitas incredibile memoratu est adepta libertate quantum brevi creverit; tanta cupido gloriae incesserat.‘ Ista ergo laudis aviditas et cupido gloriae multa illa miranda fecit."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.705, 9.97, 0.934)
(0.836, 9.675, 0.963)
(2.03, 0.493, 0.67, 0.33)
0.941
(0.219, 0.141, 0.643)
(0.094, 0.133, 0.413)


In [11]:
sentence = "Nullum tamen clementia ex omnibus magis quam regem aut principem decet. Non est hic sine ratione populis urbibusque consensus sic protegendi amandique reges et se suaque iactandi, quocumque desideravit imperantis salus; nec haec vilitas sui est aut dementia pro uno capite tot milia excipere ferrum ac multis mortibus unam animam redimere nonnumquam senis et invalidi. Quemadmodum totum corpus animo deservit et, cum hoc tanto maius tantoque speciosius sit, ille in occulto maneat tenuis et in qua sede latitet incertus, tamen manus, pedes, oculi negotium illi gerunt, illum haec cutis munit, illius iussu iacemus aut inquieti discurrimus, cum ille imperavit, sive avarus dominus est, mare lucri causa scrutamur, sive ambitiosus, iam dudum dextram flammis obiecimus aut voluntarii terram subsiluimus, sic haec immensa multitudo unius animae circumdata illius spiritu regitur, illius ratione flectitur pressura se ac fractura viribus suis, nisi consilio sustineretur. Suam itaque incolumitatem amant, cum pro uno homine denas legiones in aciem deducunt, cum in primam frontem procurrunt et adversa volneribus pectora ferunt, ne imperatoris sui signa vertantur. Ille est enim vinculum, per quod res publica cohaeret, ille spiritus vitalis, quem haec tot milia trahunt nihil ipsa per se futura nisi onus et praeda, si mens illa imperii subtrahatur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.714, 10.066, 0.936)
(0.927, 10.279, 0.984)
(1.618, 0.618, 0.618, 0.382)
0.949
(0.295, 0.105, 0.356)
(0.12, 0.055, 0.686)


In [12]:
sentence = "Nullum tamen clementia ex omnibus magis quam regem aut principem decet. Non est hic sine ratione populis urbibusque consensus sic protegendi amandique reges et se suaque, quocumque desideravit imperantis salus; nec haec sui est aut dementia pro uno capite tot milia excipere ferrum ac multis mortibus unam animam redimere nonnumquam senis et invalidi. Quemadmodum totum corpus animo deservit et, cum hoc tanto maius tantoque speciosius sit, ille in occulto maneat tenuis et in qua sede incertus, tamen manus, pedes, oculi negotium illi gerunt, illum haec munit, illius iussu iacemus aut inquieti discurrimus, cum ille imperavit, sive avarus dominus est, mare lucri causa scrutamur, sive ambitiosus, iam dudum dextram flammis obiecimus aut voluntarii sic haec immensa multitudo unius animae circumdata illius spiritu regitur, illius ratione flectitur se ac fractura viribus suis, nisi consilio sustineretur. Suam itaque incolumitatem amant, cum pro uno homine legiones in aciem deducunt, cum in primam frontem procurrunt et adversa volneribus pectora ferunt, ne imperatoris sui signa vertantur. Ille est enim vinculum, per quod res publica cohaeret, ille, quem haec tot milia nihil ipsa per se futura nisi onus et praeda, si mens illa imperii subtrahatur."
print(basic_TTR(sentence))
print(basic_TTR_cont(sentence))
print(content_word_ratio(sentence))
print(curve_length(sentence))
print(POS(sentence))
print(POS_verb(sentence))

(0.702, 9.627, 0.932)
(0.929, 9.827, 0.984)
(1.474, 0.679, 0.596, 0.404)
0.945
(0.28, 0.095, 0.34)
(0.111, 0.053, 0.677)
